<a href="https://colab.research.google.com/github/HAP2Y/Astro-Finance/blob/main/AstroFinanceProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PHASE 1 - 🔭 Data Acquisition & Alignment

In [1]:
# Cell 1: Financial Data Acquisition (Phase 1 - Part 1 of 3)
# ================================================================

import logging
import sys

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 1: FINANCIAL DATA ACQUISITION")
logger.info("Phase 1 Progress: Part 1 of 3 (Financial Data)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Install Required Libraries
# ============================================================================
logger.info("\n[Installing Libraries]")
logger.info("  → Installing yfinance, tabulate, pyswisseph...")

!pip install -q yfinance tabulate pyswisseph

logger.info("  ✓ All libraries installed successfully")

# ============================================================================
# STEP 2: Import Libraries
# ============================================================================
logger.info("\n[1/4] Importing libraries...")

import yfinance as yf
import pandas as pd
from datetime import datetime
from tabulate import tabulate
import time
import os
from google.colab import drive

logger.info("  ✓ Libraries imported")

# ============================================================================
# STEP 3: Setup Google Drive and Folder Structure
# ============================================================================
logger.info("\n[2/4] Setting up Google Drive and project folders...")

try:
    drive.mount('/content/drive', force_remount=False)
    logger.info("  ✓ Google Drive mounted")

    # Define project paths
    BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
    FINANCIAL_DATA_PATH = os.path.join(BASE_PATH, 'financial_data')
    ASTRO_DATA_PATH = os.path.join(BASE_PATH, 'astro_data')

    # Create directories
    os.makedirs(FINANCIAL_DATA_PATH, exist_ok=True)
    os.makedirs(ASTRO_DATA_PATH, exist_ok=True)

    logger.info(f"  ✓ Project root: {BASE_PATH}")
    logger.info(f"  ✓ Financial data folder: financial_data/")
    logger.info(f"  ✓ Astro data folder: astro_data/")

except Exception as e:
    logger.critical(f"\n✗ FATAL ERROR: Could not mount Google Drive")
    logger.critical(f"  Error: {e}")
    raise SystemExit(1)

# ============================================================================
# STEP 4: Configure Tickers and Download Parameters
# ============================================================================
logger.info("\n[3/4] Configuring market data parameters...")

# Ticker configuration
TICKER_INFO = {
    # Indian Markets (Indices)
    '^NSEI':                {'currency': 'INR', 'volume_unit': 'shares', 'name': 'NIFTY 50'},
    '^NSEBANK':             {'currency': 'INR', 'volume_unit': 'shares', 'name': 'NIFTY BANK'},
    'NIFTY_FIN_SERVICE.NS': {'currency': 'INR', 'volume_unit': 'shares', 'name': 'NIFTY FIN SERVICES'},
    '^CNXIT':               {'currency': 'INR', 'volume_unit': 'shares', 'name': 'NIFTY IT'},
    '^CNXPHARMA':           {'currency': 'INR', 'volume_unit': 'shares', 'name': 'NIFTY PHARMA'},
    '^CNXAUTO':             {'currency': 'INR', 'volume_unit': 'shares', 'name': 'NIFTY AUTO'},
    '^CNXMETAL':            {'currency': 'INR', 'volume_unit': 'shares', 'name': 'NIFTY METAL'},
    '^CNXFMCG':             {'currency': 'INR', 'volume_unit': 'shares', 'name': 'NIFTY FMCG'},
    '^INDIAVIX':            {'currency': 'INR', 'volume_unit': 'points', 'name': 'INDIA VIX'},

    # Indian Markets (Key Stocks)
    'RELIANCE.NS':          {'currency': 'INR', 'volume_unit': 'shares', 'name': 'Reliance Industries'},
    'TCS.NS':               {'currency': 'INR', 'volume_unit': 'shares', 'name': 'TCS'},
    'HDFCBANK.NS':          {'currency': 'INR', 'volume_unit': 'shares', 'name': 'HDFC Bank'},

    # US Markets (Indices)
    '^GSPC':                {'currency': 'USD', 'volume_unit': 'points', 'name': 'S&P 500'},
    '^DJI':                 {'currency': 'USD', 'volume_unit': 'points', 'name': 'Dow Jones'},
    '^NDX':                 {'currency': 'USD', 'volume_unit': 'points', 'name': 'NASDAQ 100'},
    '^RUT':                 {'currency': 'USD', 'volume_unit': 'points', 'name': 'Russell 2000'},
    '^VIX':                 {'currency': 'USD', 'volume_unit': 'points', 'name': 'VIX'},
    '^TNX':                 {'currency': 'USD', 'volume_unit': 'points', 'name': '10Y Treasury'},

    # US Markets (Key Stocks)
    'AAPL':                 {'currency': 'USD', 'volume_unit': 'shares', 'name': 'Apple'},
    'MSFT':                 {'currency': 'USD', 'volume_unit': 'shares', 'name': 'Microsoft'},
    'NVDA':                 {'currency': 'USD', 'volume_unit': 'shares', 'name': 'NVIDIA'},

    # Global Markets (Indices)
    '^N225':                {'currency': 'JPY', 'volume_unit': 'points', 'name': 'Nikkei 225'},
    '^FTSE':                {'currency': 'GBP', 'volume_unit': 'points', 'name': 'FTSE 100'},
    '^GDAXI':               {'currency': 'EUR', 'volume_unit': 'points', 'name': 'DAX'},
    '000001.SS':            {'currency': 'CNY', 'volume_unit': 'shares', 'name': 'SSE Composite'},
    '^HSI':                 {'currency': 'HKD', 'volume_unit': 'points', 'name': 'Hang Seng'},

    # Commodities
    'GC=F':                 {'currency': 'USD', 'volume_unit': 'contracts', 'name': 'Gold'},
    'CL=F':                 {'currency': 'USD', 'volume_unit': 'contracts', 'name': 'Crude Oil'},
    'SI=F':                 {'currency': 'USD', 'volume_unit': 'contracts', 'name': 'Silver'},

    # Currencies & DXY
    'DX-Y.NYB':             {'currency': 'USD', 'volume_unit': 'points', 'name': 'US Dollar Index'},
    'USDINR=X':             {'currency': 'INR', 'volume_unit': 'rate', 'name': 'USD/INR'},
    'EURUSD=X':             {'currency': 'USD', 'volume_unit': 'rate', 'name': 'EUR/USD'},
}

TICKERS = list(TICKER_INFO.keys())
START_DATE = '2000-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')
WAIT_TIME_SECONDS = 1

logger.info(f"  ✓ Configured {len(TICKERS)} tickers")
logger.info(f"  ✓ Date range: {START_DATE} to {END_DATE}")
logger.info(f"  ✓ Rate limit: {WAIT_TIME_SECONDS}s between requests")

# ============================================================================
# STEP 5: Download Financial Data
# ============================================================================
logger.info("\n[4/4] Downloading financial data...")
logger.info("  (This may take several minutes)")
logger.info("")

financial_results_summary = []
success_count = 0
failed_count = 0
skipped_count = 0

for i, ticker in enumerate(TICKERS):
    safe_ticker_name = ticker.replace('^', '').replace('=X', '').replace('=F', '').replace('-','_').replace('.','_')
    ticker_display_name = TICKER_INFO[ticker]['name']

    logger.info(f"  [{i+1}/{len(TICKERS)}] {ticker_display_name} ({ticker})...", end=" ")

    try:
        filename = os.path.join(FINANCIAL_DATA_PATH, f"financial_data_{safe_ticker_name}.parquet")

        # Download data (auto_adjust=False to suppress FutureWarning)
        data = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, auto_adjust=False)

        if data.empty:
            message = "No data available"
            logger.warning(f"⚠ SKIPPED ({message})")
            financial_results_summary.append({
                'Ticker': ticker,
                'Name': ticker_display_name,
                'Status': 'Skipped',
                'Details': message
            })
            skipped_count += 1
            continue

        # Flatten MultiIndex columns if present (yfinance creates MultiIndex for single tickers)
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        # Add metadata
        data['currency'] = TICKER_INFO[ticker]['currency']
        data['volume_unit'] = TICKER_INFO[ticker]['volume_unit']

        # Standardize columns
        data.reset_index(inplace=True)
        data.rename(columns={
            'Date': 'date',
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Adj Close': 'adj_close',
            'Volume': 'volume'
        }, inplace=True)

        data['date'] = pd.to_datetime(data['date']).dt.date

        # Reorder columns
        column_order = ['date', 'open', 'high', 'low', 'close', 'volume',
                       'currency', 'volume_unit', 'adj_close']
        final_columns = [col for col in column_order if col in data.columns]
        data = data[final_columns]

        # Save to parquet
        data.to_parquet(filename, index=False)

        # Summary info
        first_date = data['date'].min()
        last_date = data['date'].max()
        row_count = len(data)

        message = f"{row_count} rows | {first_date} to {last_date}"
        logger.info(f"✓ ({message})")

        financial_results_summary.append({
            'Ticker': ticker,
            'Name': ticker_display_name,
            'Status': 'Success',
            'Details': message
        })
        success_count += 1

    except Exception as e:
        message = str(e)[:60]
        logger.error(f"✗ FAILED ({message})")
        financial_results_summary.append({
            'Ticker': ticker,
            'Name': ticker_display_name,
            'Status': 'Failed',
            'Details': message
        })
        failed_count += 1

    finally:
        time.sleep(WAIT_TIME_SECONDS)

# ============================================================================
# FINAL SUMMARY
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("DOWNLOAD SUMMARY")
logger.info("=" * 70)

logger.info(f"\n  ✓ Successful: {success_count}")
logger.warning(f"  ⚠ Skipped: {skipped_count}")
logger.error(f"  ✗ Failed: {failed_count}")
logger.info(f"  Total: {len(TICKERS)}")

logger.info("\n" + "=" * 70)
logger.info("DETAILED RESULTS")
logger.info("=" * 70)

summary_df = pd.DataFrame(financial_results_summary)
print("\n" + tabulate(summary_df, headers='keys', tablefmt='grid', showindex=False))

logger.info("\n" + "=" * 70)
logger.info("PHASE 1 (FINANCIAL DATA) - STATUS: COMPLETE ✓")
logger.info("=" * 70)

logger.info("\n📋 Next Steps:")
logger.info("  1. ▶ Run Cell 2: Generate Vedic ephemeris data")
logger.info("  2. ▶ Run Cell 3: Align financial + astro data by date")
logger.info("  3. Then proceed to Phase 2: Feature Engineering")

logger.info(f"\n📂 Output Location:")
logger.info(f"  {FINANCIAL_DATA_PATH}")
logger.info(f"  ({success_count} parquet files saved)")

logger.info("\n" + "=" * 70)

SyntaxError: unterminated string literal (detected at line 45) (ipython-input-2994976661.py, line 45)

In [ ]:
# Cell 2: Vedic Astrological Data Generation (Phase 1 - Part 2 of 3)
# ================================================================

import logging
import sys

import os
import pandas as pd
import swisseph as swe
from datetime import datetime, timedelta
from tabulate import tabulate
import requests

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 1: VEDIC EPHEMERIS GENERATION")
logger.info("Phase 1 Progress: Part 2 of 3 (Astrological Data)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup Paths
# ============================================================================
logger.info("\n[1/6] Setting up directories...")

# Ephemeris files (local runtime)
EPHE_DIR = "/content/ephe_data"
os.makedirs(EPHE_DIR, exist_ok=True)
logger.info(f"  ✓ Ephemeris directory: {EPHE_DIR}")

# Output directory (Google Drive)
BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
ASTRO_DATA_PATH = os.path.join(BASE_PATH, 'astro_data')
os.makedirs(ASTRO_DATA_PATH, exist_ok=True)
logger.info(f"  ✓ Output directory: {ASTRO_DATA_PATH}")

# ============================================================================
# STEP 2: Download Ephemeris Files from VERIFIED WORKING SOURCES
# ============================================================================
logger.info("\n[2/6] Downloading ephemeris files...")
logger.info("  Source: GitHub aloistr/swisseph (official Swiss Ephemeris)")

# The correct GitHub repository is aloistr/swisseph
GITHUB_BASE = "https://raw.githubusercontent.com/aloistr/swisseph/master/ephe/"

# Files needed for 2000-2025 (these cover 1800-2399 CE)
EPHE_FILES = {
    "semo_18.se1": "Moon ephemeris 1800-2399",
    "sepl_18.se1": "Planets ephemeris 1800-2399",
}

download_success = True
downloaded_files = []

for filename, description in EPHE_FILES.items():
    file_path = os.path.join(EPHE_DIR, filename)
    url = GITHUB_BASE + filename

    try:
        logger.info(f"  → {filename} ({description})...", end=" ")
        response = requests.get(url, timeout=60, allow_redirects=True)

        if response.status_code == 200 and len(response.content) > 1000:
            with open(file_path, 'wb') as f:
                f.write(response.content)

            # Verify file was written correctly
            if os.path.exists(file_path) and os.path.getsize(file_path) > 1000:
                size_kb = os.path.getsize(file_path) / 1024
                logger.info(f"✓ ({size_kb:.1f} KB)")
                downloaded_files.append(filename)
            else:
                logger.error("✗ FAILED (file too small)")
                download_success = False
        else:
            logger.error(f"✗ FAILED (HTTP {response.status_code})")
            download_success = False

    except Exception as e:
        logger.error(f"✗ FAILED ({str(e)[:60]})")
        download_success = False

# ============================================================================
# BACKUP: Try Dropbox if GitHub fails
# ============================================================================
if not download_success or len(downloaded_files) < 2:
    logger.info("\n  GitHub download incomplete. Trying backup source...")
    logger.info("  Source: Dropbox (Alois Treindl's public folder)")

    # Dropbox direct download links
    DROPBOX_FILES = {
        "semo_18.se1": "https://www.dropbox.com/scl/fo/y3naz62gy6f6qfrhquu7u/h/semo_18.se1?rlkey=ejltdhb262zglm7eo6yfj2940&dl=1",
        "sepl_18.se1": "https://www.dropbox.com/scl/fo/y3naz62gy6f6qfrhquu7u/h/sepl_18.se1?rlkey=ejltdhb262zglm7eo6yfj2940&dl=1",
    }

    downloaded_files = []  # Reset
    download_success = True

    for filename, url in DROPBOX_FILES.items():
        file_path = os.path.join(EPHE_DIR, filename)

        try:
            logger.info(f"  → {filename}...", end=" ")
            response = requests.get(url, timeout=60, allow_redirects=True)

            if response.status_code == 200 and len(response.content) > 1000:
                with open(file_path, 'wb') as f:
                    f.write(response.content)

                if os.path.exists(file_path) and os.path.getsize(file_path) > 1000:
                    size_kb = os.path.getsize(file_path) / 1024
                    logger.info(f"✓ ({size_kb:.1f} KB)")
                    downloaded_files.append(filename)
                else:
                    logger.error("✗ FAILED")
                    download_success = False
            else:
                logger.error(f"✗ FAILED (HTTP {response.status_code})")
                download_success = False

        except Exception as e:
            logger.error(f"✗ FAILED ({str(e)[:60]})")
            download_success = False

# ============================================================================
# STEP 3: Fatal Error Check
# ============================================================================
if not download_success or len(downloaded_files) < 2:
    logger.info("\n" + "!" * 70)
    logger.critical("FATAL ERROR: Ephemeris file download failed from all sources!")
    logger.info("!" * 70)
    logger.info("\nDiagnostics:")
    logger.info(f"  - Required files: 2")
    logger.info(f"  - Successfully downloaded: {len(downloaded_files)}")
    logger.info(f"  - Files in directory: {os.listdir(EPHE_DIR)}")

    logger.info("\n🔧 MANUAL WORKAROUND - Run these commands in a NEW cell:")
    logger.info("```python")
    logger.info("# Method 1: Direct wget from GitHub")
    logger.info("!wget -P /content/ephe_data https://raw.githubusercontent.com/aloistr/swisseph/master/ephe/semo_18.se1")
    logger.info("!wget -P /content/ephe_data https://raw.githubusercontent.com/aloistr/swisseph/master/ephe/sepl_18.se1")
    logger.info("")
    logger.info("# Method 2: If GitHub blocked, use curl from Dropbox")
    logger.info("!curl -L -o /content/ephe_data/semo_18.se1 'https://www.dropbox.com/scl/fo/y3naz62gy6f6qfrhquu7u/h/semo_18.se1?rlkey=ejltdhb262zglm7eo6yfj2940&dl=1'")
    logger.info("!curl -L -o /content/ephe_data/sepl_18.se1 'https://www.dropbox.com/scl/fo/y3naz62gy6f6qfrhquu7u/h/sepl_18.se1?rlkey=ejltdhb262zglm7eo6yfj2940&dl=1'")
    logger.info("```")
    logger.info("\nThen re-run this cell.")

    raise SystemExit(1)

logger.info(f"\n  ✓ Successfully downloaded {len(downloaded_files)} ephemeris files")

# ============================================================================
# STEP 4: Configure PySwisseph for Vedic Calculations
# ============================================================================
logger.info("\n[3/6] Configuring PySwisseph for Vedic (Lahiri) mode...")

swe.set_ephe_path(EPHE_DIR)
swe.set_sid_mode(swe.SIDM_LAHIRI)

logger.info(f"  ✓ Ephemeris path: {EPHE_DIR}")
logger.info(f"  ✓ Ayanamsha: Lahiri (Vedic)")

# Quick test to ensure files are working
try:
    test_jd = swe.julday(2000, 1, 1, 12.0)
    test_pos, _ = swe.calc_ut(test_jd, swe.SUN, swe.FLG_SIDEREAL)
    logger.info(f"  ✓ Verification: Sun position on 2000-01-01 = {test_pos[0]:.2f}°")
except Exception as e:
    logger.error(f"  ✗ WARNING: Test calculation failed: {e}")
    raise SystemExit(1)

# ============================================================================
# STEP 5: Generate Full Ephemeris Data (2000-01-01 to 2025-10-29)
# ============================================================================
logger.info("\n[4/6] Calculating planetary positions for full date range...")
logger.info("  Date range: 2000-01-01 to 2025-10-29")
logger.info("  (This will take several minutes - ~9,400 days to calculate)")

start_date = datetime(2000, 1, 1)
end_date = datetime(2025, 10, 29)
total_days = (end_date - start_date).days + 1

logger.info(f"  Total days: {total_days}")

PLANETS = {
    'Sun': swe.SUN,
    'Moon': swe.MOON,
    'Mercury': swe.MERCURY,
    'Venus': swe.VENUS,
    'Mars': swe.MARS,
    'Jupiter': swe.JUPITER,
    'Saturn': swe.SATURN,
    'Rahu': swe.MEAN_NODE,
}

ephemeris_data = []
calculation_warnings = []

# Progress tracking
current_date = start_date
days_processed = 0
progress_interval = 500  # Print progress every 500 days

logger.info("\n  Progress:")

while current_date <= end_date:
    jd = swe.julday(current_date.year, current_date.month, current_date.day, 12.0)

    day_data = {
        'date': current_date.strftime('%Y-%m-%d'),
        'julian_day': jd
    }

    for planet_name, planet_id in PLANETS.items():
        try:
            position, ret_flag = swe.calc_ut(jd, planet_id, swe.FLG_SIDEREAL | swe.FLG_SPEED)

            if position is not None and len(position) >= 4:
                day_data[f'{planet_name.lower()}_longitude'] = round(position[0], 6)
                day_data[f'{planet_name.lower()}_speed'] = round(position[3], 6)
            else:
                day_data[f'{planet_name.lower()}_longitude'] = None
                day_data[f'{planet_name.lower()}_speed'] = None
                calculation_warnings.append(
                    f"{planet_name} on {current_date.strftime('%Y-%m-%d')}: returned None"
                )
        except Exception as e:
            day_data[f'{planet_name.lower()}_longitude'] = None
            day_data[f'{planet_name.lower()}_speed'] = None
            calculation_warnings.append(
                f"{planet_name} on {current_date.strftime('%Y-%m-%d')}: {str(e)}"
            )

    ephemeris_data.append(day_data)
    current_date += timedelta(days=1)
    days_processed += 1

    # Progress indicator
    if days_processed % progress_interval == 0 or days_processed == total_days:
        progress_pct = (days_processed / total_days) * 100
        logger.info(f"    [{days_processed}/{total_days}] {progress_pct:.1f}% complete")

df_ephemeris = pd.DataFrame(ephemeris_data)

logger.info(f"\n  ✓ Calculated {len(df_ephemeris)} days")
logger.info(f"  ✓ DataFrame shape: {df_ephemeris.shape}")
logger.info(f"  ✓ Columns: {len(df_ephemeris.columns)}")

# ============================================================================
# STEP 6: Validation & Quality Check
# ============================================================================
logger.info("\n[5/6] Validating data quality...")

if calculation_warnings:
    logger.warning(f"\n  ⚠ CALCULATION WARNINGS: {len(calculation_warnings)} warnings detected")
    for warning in calculation_warnings[:5]:
        logger.info(f"    • {warning}")
    if len(calculation_warnings) > 5:
        logger.info(f"    • ... and {len(calculation_warnings) - 5} more warnings")

null_counts = df_ephemeris.isnull().sum()
total_nulls = null_counts.sum()

if total_nulls > 0:
    logger.warning(f"\n  ⚠ NULL VALUES: {total_nulls} cells contain null values")
    logger.info("    Columns with nulls:")
    for col, count in null_counts[null_counts > 0].items():
        logger.info(f"      • {col}: {count}")
else:
    logger.info("\n  ✓ DATA QUALITY: No null values detected")

# Sanity checks on the data
logger.info("\n  Data Validation:")
sun_longs = df_ephemeris['sun_longitude'].dropna()
if len(sun_longs) > 0:
    logger.info(f"    ✓ Sun longitude range: {sun_longs.min():.2f}° to {sun_longs.max():.2f}°")
    if sun_longs.min() >= 0 and sun_longs.max() <= 360:
        logger.info("    ✓ All longitudes within valid range (0-360°)")
    else:
        logger.warning("    ⚠ WARNING: Some longitudes outside valid range!")

moon_speeds = df_ephemeris['moon_speed'].dropna()
if len(moon_speeds) > 0:
    logger.info(f"    ✓ Moon speed range: {moon_speeds.min():.4f}° to {moon_speeds.max():.4f}° per day")
    if 11 < moon_speeds.mean() < 15:
        logger.info(f"    ✓ Moon speed looks reasonable (avg: {moon_speeds.mean():.2f}°/day)")

# ============================================================================
# STEP 7: Save Results to Google Drive
# ============================================================================
logger.info("\n[6/6] Saving ephemeris data to Google Drive...")

OUTPUT_FILE = os.path.join(ASTRO_DATA_PATH, 'vedic_ephemeris_2000_2025.parquet')
df_ephemeris.to_parquet(OUTPUT_FILE, index=False, engine='pyarrow')

file_size_kb = os.path.getsize(OUTPUT_FILE) / 1024
logger.info(f"  ✓ Saved: vedic_ephemeris_2000_2025.parquet")
logger.info(f"  ✓ Size: {file_size_kb:.1f} KB")

# ============================================================================
# STEP 8: Display Sample Results
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("SAMPLE RESULTS: PLANETARY LONGITUDES (First 3 Days)")
logger.info("=" * 70)

sample_cols = ['date', 'sun_longitude', 'moon_longitude', 'mercury_longitude',
               'venus_longitude', 'mars_longitude', 'jupiter_longitude',
               'saturn_longitude', 'rahu_longitude']

sample_df = df_ephemeris[sample_cols].head(3)
print("\n" + tabulate(sample_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))

logger.info("\n" + "=" * 70)
logger.info("SAMPLE RESULTS: PLANETARY SPEEDS (First 3 Days)")
logger.info("=" * 70)

speed_cols = ['date', 'sun_speed', 'moon_speed', 'mercury_speed',
              'venus_speed', 'mars_speed', 'jupiter_speed',
              'saturn_speed', 'rahu_speed']

speed_df = df_ephemeris[speed_cols].head(3)
print("\n" + tabulate(speed_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".4f"))

# ============================================================================
# FINAL STATUS
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("PHASE 1 (VEDIC EPHEMERIS) - STATUS: COMPLETE ✓")
logger.info("=" * 70)

logger.info("\n📋 Next Steps:")
logger.info("  1. ✓ Financial data acquired (Cell 1 complete)")
logger.info("  2. ✓ Vedic ephemeris generated (Cell 2 complete)")
logger.info("  3. ▶ Run Cell 3: Align & merge datasets by date")
logger.info("  4. Then proceed to Phase 2: Feature Engineering")

logger.info("\n📂 Output Files:")
logger.info(f"  • Ephemeris files: {EPHE_DIR}")
logger.info(f"  • Vedic data: {ASTRO_DATA_PATH}")
logger.info(f"  • File: vedic_ephemeris_2000_2025.parquet ({len(df_ephemeris)} rows)")

logger.info("\n" + "=" * 70)

In [ ]:
# Cell 3: Data Alignment & Merging (Phase 1 - Part 3 of 3)
# ================================================================

import logging
import sys

import os
import pandas as pd
from datetime import datetime
from tabulate import tabulate
import glob

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 1: DATA ALIGNMENT & MERGING")
logger.info("Phase 1 Progress: Part 3 of 3 (Dataset Integration)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup Paths
# ============================================================================
logger.info("\n[1/5] Setting up paths...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
FINANCIAL_DATA_PATH = os.path.join(BASE_PATH, 'financial_data')
ASTRO_DATA_PATH = os.path.join(BASE_PATH, 'astro_data')
ALIGNED_DATA_PATH = os.path.join(BASE_PATH, 'aligned_data')

# Create aligned data directory
os.makedirs(ALIGNED_DATA_PATH, exist_ok=True)

logger.info(f"  ✓ Financial data: {FINANCIAL_DATA_PATH}")
logger.info(f"  ✓ Astro data: {ASTRO_DATA_PATH}")
logger.info(f"  ✓ Output directory: {ALIGNED_DATA_PATH}")

# ============================================================================
# STEP 2: Load Vedic Ephemeris Data
# ============================================================================
logger.info("\n[2/5] Loading Vedic ephemeris data...")

EPHEMERIS_FILE = os.path.join(ASTRO_DATA_PATH, 'vedic_ephemeris_2000_2025.parquet')

try:
    df_ephemeris = pd.read_parquet(EPHEMERIS_FILE)
    df_ephemeris['date'] = pd.to_datetime(df_ephemeris['date']).dt.date

    logger.info(f"  ✓ Loaded ephemeris data")
    logger.info(f"  ✓ Shape: {df_ephemeris.shape}")
    logger.info(f"  ✓ Date range: {df_ephemeris['date'].min()} to {df_ephemeris['date'].max()}")
    logger.info(f"  ✓ Columns: {len(df_ephemeris.columns)}")

except Exception as e:
    logger.critical(f"\n✗ FATAL ERROR: Could not load ephemeris data")
    logger.critical(f"  Error: {e}")
    logger.info(f"\n  Make sure Cell 2 has been run successfully!")
    raise SystemExit(1)

# ============================================================================
# STEP 3: Load and Align Financial Data
# ============================================================================
logger.info("\n[3/5] Loading and aligning financial data with ephemeris...")

# Find all financial data parquet files
financial_files = glob.glob(os.path.join(FINANCIAL_DATA_PATH, 'financial_data_*.parquet'))

if len(financial_files) == 0:
    logger.critical(f"\n✗ FATAL ERROR: No financial data files found")
    logger.info(f"  Make sure Cell 1 has been run successfully!")
    raise SystemExit(1)

logger.info(f"\n  Found {len(financial_files)} financial data files")

# Debug: Check first file to see its structure
logger.info("\n  Debugging first file structure...")
first_file = financial_files[0]
df_test = pd.read_parquet(first_file)
logger.info(f"  • Columns: {list(df_test.columns)}")
logger.info(f"  • Date column type: {df_test['date'].dtype if 'date' in df_test.columns else 'Column not found!'}")
logger.info(f"  • Sample dates: {df_test['date'].head(3).tolist() if 'date' in df_test.columns else 'N/A'}")
logger.info(f"  • Shape: {df_test.shape}")

logger.info("\n  Processing all tickers...")

alignment_results = []
success_count = 0
failed_count = 0

for i, file_path in enumerate(financial_files):
    # Extract ticker name from filename
    filename = os.path.basename(file_path)
    ticker_name = filename.replace('financial_data_', '').replace('.parquet', '')

    logger.info(f"\n  [{i+1}/{len(financial_files)}] Processing {ticker_name}...", end=" ")

    try:
        # Load financial data
        df_financial = pd.read_parquet(file_path)

        # Ensure date column exists and convert to date type
        if 'date' not in df_financial.columns:
            raise ValueError("'date' column not found in financial data")

        # Convert date to datetime.date for consistent merging
        df_financial['date'] = pd.to_datetime(df_financial['date']).dt.date

        original_rows = len(df_financial)

        # Merge with ephemeris data on date
        df_aligned = pd.merge(
            df_financial,
            df_ephemeris,
            on='date',
            how='inner'  # Only keep dates that exist in both datasets
        )

        aligned_rows = len(df_aligned)
        date_range = f"{df_aligned['date'].min()} to {df_aligned['date'].max()}"

        # Save aligned data
        output_file = os.path.join(ALIGNED_DATA_PATH, f'aligned_{ticker_name}.parquet')
        df_aligned.to_parquet(output_file, index=False, engine='pyarrow')

        logger.info(f"✓ ({aligned_rows} rows | {date_range})")

        alignment_results.append({
            'Ticker': ticker_name,
            'Original_Rows': original_rows,
            'Aligned_Rows': aligned_rows,
            'Match_Rate': f"{(aligned_rows/original_rows)*100:.1f}%",
            'Date_Range': date_range,
            'Status': 'Success'
        })

        success_count += 1

    except KeyError as e:
        error_msg = f"Missing column: {str(e)}"
        logger.error(f"✗ FAILED ({error_msg})")

        alignment_results.append({
            'Ticker': ticker_name,
            'Original_Rows': 0,
            'Aligned_Rows': 0,
            'Match_Rate': '0%',
            'Date_Range': 'N/A',
            'Status': f'Failed: {error_msg}'
        })

        failed_count += 1

    except Exception as e:
        error_msg = str(e)[:50]
        logger.error(f"✗ FAILED ({error_msg})")

        alignment_results.append({
            'Ticker': ticker_name,
            'Original_Rows': 0,
            'Aligned_Rows': 0,
            'Match_Rate': '0%',
            'Date_Range': 'N/A',
            'Status': f'Failed: {error_msg}'
        })

        failed_count += 1

# ============================================================================
# STEP 4: Create Master Aligned Dataset (Optional)
# ============================================================================
logger.info("\n\n[4/5] Creating master aligned dataset...")
logger.info("  (Combining all tickers into single file for convenience)")

# Create a wide-format master dataset with all tickers
master_data = df_ephemeris.copy()

for file_path in glob.glob(os.path.join(ALIGNED_DATA_PATH, 'aligned_*.parquet')):
    filename = os.path.basename(file_path)
    ticker_name = filename.replace('aligned_', '').replace('.parquet', '')

    try:
        df_ticker = pd.read_parquet(file_path)

        # Select only financial columns (not ephemeris columns to avoid duplication)
        financial_cols = ['date', 'open', 'high', 'low', 'close', 'volume', 'adj_close']
        df_ticker_subset = df_ticker[financial_cols].copy()

        # Rename columns to include ticker name
        rename_dict = {
            'open': f'{ticker_name}_open',
            'high': f'{ticker_name}_high',
            'low': f'{ticker_name}_low',
            'close': f'{ticker_name}_close',
            'volume': f'{ticker_name}_volume',
            'adj_close': f'{ticker_name}_adj_close'
        }
        df_ticker_subset.rename(columns=rename_dict, inplace=True)

        # Merge into master dataset
        master_data = pd.merge(master_data, df_ticker_subset, on='date', how='left')

    except Exception as e:
        logger.warning(f"  ⚠ Warning: Could not add {ticker_name} to master dataset: {e}")

# Save master dataset
master_file = os.path.join(ALIGNED_DATA_PATH, 'master_aligned_dataset.parquet')
master_data.to_parquet(master_file, index=False, engine='pyarrow')

file_size_mb = os.path.getsize(master_file) / (1024 * 1024)
logger.info(f"  ✓ Created master dataset")
logger.info(f"  ✓ Shape: {master_data.shape}")
logger.info(f"  ✓ Size: {file_size_mb:.2f} MB")
logger.info(f"  ✓ Saved: master_aligned_dataset.parquet")

# ============================================================================
# STEP 5: Generate Summary Statistics
# ============================================================================
logger.info("\n[5/5] Generating summary statistics...")

summary_df = pd.DataFrame(alignment_results)

# ============================================================================
# DISPLAY RESULTS
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("ALIGNMENT SUMMARY")
logger.info("=" * 70)

logger.info(f"\n  ✓ Successful: {success_count}")
logger.error(f"  ✗ Failed: {failed_count}")
logger.info(f"  Total: {len(financial_files)}")

logger.info("\n" + "=" * 70)
logger.info("DETAILED ALIGNMENT RESULTS")
logger.info("=" * 70)

print("\n" + tabulate(summary_df, headers='keys', tablefmt='grid', showindex=False))

# ============================================================================
# SAMPLE DATA PREVIEW
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("SAMPLE: MASTER ALIGNED DATASET (First 3 Rows)")
logger.info("=" * 70)

# Show a subset of columns for readability
preview_cols = ['date', 'sun_longitude', 'moon_longitude', 'mercury_longitude']

# Add first ticker's financial data to preview
first_ticker_cols = [col for col in master_data.columns if '_close' in col][:3]
preview_cols.extend(first_ticker_cols)

# Make sure columns exist
preview_cols = [col for col in preview_cols if col in master_data.columns]

if len(preview_cols) > 0:
    sample_data = master_data[preview_cols].head(3)
    print("\n" + tabulate(sample_data, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
else:
    logger.info("\n  (No data to preview)")

# ============================================================================
# DATA QUALITY METRICS
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("DATA QUALITY METRICS")
logger.info("=" * 70)

logger.info(f"\n  Total aligned records: {len(master_data)}")
logger.info(f"  Date range: {master_data['date'].min()} to {master_data['date'].max()}")
logger.info(f"  Total columns: {len(master_data.columns)}")

# Count null values in planetary data
planetary_cols = [col for col in master_data.columns if any(
    planet in col for planet in ['sun_', 'moon_', 'mercury_', 'venus_', 'mars_', 'jupiter_', 'saturn_', 'rahu_']
)]
planetary_nulls = master_data[planetary_cols].isnull().sum().sum()

logger.info(f"  Planetary data nulls: {planetary_nulls}")

# Count completeness of financial data
financial_cols = [col for col in master_data.columns if '_close' in col]
if len(financial_cols) > 0:
    financial_completeness = (1 - master_data[financial_cols].isnull().sum().sum() /
                             (len(master_data) * len(financial_cols))) * 100
    logger.info(f"  Financial data completeness: {financial_completeness:.1f}%")

# ============================================================================
# FINAL STATUS
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("PHASE 1 (DATA ALIGNMENT) - STATUS: COMPLETE ✓")
logger.info("=" * 70)

logger.info("\n📋 Phase 1 Summary:")
logger.info("  ✓ Cell 1: Financial data acquired (32 tickers)")
logger.info("  ✓ Cell 2: Vedic ephemeris generated (9,434 days)")
logger.info("  ✓ Cell 3: Datasets aligned and merged")

logger.info("\n📂 Output Files:")
logger.info(f"  • Individual aligned files: {ALIGNED_DATA_PATH}/aligned_*.parquet")
logger.info(f"  • Master dataset: {ALIGNED_DATA_PATH}/master_aligned_dataset.parquet")
logger.info(f"  • Total files created: {success_count + 1}")

logger.info("\n🎯 Ready for Phase 2: Feature Engineering")
logger.info("  The aligned datasets are now ready for:")
logger.info("  • Aspect calculations (conjunctions, trines, squares, etc.)")
logger.info("  • Transit analysis (planet-to-planet relationships)")
logger.info("  • Nakshatra mapping (lunar mansion positions)")
logger.info("  • Dasha period calculations")
logger.info("  • Technical indicators integration")

logger.info("\n" + "=" * 70)

# PHASE 2 - ⚙️ Feature Engineering

In [ ]:
# Cell 4: Planetary Aspect Calculations (Phase 2 - Part 1 of 5)
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
from datetime import datetime
from tabulate import tabulate
import itertools

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 2: FEATURE ENGINEERING")
logger.info("Phase 2 Progress: Part 1 of 5 (Planetary Aspects)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup Paths
# ============================================================================
logger.info("\n[1/7] Setting up paths...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
ALIGNED_DATA_PATH = os.path.join(BASE_PATH, 'aligned_data')
FEATURE_DATA_PATH = os.path.join(BASE_PATH, 'feature_data')

# Create feature data directory
os.makedirs(FEATURE_DATA_PATH, exist_ok=True)

INPUT_FILE = os.path.join(ALIGNED_DATA_PATH, 'master_aligned_dataset.parquet')
OUTPUT_FILE = os.path.join(FEATURE_DATA_PATH, 'aspects_features.parquet')

logger.info(f"  ✓ Input: {ALIGNED_DATA_PATH}/master_aligned_dataset.parquet")
logger.info(f"  ✓ Output: {FEATURE_DATA_PATH}/aspects_features.parquet")

# ============================================================================
# STEP 2: Load Master Aligned Data
# ============================================================================
logger.info("\n[2/7] Loading master aligned dataset...")

if not os.path.exists(INPUT_FILE):
    logger.critical(f"\n✗ FATAL ERROR: Input file not found")
    logger.info("  Please run Cell 3 first to generate aligned data.")
    raise SystemExit(1)

df = pd.read_parquet(INPUT_FILE)
df['date'] = pd.to_datetime(df['date'])

logger.info(f"  ✓ Loaded dataset")
logger.info(f"  ✓ Shape: {df.shape}")
logger.info(f"  ✓ Date range: {df['date'].min().date()} to {df['date'].max().date()}")

# ============================================================================
# STEP 3: Define Astrological Constants
# ============================================================================
logger.info("\n[3/7] Setting up astrological parameters...")

# Planets (must match column names from Cell 2)
PLANETS = ['sun', 'moon', 'mercury', 'venus', 'mars',
           'jupiter', 'saturn', 'rahu']

# Planet pairs for aspects (28 combinations)
PLANET_PAIRS = list(itertools.combinations(PLANETS, 2))

# Major aspects with traditional orbs (per aspect type)
# Format: {aspect_name: (angle, base_orb)}
ASPECT_DEFINITIONS = {
    'conjunction': (0, 10),      # Most powerful, widest orb
    'opposition': (180, 10),     # Very powerful
    'trine': (120, 8),           # Harmonious, medium orb
    'square': (90, 8),           # Challenging, medium orb
    'sextile': (60, 6),          # Mild, smaller orb
}

# Tight orb threshold for "exact" aspects
TIGHT_ORB = 2.0

# Planet importance for orb adjustments (Luminaries get more)
PLANET_IMPORTANCE = {
    'sun': 1.2,      # Luminary boost
    'moon': 1.2,     # Luminary boost
    'mercury': 1.0,
    'venus': 1.0,
    'mars': 1.0,
    'jupiter': 1.0,
    'saturn': 1.0,
    'rahu': 0.8,     # Node, slightly reduced
}

logger.info(f"  ✓ Configured {len(PLANETS)} planets")
logger.info(f"  ✓ Generated {len(PLANET_PAIRS)} planet pairs")
logger.info(f"  ✓ Defined {len(ASPECT_DEFINITIONS)} major aspects")
logger.info(f"  ✓ Tight orb threshold: {TIGHT_ORB}°")

# ============================================================================
# STEP 4: Define Calculation Functions
# ============================================================================
logger.info("\n[4/7] Defining calculation functions...")

def normalize_angle(angle):
    """Normalize angle to 0-360 range."""
    return angle % 360.0

def angular_distance(lon1, lon2):
    """
    Calculate shortest angular distance between two longitudes.
    Returns value between 0 and 180 degrees.
    """
    diff = np.abs(lon1 - lon2)
    # Use modulo to handle wrap-around
    diff = np.where(diff > 180, 360.0 - diff, diff)
    return diff

def get_aspect_orb(planet1, planet2, aspect_name):
    """
    Calculate adjusted orb for a planet pair and aspect.
    Luminaries (Sun/Moon) get wider orbs.
    """
    base_orb = ASPECT_DEFINITIONS[aspect_name][1]

    # Apply importance multipliers
    p1_mult = PLANET_IMPORTANCE.get(planet1, 1.0)
    p2_mult = PLANET_IMPORTANCE.get(planet2, 1.0)

    # Use the larger multiplier
    max_mult = max(p1_mult, p2_mult)

    return base_orb * max_mult

def calculate_aspect_strength(distance_from_exact, orb):
    """
    Calculate aspect strength (0 to 1).
    1.0 = exact aspect
    0.0 = at orb limit
    Uses cosine curve for smooth falloff.
    Vectorized to handle arrays.
    """
    # Ensure inputs are arrays
    distance_from_exact = np.asarray(distance_from_exact)

    # Cosine curve: 1 at center, 0 at orb
    # Only calculate for values within orb
    strength = np.cos((distance_from_exact / orb) * (np.pi / 2))

    # Clip to ensure values stay in [0, 1] range
    strength = np.clip(strength, 0.0, 1.0)

    return strength

def determine_applying_separating(p1_lon, p1_speed, p2_lon, p2_speed, aspect_angle):
    """
    Determine if aspect is applying (forming) or separating (dissolving).

    Logic:
    - Calculate current distance to aspect
    - Estimate future distance using speeds
    - If future distance < current distance → applying
    - If future distance > current distance → separating
    """
    # Current angular distance
    current_dist = angular_distance(p1_lon, p2_lon)

    # Estimate positions in ~1 day (speeds are in degrees/day)
    p1_future = normalize_angle(p1_lon + p1_speed)
    p2_future = normalize_angle(p2_lon + p2_speed)

    # Future angular distance
    future_dist = angular_distance(p1_future, p2_future)

    # Calculate distance from exact aspect angle
    current_from_exact = np.abs(current_dist - aspect_angle)
    current_from_exact = np.minimum(current_from_exact, 360 - current_from_exact)

    future_from_exact = np.abs(future_dist - aspect_angle)
    future_from_exact = np.minimum(future_from_exact, 360 - future_from_exact)

    # Applying if getting closer to exact aspect
    is_applying = future_from_exact < current_from_exact

    return is_applying

logger.info("  ✓ Angular distance calculation")
logger.info("  ✓ Dynamic orb adjustment")
logger.info("  ✓ Aspect strength scoring")
logger.info("  ✓ Applying/separating detection")

# ============================================================================
# STEP 5: Calculate Aspect Features (Vectorized)
# ============================================================================
logger.info("\n[5/7] Calculating aspect features...")
logger.info(f"  Processing {len(PLANET_PAIRS)} planet pairs × {len(ASPECT_DEFINITIONS)} aspects")
logger.info(f"  Expected features: ~{len(PLANET_PAIRS) * len(ASPECT_DEFINITIONS) * 5}")

start_time = datetime.now()

# Initialize feature tracking
aspect_features = {}
feature_count = 0

# Progress tracking
total_calculations = len(PLANET_PAIRS) * len(ASPECT_DEFINITIONS)
calc_count = 0
progress_interval = 20

for p1, p2 in PLANET_PAIRS:
    # Get column names
    p1_lon_col = f'{p1}_longitude'
    p1_speed_col = f'{p1}_speed'
    p2_lon_col = f'{p2}_longitude'
    p2_speed_col = f'{p2}_speed'

    # Extract values as numpy arrays for speed
    p1_lon = df[p1_lon_col].values
    p1_speed = df[p1_speed_col].values
    p2_lon = df[p2_lon_col].values
    p2_speed = df[p2_speed_col].values

    # Calculate angular distance between planets
    ang_dist = angular_distance(p1_lon, p2_lon)

    for aspect_name, (aspect_angle, base_orb) in ASPECT_DEFINITIONS.items():
        calc_count += 1

        # Get adjusted orb for this planet pair
        orb = get_aspect_orb(p1, p2, aspect_name)

        # Calculate distance from exact aspect
        dist_from_exact = np.abs(ang_dist - aspect_angle)

        # Handle 360° wrap (e.g., 359° is close to 0°)
        dist_from_exact = np.minimum(dist_from_exact, 360 - dist_from_exact)

        # --- Feature 1: Is aspect active? (within orb) ---
        is_active = (dist_from_exact <= orb).astype(np.int8)

        # --- Feature 2: Is it a tight/exact aspect? ---
        is_tight = (dist_from_exact <= TIGHT_ORB).astype(np.int8)

        # --- Feature 3: Aspect strength (0 to 1) ---
        strength = np.zeros_like(dist_from_exact, dtype=np.float32)
        active_mask = dist_from_exact <= orb
        strength[active_mask] = calculate_aspect_strength(
            dist_from_exact[active_mask],
            orb
        )

        # --- Feature 4: Distance from exact (in degrees) ---
        exact_distance = dist_from_exact.astype(np.float32)

        # --- Feature 5: Is applying (vs separating)? ---
        is_applying = determine_applying_separating(
            p1_lon, p1_speed, p2_lon, p2_speed, aspect_angle
        ).astype(np.int8)

        # Store features with descriptive names
        base_name = f'{p1}_{p2}_{aspect_name}'
        aspect_features[f'{base_name}_active'] = is_active
        aspect_features[f'{base_name}_tight'] = is_tight
        aspect_features[f'{base_name}_strength'] = strength
        aspect_features[f'{base_name}_exact_dist'] = exact_distance
        aspect_features[f'{base_name}_applying'] = is_applying

        feature_count += 5

        # Progress indicator
        if calc_count % progress_interval == 0 or calc_count == total_calculations:
            progress_pct = (calc_count / total_calculations) * 100
            logger.info(f"    [{calc_count}/{total_calculations}] {progress_pct:.0f}% complete")

# Convert to DataFrame
df_aspects = pd.DataFrame(aspect_features)

# Add date column for merging
df_aspects['date'] = df['date'].values

end_time = datetime.now()
elapsed = (end_time - start_time).total_seconds()

logger.info(f"\n  ✓ Calculations complete in {elapsed:.1f} seconds")
logger.info(f"  ✓ Generated {feature_count} aspect features")

# ============================================================================
# STEP 6: Calculate Aggregate Aspect Metrics
# ============================================================================
logger.info("\n[6/7] Calculating aggregate aspect metrics...")

# Count total active aspects per day
aspect_active_cols = [col for col in df_aspects.columns if col.endswith('_active')]
df_aspects['total_aspects_active'] = df_aspects[aspect_active_cols].sum(axis=1)

# Count tight aspects per day
aspect_tight_cols = [col for col in df_aspects.columns if col.endswith('_tight')]
df_aspects['total_aspects_tight'] = df_aspects[aspect_tight_cols].sum(axis=1)

# Average aspect strength (for active aspects)
aspect_strength_cols = [col for col in df_aspects.columns if col.endswith('_strength')]
df_aspects['avg_aspect_strength'] = df_aspects[aspect_strength_cols].mean(axis=1)

# Count applying aspects (momentum indicator)
aspect_applying_cols = [col for col in df_aspects.columns if col.endswith('_applying')]
df_aspects['total_aspects_applying'] = df_aspects[aspect_applying_cols].sum(axis=1)

# Count by aspect type
for aspect_name in ASPECT_DEFINITIONS.keys():
    type_cols = [col for col in aspect_active_cols if f'_{aspect_name}_active' in col]
    df_aspects[f'count_{aspect_name}s'] = df_aspects[type_cols].sum(axis=1)

logger.info(f"  ✓ Added {5 + len(ASPECT_DEFINITIONS)} aggregate metrics")

# ============================================================================
# STEP 7: Save and Validate
# ============================================================================
logger.info("\n[7/7] Saving aspect features...")

try:
    df_aspects.to_parquet(OUTPUT_FILE, index=False, engine='pyarrow')
    file_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
    logger.info(f"  ✓ Saved: aspects_features.parquet")
    logger.info(f"  ✓ File size: {file_size_mb:.2f} MB")
except Exception as e:
    logger.critical(f"\n✗ FATAL ERROR: Could not save file")
    logger.critical(f"  Error: {e}")
    raise SystemExit(1)

# ============================================================================
# SAMPLE OUTPUT & VALIDATION
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("VALIDATION: Sun-Moon Aspects Analysis")
logger.info("=" * 70)

# Check if Sun-Moon aspects ever occur
sun_moon_cols = [col for col in df_aspects.columns if col.startswith('sun_moon_')]
sun_moon_active = df_aspects[[col for col in sun_moon_cols if col.endswith('_active')]].sum()

logger.info("\nSun-Moon Aspect Occurrence (Total Days Active):")
for col in sun_moon_active.index:
    aspect_type = col.replace('sun_moon_', '').replace('_active', '')
    count = sun_moon_active[col]
    pct = (count / len(df_aspects)) * 100
    logger.info(f"  • {aspect_type.capitalize()}: {count} days ({pct:.1f}%)")

# Find a day with active Sun-Moon aspects
logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Days WITH Sun-Moon Aspects")
logger.info("=" * 70)

# Find days where ANY sun-moon aspect is active
any_sun_moon_active = df_aspects[[col for col in sun_moon_cols if col.endswith('_active')]].sum(axis=1) > 0
days_with_aspects = df_aspects[any_sun_moon_active]

if len(days_with_aspects) > 0:
    logger.info(f"\nFound {len(days_with_aspects)} days with Sun-Moon aspects")
    logger.info("\nShowing first 5 days with active aspects:")

    sample_cols = ['date',
                   'sun_moon_conjunction_active',
                   'sun_moon_conjunction_tight',
                   'sun_moon_conjunction_strength',
                   'sun_moon_opposition_active',
                   'sun_moon_square_active',
                   'sun_moon_trine_active']

    existing_cols = [col for col in sample_cols if col in days_with_aspects.columns]
    print("\n" + tabulate(days_with_aspects[existing_cols].head(5),
                         headers='keys', tablefmt='grid',
                         showindex=False, floatfmt=".3f"))
else:
    logger.warning("\n⚠ WARNING: No Sun-Moon aspects found in dataset!")
    logger.info("  This may indicate an issue with aspect calculations.")

# Show actual Sun-Moon positions for verification
logger.info("\n" + "=" * 70)
logger.info("VERIFICATION: Sun & Moon Positions (First 5 Days)")
logger.info("=" * 70)

# Get original planetary data
verify_cols = ['date', 'sun_longitude', 'moon_longitude']
if all(col in df.columns for col in verify_cols):
    df_verify = df[verify_cols].head(5).copy()

    # Calculate angular distance manually
    df_verify['angular_distance'] = angular_distance(
        df['sun_longitude'].head(5).values,
        df['moon_longitude'].head(5).values
    )

    print("\n" + tabulate(df_verify, headers='keys', tablefmt='grid',
                         showindex=False, floatfmt=".2f"))

    logger.info("\nExpected aspects based on angular distance:")
    logger.info("  • Conjunction (0°): distance < 12° (Sun/Moon orb)")
    logger.info("  • Opposition (180°): |distance - 180°| < 12°")
    logger.info("  • Square (90°): |distance - 90°| < 9.6°")
    logger.info("  • Trine (120°): |distance - 120°| < 9.6°")
    logger.info("  • Sextile (60°): |distance - 60°| < 7.2°")

logger.info("\n" + "=" * 70)
logger.info("AGGREGATE METRICS (All Days)")
logger.info("=" * 70)

agg_cols = ['total_aspects_active', 'total_aspects_tight',
            'avg_aspect_strength', 'count_conjunctions', 'count_squares',
            'count_trines', 'count_oppositions', 'count_sextiles']
existing_agg = [col for col in agg_cols if col in df_aspects.columns]

agg_stats = df_aspects[existing_agg].describe()
print("\n" + tabulate(agg_stats, headers='keys', tablefmt='grid', floatfmt=".2f"))

# Validation statistics
logger.info("\n" + "=" * 70)
logger.info("DATA QUALITY VALIDATION")
logger.info("=" * 70)

logger.info(f"\n  Dataset shape: {df_aspects.shape}")
logger.info(f"  Total features: {len(df_aspects.columns)}")
logger.info(f"  Date range: {df_aspects['date'].min().date()} to {df_aspects['date'].max().date()}")

# Check for nulls
null_counts = df_aspects.isnull().sum().sum()
logger.info(f"  Null values: {null_counts} ({null_counts / df_aspects.size * 100:.2f}%)")

# Sample statistics
logger.info(f"\n  Average active aspects per day: {df_aspects['total_aspects_active'].mean():.1f}")
logger.info(f"  Max active aspects on single day: {df_aspects['total_aspects_active'].max()}")
logger.info(f"  Days with tight aspects: {(df_aspects['total_aspects_tight'] > 0).sum()} ({(df_aspects['total_aspects_tight'] > 0).sum() / len(df_aspects) * 100:.1f}%)")

# ============================================================================
# FINAL STATUS
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("PHASE 2 (ASPECT FEATURES) - STATUS: COMPLETE ✓")
logger.info("=" * 70)

logger.info("\n📋 Integration Points for Downstream Phases:")
logger.info("  → Phase 3 (Model Training):")
logger.info(f"    • {feature_count} aspect features ready for ML input")
logger.info("    • Binary flags for classification, strength scores for regression")
logger.info("  → Phase 4 (Backtesting):")
logger.info("    • Use '_active' columns to filter specific aspect events")
logger.info("    • Example: days_with_mars_saturn_square = df[df['mars_saturn_square_active'] == 1]")
logger.info("  → Phase 5 (Insight Extraction):")
logger.info("    • Feature names follow '{planet1}_{planet2}_{aspect}_{metric}' pattern")
logger.info("    • Use for SHAP analysis and feature importance ranking")

logger.info("\n📋 Next Steps:")
logger.info("  1. ✓ Planetary aspects calculated (Cell 4 complete)")
logger.info("  2. ▶ Run Cell 5: Transit & Positional Features")
logger.info("  3. ▶ Run Cell 6: Cyclic & Temporal Features")
logger.info("  4. ▶ Run Cell 7: Advanced Astrological Indicators")
logger.info("  5. ▶ Run Cell 8: Feature Integration & Final Dataset")

logger.info("\n📂 Output Files:")
logger.info(f"  {OUTPUT_FILE}")
logger.info(f"  ({len(df_aspects)} rows × {len(df_aspects.columns)} features)")

logger.info("\n" + "=" * 70)

In [ ]:
# Cell 5: Transit & Positional Features (Phase 2 - Part 2 of 5)
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
from datetime import datetime
from tabulate import tabulate

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 2: FEATURE ENGINEERING")
logger.info("Phase 2 Progress: Part 2 of 5 (Transits & Positions)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup Paths
# ============================================================================
logger.info("\n[1/6] Setting up paths...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
ALIGNED_DATA_PATH = os.path.join(BASE_PATH, 'aligned_data')
FEATURE_DATA_PATH = os.path.join(BASE_PATH, 'feature_data')

INPUT_FILE = os.path.join(ALIGNED_DATA_PATH, 'master_aligned_dataset.parquet')
OUTPUT_FILE = os.path.join(FEATURE_DATA_PATH, 'transit_features.parquet')

logger.info(f"  ✓ Input: master_aligned_dataset.parquet")
logger.info(f"  ✓ Output: transit_features.parquet")

# ============================================================================
# STEP 2: Load Master Aligned Data
# ============================================================================
logger.info("\n[2/6] Loading master aligned dataset...")

if not os.path.exists(INPUT_FILE):
    logger.critical(f"\n✗ FATAL ERROR: Input file not found")
    logger.info("  Please run Cell 3 first.")
    raise SystemExit(1)

df = pd.read_parquet(INPUT_FILE)
df['date'] = pd.to_datetime(df['date'])

logger.info(f"  ✓ Loaded dataset")
logger.info(f"  ✓ Shape: {df.shape}")
logger.info(f"  ✓ Date range: {df['date'].min().date()} to {df['date'].max().date()}")

# ============================================================================
# STEP 3: Define Astrological Constants
# ============================================================================
logger.info("\n[3/6] Defining astrological constants...")

# Planets to process
PLANETS = ['sun', 'moon', 'mercury', 'venus', 'mars',
           'jupiter', 'saturn', 'rahu']

# Zodiac signs (Vedic/Sidereal)
ZODIAC_SIGNS = [
    'Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo',
    'Liberia', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces'
]

# Nakshatras (27 lunar mansions, 13°20' each)
NAKSHATRAS = [
    'Ashwini', 'Bharani', 'Krittika', 'Rohini', 'Mrigashira', 'Ardra',
    'Punarvasu', 'Pushya', 'Ashlesha', 'Magha', 'Purva Phalguni', 'Uttara Phalguni',
    'Hasta', 'Chitra', 'Swati', 'Vishakha', 'Anuradha', 'Jyeshtha',
    'Mula', 'Purva Ashadha', 'Uttara Ashadha', 'Shravana', 'Dhanishta', 'Shatabhisha',
    'Purva Bhadrapada', 'Uttara Bhadrapada', 'Revati'
]

# Planetary dignities (exaltation/debilitation points in degrees)
DIGNITY_POINTS = {
    'sun': {'exalted': 10, 'debilitated': 190},      # Exalted in Aries 10°, Debilitated in Libra 10°
    'moon': {'exalted': 33, 'debilitated': 213},     # Exalted in Taurus 3°, Debilitated in Scorpio 3°
    'mercury': {'exalted': 165, 'debilitated': 345}, # Exalted in Virgo 15°, Debilitated in Pisces 15°
    'venus': {'exalted': 357, 'debilitated': 177},   # Exalted in Pisces 27°, Debilitated in Virgo 27°
    'mars': {'exalted': 298, 'debilitated': 118},    # Exalted in Capricorn 28°, Debilitated in Cancer 28°
    'jupiter': {'exalted': 95, 'debilitated': 275},  # Exalted in Cancer 5°, Debilitated in Capricorn 5°
    'saturn': {'exalted': 200, 'debilitated': 20},   # Exalted in Libra 20°, Debilitated in Aries 20°
}

# Retrograde speed thresholds (approximately when planet appears stationary)
RETROGRADE_STATIONARY_THRESHOLD = 0.05  # degrees/day

logger.info(f"  ✓ Configured {len(PLANETS)} planets")
logger.info(f"  ✓ Defined {len(ZODIAC_SIGNS)} zodiac signs")
logger.info(f"  ✓ Defined {len(NAKSHATRAS)} nakshatras")
logger.info(f"  ✓ Dignity points for 7 planets")

# ============================================================================
# STEP 4: Calculate Zodiac Sign Features
# ============================================================================
logger.info("\n[4/6] Calculating zodiac sign positions...")

transit_features = {'date': df['date'].values}
feature_count = 0

for planet in PLANETS:
    lon_col = f'{planet}_longitude'

    if lon_col not in df.columns:
        logger.warning(f"  ⚠ Skipping {planet} - longitude column not found")
        continue

    longitude = df[lon_col].values

    # Calculate zodiac sign (0-11, where 0=Aries, 1=Taurus, etc.)
    # Each sign is 30 degrees
    sign_index = (longitude // 30).astype(np.int8)

    # Calculate degrees within sign (0-29.99)
    degrees_in_sign = longitude % 30

    # Detect sign ingress (planet just entered new sign)
    # Check if degrees_in_sign < previous day's degrees (wrapped around)
    ingress = np.zeros(len(longitude), dtype=np.int8)
    if len(longitude) > 1:
        ingress[1:] = (degrees_in_sign[1:] < degrees_in_sign[:-1]).astype(np.int8)

    # Store features
    transit_features[f'{planet}_sign'] = sign_index
    transit_features[f'{planet}_degrees_in_sign'] = degrees_in_sign.astype(np.float32)
    transit_features[f'{planet}_sign_ingress'] = ingress

    feature_count += 3

logger.info(f"  ✓ Created {feature_count} zodiac sign features")

# ============================================================================
# STEP 5: Calculate Nakshatra Features
# ============================================================================
logger.info("\n[5/6] Calculating nakshatra positions...")

# Focus on Moon and Sun (most important for nakshatras)
for planet in ['moon', 'sun']:
    lon_col = f'{planet}_longitude'

    if lon_col not in df.columns:
        continue

    longitude = df[lon_col].values

    # Each nakshatra is 13.333... degrees (360/27)
    nakshatra_width = 360.0 / 27
    nakshatra_index = (longitude / nakshatra_width).astype(np.int8)

    # Nakshatra pada (quarter): 1-4
    # Each nakshatra has 4 padas of 3°20' each
    pada_within_nakshatra = ((longitude % nakshatra_width) / (nakshatra_width / 4))
    pada = (pada_within_nakshatra.astype(np.int8) + 1).clip(1, 4)  # 1-4

    # Degrees within nakshatra
    degrees_in_nakshatra = longitude % nakshatra_width

    # Detect nakshatra change
    nakshatra_change = np.zeros(len(longitude), dtype=np.int8)
    if len(longitude) > 1:
        nakshatra_change[1:] = (nakshatra_index[1:] != nakshatra_index[:-1]).astype(np.int8)

    # Store features
    transit_features[f'{planet}_nakshatra'] = nakshatra_index
    transit_features[f'{planet}_nakshatra_pada'] = pada
    transit_features[f'{planet}_degrees_in_nakshatra'] = degrees_in_nakshatra.astype(np.float32)
    transit_features[f'{planet}_nakshatra_change'] = nakshatra_change

    feature_count += 4

logger.info(f"  ✓ Created nakshatra features for Moon and Sun")

# ============================================================================
# STEP 6: Calculate Speed & Motion Features
# ============================================================================
logger.info("\n[6/6] Calculating planetary motion features...")

for planet in PLANETS:
    if planet == 'rahu':  # Rahu is always retrograde by definition
        continue

    speed_col = f'{planet}_speed'

    if speed_col not in df.columns:
        continue

    speed = df[speed_col].values

    # Retrograde flag (speed < 0)
    is_retrograde = (speed < 0).astype(np.int8)

    # Stationary flag (speed near 0, within threshold)
    is_stationary = (np.abs(speed) < RETROGRADE_STATIONARY_THRESHOLD).astype(np.int8)

    # Speed category: -1 (retrograde), 0 (stationary), 1 (direct)
    speed_category = np.zeros_like(speed, dtype=np.int8)
    speed_category[speed < -RETROGRADE_STATIONARY_THRESHOLD] = -1  # Retrograde
    speed_category[speed > RETROGRADE_STATIONARY_THRESHOLD] = 1    # Direct
    # Stationary = 0 (default)

    # Detect station (change in direction)
    # Station occurs when speed crosses zero
    station = np.zeros(len(speed), dtype=np.int8)
    if len(speed) > 1:
        # Check for sign change in speed
        station[1:] = ((speed[:-1] * speed[1:]) < 0).astype(np.int8)

    # Store features
    transit_features[f'{planet}_retrograde'] = is_retrograde
    transit_features[f'{planet}_stationary'] = is_stationary
    transit_features[f'{planet}_speed_category'] = speed_category
    transit_features[f'{planet}_station'] = station

    feature_count += 4

logger.info(f"  ✓ Created motion features for {len(PLANETS)-1} planets")

# ============================================================================
# STEP 7: Calculate Dignity Features
# ============================================================================
logger.info("\n[7/7] Calculating planetary dignity features...")

for planet, dignity_data in DIGNITY_POINTS.items():
    lon_col = f'{planet}_longitude'

    if lon_col not in df.columns:
        continue

    longitude = df[lon_col].values

    exalted_point = dignity_data['exalted']
    debilitated_point = dignity_data['debilitated']

    # Calculate angular distance to exaltation point
    dist_to_exalted = np.abs(longitude - exalted_point)
    dist_to_exalted = np.minimum(dist_to_exalted, 360 - dist_to_exalted)

    # Calculate angular distance to debilitation point
    dist_to_debilitated = np.abs(longitude - debilitated_point)
    dist_to_debilitated = np.minimum(dist_to_debilitated, 360 - dist_to_debilitated)

    # Exalted flag (within 5 degrees of exaltation point)
    is_exalted = (dist_to_exalted <= 5).astype(np.int8)

    # Debilitated flag (within 5 degrees of debilitation point)
    is_debilitated = (dist_to_debilitated <= 5).astype(np.int8)

    # Dignity score: positive near exaltation, negative near debilitation
    # Scale: +1.0 at exaltation point, -1.0 at debilitation point, 0 neutral
    dignity_score = np.zeros_like(longitude, dtype=np.float32)

    # Positive contribution from exaltation (0 to 1)
    exalted_contribution = np.maximum(0, 1 - (dist_to_exalted / 30))

    # Negative contribution from debilitation (0 to -1)
    debilitated_contribution = -np.maximum(0, 1 - (dist_to_debilitated / 30))

    dignity_score = exalted_contribution + debilitated_contribution

    # Store features
    transit_features[f'{planet}_is_exalted'] = is_exalted
    transit_features[f'{planet}_is_debilitated'] = is_debilitated
    transit_features[f'{planet}_dignity_score'] = dignity_score

    feature_count += 3

logger.info(f"  ✓ Created dignity features for 7 planets")

# ============================================================================
# STEP 8: Create DataFrame and Save
# ============================================================================
logger.info(f"\n[8/8] Saving transit features...")

df_transit = pd.DataFrame(transit_features)

try:
    df_transit.to_parquet(OUTPUT_FILE, index=False, engine='pyarrow')
    file_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
    logger.info(f"  ✓ Saved: transit_features.parquet")
    logger.info(f"  ✓ File size: {file_size_mb:.2f} MB")
except Exception as e:
    logger.critical(f"\n✗ FATAL ERROR: Could not save file")
    logger.critical(f"  Error: {e}")
    raise SystemExit(1)

# ============================================================================
# VALIDATION & SAMPLE OUTPUT
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Moon Positional Features (First 5 Days)")
logger.info("=" * 70)

sample_cols = ['date', 'moon_sign', 'moon_degrees_in_sign',
               'moon_nakshatra', 'moon_nakshatra_pada',
               'moon_retrograde']

existing_cols = [col for col in sample_cols if col in df_transit.columns]
sample_data = df_transit[existing_cols].head(5).copy()

# Add sign names for readability
if 'moon_sign' in sample_data.columns:
    sample_data['moon_sign_name'] = sample_data['moon_sign'].apply(lambda x: ZODIAC_SIGNS[x] if 0 <= x < 12 else 'Unknown')

# Add nakshatra names
if 'moon_nakshatra' in sample_data.columns:
    sample_data['moon_nakshatra_name'] = sample_data['moon_nakshatra'].apply(lambda x: NAKSHATRAS[x] if 0 <= x < 27 else 'Unknown')

print("\n" + tabulate(sample_data, headers='keys', tablefmt='grid',
                     showindex=False, floatfmt=".2f"))

logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Retrograde Planets (Days with Retrogrades)")
logger.info("=" * 70)

# Find days with any retrograde planets
retrograde_cols = [col for col in df_transit.columns if col.endswith('_retrograde')]
any_retrograde = df_transit[retrograde_cols].sum(axis=1) > 0
days_with_rx = df_transit[any_retrograde]

if len(days_with_rx) > 0:
    logger.info(f"\nFound {len(days_with_rx)} days with retrograde planets ({len(days_with_rx)/len(df_transit)*100:.1f}%)")

    rx_sample_cols = ['date'] + [col for col in retrograde_cols if col in days_with_rx.columns][:5]
    logger.info("\nFirst 5 days with retrogrades:")
    print("\n" + tabulate(days_with_rx[rx_sample_cols].head(5),
                         headers='keys', tablefmt='grid', showindex=False))
else:
    logger.info("\n  No retrograde periods found in dataset")

logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Planetary Dignity (Jupiter Exalted)")
logger.info("=" * 70)

if 'jupiter_is_exalted' in df_transit.columns:
    jupiter_exalted_days = df_transit[df_transit['jupiter_is_exalted'] == 1]

    if len(jupiter_exalted_days) > 0:
        logger.info(f"\nJupiter in exaltation: {len(jupiter_exalted_days)} days ({len(jupiter_exalted_days)/len(df_transit)*100:.1f}%)")

        dignity_cols = ['date', 'jupiter_is_exalted', 'jupiter_dignity_score']
        existing_dignity = [col for col in dignity_cols if col in jupiter_exalted_days.columns]

        logger.info("\nFirst 5 days:")
        print("\n" + tabulate(jupiter_exalted_days[existing_dignity].head(5),
                             headers='keys', tablefmt='grid',
                             showindex=False, floatfmt=".3f"))
    else:
        logger.info("\n  Jupiter not in exaltation during this period")

# ============================================================================
# DATA QUALITY VALIDATION
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("DATA QUALITY VALIDATION")
logger.info("=" * 70)

logger.info(f"\n  Dataset shape: {df_transit.shape}")
logger.info(f"  Total features: {len(df_transit.columns)}")
logger.info(f"  Date range: {df_transit['date'].min().date()} to {df_transit['date'].max().date()}")

null_counts = df_transit.isnull().sum().sum()
logger.info(f"  Null values: {null_counts} ({null_counts / df_transit.size * 100:.2f}%)")

# Feature category counts
logger.info("\n  Feature breakdown:")
sign_features = len([col for col in df_transit.columns if '_sign' in col])
nakshatra_features = len([col for col in df_transit.columns if '_nakshatra' in col])
motion_features = len([col for col in df_transit.columns if any(x in col for x in ['_retrograde', '_stationary', '_station'])])
dignity_features = len([col for col in df_transit.columns if any(x in col for x in ['_exalted', '_debilitated', '_dignity'])])

logger.info(f"    • Zodiac sign features: {sign_features}")
logger.info(f"    • Nakshatra features: {nakshatra_features}")
logger.info(f"    • Motion features: {motion_features}")
logger.info(f"    • Dignity features: {dignity_features}")

# ============================================================================
# FINAL STATUS
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("PHASE 2 (TRANSIT FEATURES) - STATUS: COMPLETE ✓")
logger.info("=" * 70)

logger.info("\n📋 Integration Points for Downstream Phases:")
logger.info("  → Phase 3 (Model Training):")
logger.info("    • Sign/nakshatra as categorical features")
logger.info("    • Retrograde/dignity as binary flags")
logger.info("    • Dignity scores as continuous features")
logger.info("  → Phase 4 (Backtesting):")
logger.info("    • Filter on '_ingress' columns for sign change events")
logger.info("    • Filter on '_retrograde' for Mercury Rx periods")
logger.info("  → Phase 5 (Insight Extraction):")
logger.info("    • Analyze sector sensitivity to retrogrades")
logger.info("    • Identify most influential nakshatras")

logger.info("\n📋 Next Steps:")
logger.info("  1. ✓ Planetary aspects calculated (Cell 4)")
logger.info("  2. ✓ Transit & positional features (Cell 5 complete)")
logger.info("  3. ▶ Run Cell 6: Cyclic & Temporal Features")
logger.info("  4. ▶ Run Cell 7: Advanced Astrological Indicators")
logger.info("  5. ▶ Run Cell 8: Feature Integration & Final Dataset")

logger.info("\n📂 Output Files:")
logger.info(f"  {OUTPUT_FILE}")
logger.info(f"  ({len(df_transit)} rows × {len(df_transit.columns)} features)")

logger.info("\n" + "=" * 70)

In [ ]:
# Cell 6: Cyclic & Temporal Features (Phase 2 - Part 3 of 5)
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tabulate import tabulate

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 2: FEATURE ENGINEERING")
logger.info("Phase 2 Progress: Part 3 of 5 (Cyclic & Temporal)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup Paths
# ============================================================================
logger.info("\n[1/6] Setting up paths...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
ALIGNED_DATA_PATH = os.path.join(BASE_PATH, 'aligned_data')
FEATURE_DATA_PATH = os.path.join(BASE_PATH, 'feature_data')

INPUT_FILE = os.path.join(ALIGNED_DATA_PATH, 'master_aligned_dataset.parquet')
OUTPUT_FILE = os.path.join(FEATURE_DATA_PATH, 'temporal_features.parquet')

logger.info(f"  ✓ Input: master_aligned_dataset.parquet")
logger.info(f"  ✓ Output: temporal_features.parquet")

# ============================================================================
# STEP 2: Load Master Aligned Data
# ============================================================================
logger.info("\n[2/6] Loading master aligned dataset...")

if not os.path.exists(INPUT_FILE):
    logger.critical(f"\n✗ FATAL ERROR: Input file not found")
    logger.info("  Please run Cell 3 first.")
    raise SystemExit(1)

df = pd.read_parquet(INPUT_FILE)
df['date'] = pd.to_datetime(df['date'])

logger.info(f"  ✓ Loaded dataset")
logger.info(f"  ✓ Shape: {df.shape}")
logger.info(f"  ✓ Date range: {df['date'].min().date()} to {df['date'].max().date()}")

# ============================================================================
# STEP 3: Calculate Lunar Cycle Features
# ============================================================================
logger.info("\n[3/6] Calculating lunar cycle features...")

# Get Sun and Moon longitudes
sun_lon = df['sun_longitude'].values
moon_lon = df['moon_longitude'].values

# Calculate Moon phase angle (elongation from Sun)
# Phase angle = Moon longitude - Sun longitude
# 0° = New Moon, 90° = First Quarter, 180° = Full Moon, 270° = Last Quarter
moon_phase_angle = (moon_lon - sun_lon) % 360.0

# Categorize moon phase (8 phases)
# New Moon: 0° ± 22.5° (337.5° - 22.5°)
# Waxing Crescent: 22.5° - 67.5°
# First Quarter: 67.5° - 112.5°
# Waxing Gibbous: 112.5° - 157.5°
# Full Moon: 157.5° - 202.5°
# Waning Gibbous: 202.5° - 247.5°
# Last Quarter: 247.5° - 292.5°
# Waning Crescent: 292.5° - 337.5°

def get_moon_phase_category(phase_angle):
    """Convert phase angle to category (0-7)"""
    # Adjust so New Moon is centered at 0
    adjusted = (phase_angle + 22.5) % 360
    category = int(adjusted / 45)
    return category

moon_phase_category = np.array([get_moon_phase_category(angle) for angle in moon_phase_angle], dtype=np.int8)

# Phase names for reference
PHASE_NAMES = [
    'New Moon', 'Waxing Crescent', 'First Quarter', 'Waxing Gibbous',
    'Full Moon', 'Waning Gibbous', 'Last Quarter', 'Waning Crescent'
]

# Binary flags for key phases
is_new_moon = ((moon_phase_angle < 15) | (moon_phase_angle > 345)).astype(np.int8)
is_full_moon = ((moon_phase_angle > 165) & (moon_phase_angle < 195)).astype(np.int8)
is_waxing = ((moon_phase_angle > 0) & (moon_phase_angle < 180)).astype(np.int8)
is_waning = ((moon_phase_angle >= 180) & (moon_phase_angle < 360)).astype(np.int8)

# Calculate days to next New Moon and Full Moon (approximate)
# Average lunar cycle is 29.53 days
LUNAR_CYCLE_DAYS = 29.53

# Days since New Moon (phase angle / 360 * cycle length)
days_since_new_moon = (moon_phase_angle / 360.0) * LUNAR_CYCLE_DAYS

# Days until next New Moon
days_to_new_moon = LUNAR_CYCLE_DAYS - days_since_new_moon

# Days to/from Full Moon
days_to_full_moon = np.where(
    moon_phase_angle < 180,
    (180 - moon_phase_angle) / 360.0 * LUNAR_CYCLE_DAYS,  # Before full
    (540 - moon_phase_angle) / 360.0 * LUNAR_CYCLE_DAYS   # After full
)

temporal_features = {
    'date': df['date'].values,
    'moon_phase_angle': moon_phase_angle.astype(np.float32),
    'moon_phase_category': moon_phase_category,
    'is_new_moon': is_new_moon,
    'is_full_moon': is_full_moon,
    'is_waxing': is_waxing,
    'is_waning': is_waning,
    'days_since_new_moon': days_since_new_moon.astype(np.float32),
    'days_to_new_moon': days_to_new_moon.astype(np.float32),
    'days_to_full_moon': days_to_full_moon.astype(np.float32),
}

logger.info(f"  ✓ Created 10 lunar cycle features")

# ============================================================================
# STEP 4: Calculate Planetary Cycle Features
# ============================================================================
logger.info("\n[4/6] Calculating planetary cycle features...")

# Calculate days in current sign for slower planets (important for timing)
SLOW_PLANETS = ['jupiter', 'saturn']  # These stay in signs for months/years

for planet in SLOW_PLANETS:
    lon_col = f'{planet}_longitude'
    speed_col = f'{planet}_speed'

    if lon_col not in df.columns or speed_col not in df.columns:
        continue

    longitude = df[lon_col].values
    speed = df[speed_col].values

    # Calculate degrees within current sign (0-30)
    degrees_in_sign = longitude % 30

    # Estimate days in current sign (degrees traveled / daily speed)
    # Protect against division by zero
    safe_speed = np.where(np.abs(speed) < 0.001, 0.001, speed)
    days_in_sign = degrees_in_sign / np.abs(safe_speed)

    # Estimate days until sign change (degrees remaining / daily speed)
    degrees_remaining = 30 - degrees_in_sign
    days_to_sign_change = degrees_remaining / np.abs(safe_speed)

    # Cap at reasonable values (max 365 days)
    days_in_sign = np.clip(days_in_sign, 0, 365).astype(np.float32)
    days_to_sign_change = np.clip(days_to_sign_change, 0, 365).astype(np.float32)

    temporal_features[f'{planet}_days_in_sign'] = days_in_sign
    temporal_features[f'{planet}_days_to_sign_change'] = days_to_sign_change

logger.info(f"  ✓ Created planetary cycle features for {len(SLOW_PLANETS)} planets")

# ============================================================================
# STEP 5: Calculate Calendar-Based Features
# ============================================================================
logger.info("\n[5/6] Calculating calendar-based features...")

dates = df['date']

# Day of week (0=Monday, 6=Sunday)
day_of_week = dates.dt.dayofweek.values.astype(np.int8)

# Is weekend?
is_weekend = (day_of_week >= 5).astype(np.int8)

# Month (1-12)
month = dates.dt.month.values.astype(np.int8)

# Quarter (1-4)
quarter = dates.dt.quarter.values.astype(np.int8)

# Day of month (1-31)
day_of_month = dates.dt.day.values.astype(np.int8)

# Week of year (1-53)
week_of_year = dates.dt.isocalendar().week.values.astype(np.int8)

# Year (for trend analysis)
year = dates.dt.year.values.astype(np.int16)

# Cyclical encodings for periodic features (sin/cos transformation)
# This helps ML models understand cyclical nature (Dec and Jan are close)

# Month cyclical encoding
month_sin = np.sin(2 * np.pi * month / 12).astype(np.float32)
month_cos = np.cos(2 * np.pi * month / 12).astype(np.float32)

# Day of week cyclical encoding
dow_sin = np.sin(2 * np.pi * day_of_week / 7).astype(np.float32)
dow_cos = np.cos(2 * np.pi * day_of_week / 7).astype(np.float32)

# Add to features
temporal_features.update({
    'day_of_week': day_of_week,
    'is_weekend': is_weekend,
    'month': month,
    'quarter': quarter,
    'day_of_month': day_of_month,
    'week_of_year': week_of_year,
    'year': year,
    'month_sin': month_sin,
    'month_cos': month_cos,
    'dow_sin': dow_sin,
    'dow_cos': dow_cos,
})

logger.info(f"  ✓ Created 12 calendar-based features")

# ============================================================================
# STEP 6: Calculate Mercury Retrograde Periods (Special Feature)
# ============================================================================
logger.info("\n[6/6] Calculating Mercury retrograde periods...")

# Mercury retrograde is famous in astrology for communication/tech disruptions
if 'mercury_speed' in df.columns:
    mercury_speed = df['mercury_speed'].values

    # Is Mercury retrograde?
    mercury_rx = (mercury_speed < 0).astype(np.int8)

    # Calculate consecutive days of Mercury Rx
    mercury_rx_days = np.zeros(len(mercury_rx), dtype=np.int16)

    count = 0
    for i in range(len(mercury_rx)):
        if mercury_rx[i] == 1:
            count += 1
            mercury_rx_days[i] = count
        else:
            count = 0

    # Days until next Mercury retrograde (approximate)
    # Find next Rx period for each day
    days_to_mercury_rx = np.zeros(len(mercury_rx), dtype=np.int16)

    for i in range(len(mercury_rx)):
        if mercury_rx[i] == 1:
            days_to_mercury_rx[i] = 0  # Already in Rx
        else:
            # Look ahead to find next Rx
            found = False
            for j in range(i + 1, min(i + 120, len(mercury_rx))):  # Look ahead max 120 days
                if mercury_rx[j] == 1:
                    days_to_mercury_rx[i] = j - i
                    found = True
                    break
            if not found:
                days_to_mercury_rx[i] = 120  # Cap at 120 if not found

    temporal_features['mercury_retrograde'] = mercury_rx
    temporal_features['mercury_rx_day_count'] = mercury_rx_days
    temporal_features['days_to_mercury_rx'] = days_to_mercury_rx

    logger.info(f"  ✓ Created 3 Mercury retrograde features")
else:
    logger.warning(f"  ⚠ Skipping Mercury Rx features - speed column not found")

# ============================================================================
# STEP 7: Create DataFrame and Save
# ============================================================================
logger.info(f"\n[7/7] Saving temporal features...")

df_temporal = pd.DataFrame(temporal_features)

try:
    df_temporal.to_parquet(OUTPUT_FILE, index=False, engine='pyarrow')
    file_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
    logger.info(f"  ✓ Saved: temporal_features.parquet")
    logger.info(f"  ✓ File size: {file_size_mb:.2f} MB")
except Exception as e:
    logger.critical(f"\n✗ FATAL ERROR: Could not save file")
    logger.critical(f"  Error: {e}")
    raise SystemExit(1)

# ============================================================================
# VALIDATION & SAMPLE OUTPUT
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Lunar Cycle Features (First 10 Days)")
logger.info("=" * 70)

sample_cols = ['date', 'moon_phase_angle', 'moon_phase_category',
               'is_new_moon', 'is_full_moon', 'days_to_new_moon', 'days_to_full_moon']

existing_cols = [col for col in sample_cols if col in df_temporal.columns]
sample_data = df_temporal[existing_cols].head(10).copy()

# Add phase name for readability
if 'moon_phase_category' in sample_data.columns:
    sample_data['phase_name'] = sample_data['moon_phase_category'].apply(
        lambda x: PHASE_NAMES[x] if 0 <= x < 8 else 'Unknown'
    )

print("\n" + tabulate(sample_data, headers='keys', tablefmt='grid',
                     showindex=False, floatfmt=".1f"))

logger.info("\n" + "=" * 70)
logger.info("SAMPLE: New Moon and Full Moon Events")
logger.info("=" * 70)

# Find actual New Moon and Full Moon days
new_moon_days = df_temporal[df_temporal['is_new_moon'] == 1]
full_moon_days = df_temporal[df_temporal['is_full_moon'] == 1]

logger.info(f"\nNew Moons in dataset: {len(new_moon_days)} days")
logger.info(f"Full Moons in dataset: {len(full_moon_days)} days")

if len(new_moon_days) > 0:
    logger.info("\nFirst 5 New Moon dates:")
    nm_cols = ['date', 'moon_phase_angle', 'days_since_new_moon']
    print("\n" + tabulate(new_moon_days[nm_cols].head(5),
                         headers='keys', tablefmt='grid',
                         showindex=False, floatfmt=".2f"))

if len(full_moon_days) > 0:
    logger.info("\nFirst 5 Full Moon dates:")
    fm_cols = ['date', 'moon_phase_angle', 'days_to_full_moon']
    print("\n" + tabulate(full_moon_days[fm_cols].head(5),
                         headers='keys', tablefmt='grid',
                         showindex=False, floatfmt=".2f"))

logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Mercury Retrograde Periods")
logger.info("=" * 70)

if 'mercury_retrograde' in df_temporal.columns:
    mercury_rx_periods = df_temporal[df_temporal['mercury_retrograde'] == 1]

    logger.info(f"\nMercury retrograde days: {len(mercury_rx_periods)} ({len(mercury_rx_periods)/len(df_temporal)*100:.1f}%)")

    if len(mercury_rx_periods) > 0:
        # Find start of retrograde periods
        rx_starts = mercury_rx_periods[mercury_rx_periods['mercury_rx_day_count'] == 1]

        logger.info(f"Number of Mercury Rx periods: {len(rx_starts)}")
        logger.info("\nFirst 5 Mercury Rx period start dates:")

        rx_cols = ['date', 'mercury_rx_day_count', 'days_to_mercury_rx']
        print("\n" + tabulate(rx_starts[rx_cols].head(5),
                             headers='keys', tablefmt='grid',
                             showindex=False))

logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Calendar Features (First 5 Days)")
logger.info("=" * 70)

calendar_cols = ['date', 'day_of_week', 'is_weekend', 'month', 'quarter', 'year']
existing_cal = [col for col in calendar_cols if col in df_temporal.columns]

calendar_sample = df_temporal[existing_cal].head(5).copy()

# Add day name
if 'day_of_week' in calendar_sample.columns:
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    calendar_sample['day_name'] = calendar_sample['day_of_week'].apply(
        lambda x: day_names[x] if 0 <= x < 7 else 'Unknown'
    )

print("\n" + tabulate(calendar_sample, headers='keys', tablefmt='grid', showindex=False))

# ============================================================================
# DATA QUALITY VALIDATION
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("DATA QUALITY VALIDATION")
logger.info("=" * 70)

logger.info(f"\n  Dataset shape: {df_temporal.shape}")
logger.info(f"  Total features: {len(df_temporal.columns)}")
logger.info(f"  Date range: {df_temporal['date'].min().date()} to {df_temporal['date'].max().date()}")

null_counts = df_temporal.isnull().sum().sum()
logger.info(f"  Null values: {null_counts} ({null_counts / df_temporal.size * 100:.2f}%)")

# Feature statistics
logger.info("\n  Feature breakdown:")
lunar_features = len([col for col in df_temporal.columns if 'moon' in col or 'lunar' in col])
calendar_features = len([col for col in df_temporal.columns if any(x in col for x in ['day', 'week', 'month', 'quarter', 'year', 'weekend'])])
mercury_features = len([col for col in df_temporal.columns if 'mercury' in col])
cycle_features = len([col for col in df_temporal.columns if 'days_in' in col or 'days_to' in col])

logger.info(f"    • Lunar cycle features: {lunar_features}")
logger.info(f"    • Calendar features: {calendar_features}")
logger.info(f"    • Mercury Rx features: {mercury_features}")
logger.info(f"    • Planetary cycle features: {cycle_features}")

# Value ranges
logger.info("\n  Value ranges:")
if 'moon_phase_angle' in df_temporal.columns:
    logger.info(f"    • Moon phase angle: {df_temporal['moon_phase_angle'].min():.1f}° to {df_temporal['moon_phase_angle'].max():.1f}°")

if 'days_to_new_moon' in df_temporal.columns:
    logger.info(f"    • Days to new moon: {df_temporal['days_to_new_moon'].min():.1f} to {df_temporal['days_to_new_moon'].max():.1f}")

# ============================================================================
# FINAL STATUS
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("PHASE 2 (TEMPORAL FEATURES) - STATUS: COMPLETE ✓")
logger.info("=" * 70)

logger.info("\n📋 Integration Points for Downstream Phases:")
logger.info("  → Phase 3 (Model Training):")
logger.info("    • Moon phase as cyclical feature (angle + category)")
logger.info("    • Calendar features for seasonal patterns")
logger.info("    • Mercury Rx as binary classification feature")
logger.info("  → Phase 4 (Backtesting):")
logger.info("    • Filter on 'is_new_moon'/'is_full_moon' for lunar event studies")
logger.info("    • Isolate Mercury Rx periods for sector analysis")
logger.info("  → Phase 5 (Insight Extraction):")
logger.info("    • Quantify Mercury Rx impact per sector")
logger.info("    • Identify most sensitive moon phases")

logger.info("\n📋 Next Steps:")
logger.info("  1. ✓ Planetary aspects calculated (Cell 4)")
logger.info("  2. ✓ Transit & positional features (Cell 5)")
logger.info("  3. ✓ Cyclic & temporal features (Cell 6 complete)")
logger.info("  4. ▶ Run Cell 7: Advanced Astrological Indicators")
logger.info("  5. ▶ Run Cell 8: Feature Integration & Final Dataset")

logger.info("\n📂 Output Files:")
logger.info(f"  {OUTPUT_FILE}")
logger.info(f"  ({len(df_temporal)} rows × {len(df_temporal.columns)} features)")

logger.info("\n" + "=" * 70)

In [ ]:
# Cell 7: Advanced Astrological Indicators (Phase 2 - Part 4 of 5)
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
from datetime import datetime
from tabulate import tabulate
import itertools

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 2: FEATURE ENGINEERING")
logger.info("Phase 2 Progress: Part 4 of 5 (Advanced Indicators)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup Paths
# ============================================================================
logger.info("\n[1/6] Setting up paths...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
ALIGNED_DATA_PATH = os.path.join(BASE_PATH, 'aligned_data')
FEATURE_DATA_PATH = os.path.join(BASE_PATH, 'feature_data')

INPUT_FILE = os.path.join(ALIGNED_DATA_PATH, 'master_aligned_dataset.parquet')
OUTPUT_FILE = os.path.join(FEATURE_DATA_PATH, 'advanced_features.parquet')

logger.info(f"  ✓ Input: master_aligned_dataset.parquet")
logger.info(f"  ✓ Output: advanced_features.parquet")

# ============================================================================
# STEP 2: Load Master Aligned Data
# ============================================================================
logger.info("\n[2/6] Loading master aligned dataset...")

if not os.path.exists(INPUT_FILE):
    logger.critical(f"\n✗ FATAL ERROR: Input file not found")
    logger.info("  Please run Cell 3 first.")
    raise SystemExit(1)

df = pd.read_parquet(INPUT_FILE)
df['date'] = pd.to_datetime(df['date'])

logger.info(f"  ✓ Loaded dataset")
logger.info(f"  ✓ Shape: {df.shape}")
logger.info(f"  ✓ Date range: {df['date'].min().date()} to {df['date'].max().date()}")

# ============================================================================
# STEP 3: Define Constants and Helper Functions
# ============================================================================
logger.info("\n[3/6] Defining advanced astrological functions...")

PLANETS = ['sun', 'moon', 'mercury', 'venus', 'mars',
           'jupiter', 'saturn', 'rahu']

# Harmonic aspects (minor aspects)
MINOR_ASPECTS = {
    'semisextile': 30,      # Mild friction
    'semisquare': 45,       # Mild tension
    'quintile': 72,         # Creativity
    'sesquiquadrate': 135,  # Adjustment
    'quincunx': 150,        # Requires adaptation
}

# Orb for minor aspects (tighter than major)
MINOR_ASPECT_ORB = 2.0

def normalize_angle(angle):
    """Normalize angle to 0-360 range."""
    return angle % 360.0

def angular_distance(lon1, lon2):
    """Calculate shortest angular distance (0-180°)."""
    diff = np.abs(lon1 - lon2)
    return np.where(diff > 180, 360.0 - diff, diff)

def calculate_midpoint(lon1, lon2):
    """
    Calculate midpoint between two planets.
    Returns both the direct midpoint and far midpoint (180° opposite).
    Vectorized to handle arrays.
    """
    lon1 = np.asarray(lon1)
    lon2 = np.asarray(lon2)

    # Direct midpoint (shorter arc)
    direct = (lon1 + lon2) / 2.0

    # Handle wrap-around case (when planets are on opposite sides of 0°)
    # If distance > 180°, the midpoint is on the other side
    needs_adjustment = np.abs(lon1 - lon2) > 180
    direct = np.where(needs_adjustment, (direct + 180) % 360, direct)

    direct = normalize_angle(direct)

    # Far midpoint (opposite point)
    far = normalize_angle(direct + 180)

    return direct, far

def check_midpoint_activation(planet_lon, midpoint_lon, orb=3.0):
    """
    Check if a planet is conjunct a midpoint.
    Returns binary flag and exactness score.
    """
    distance = angular_distance(planet_lon, midpoint_lon)
    is_active = (distance <= orb).astype(np.int8)

    # Exactness: 1.0 at exact, 0.0 at orb limit
    exactness = np.maximum(0, 1 - (distance / orb)).astype(np.float32)

    return is_active, exactness

logger.info("  ✓ Midpoint calculation functions")
logger.info("  ✓ Harmonic aspect definitions")

# ============================================================================
# STEP 4: Calculate Planetary Midpoints
# ============================================================================
logger.info("\n[4/6] Calculating planetary midpoints...")

advanced_features = {'date': df['date'].values}
feature_count = 0

# Focus on important midpoint pairs (not all combinations)
IMPORTANT_MIDPOINT_PAIRS = [
    ('sun', 'moon'),      # Personality integration
    ('sun', 'mercury'),   # Communication of identity
    ('sun', 'venus'),     # Values and pleasure
    ('venus', 'mars'),    # Passion and desire
    ('jupiter', 'saturn'), # Expansion vs contraction
]

for p1, p2 in IMPORTANT_MIDPOINT_PAIRS:
    if f'{p1}_longitude' not in df.columns or f'{p2}_longitude' not in df.columns:
        continue

    lon1 = df[f'{p1}_longitude'].values
    lon2 = df[f'{p2}_longitude'].values

    # Calculate midpoint
    midpoint, far_midpoint = calculate_midpoint(lon1, lon2)

    # Store midpoint longitude
    advanced_features[f'{p1}_{p2}_midpoint'] = midpoint.astype(np.float32)
    feature_count += 1

    # Check if other planets activate this midpoint
    for planet in ['mars', 'jupiter', 'saturn']:
        if planet in [p1, p2]:
            continue

        if f'{planet}_longitude' not in df.columns:
            continue

        planet_lon = df[f'{planet}_longitude'].values

        # Check activation
        is_active, exactness = check_midpoint_activation(planet_lon, midpoint)

        advanced_features[f'{planet}_on_{p1}_{p2}_midpoint'] = is_active
        advanced_features[f'{planet}_on_{p1}_{p2}_midpoint_exact'] = exactness
        feature_count += 2

logger.info(f"  ✓ Created {feature_count} midpoint features")

# ============================================================================
# STEP 5: Calculate Minor/Harmonic Aspects
# ============================================================================
logger.info("\n[5/6] Calculating minor harmonic aspects...")

# Focus on key planet pairs for minor aspects
KEY_MINOR_PAIRS = [
    ('mercury', 'venus'),
    ('mercury', 'mars'),
    ('venus', 'mars'),
    ('mars', 'jupiter'),
    ('mars', 'saturn'),
    ('jupiter', 'saturn'),
]

for p1, p2 in KEY_MINOR_PAIRS:
    if f'{p1}_longitude' not in df.columns or f'{p2}_longitude' not in df.columns:
        continue

    lon1 = df[f'{p1}_longitude'].values
    lon2 = df[f'{p2}_longitude'].values

    # Calculate angular distance
    ang_dist = angular_distance(lon1, lon2)

    for aspect_name, aspect_angle in MINOR_ASPECTS.items():
        # Distance from exact minor aspect
        dist_from_exact = np.abs(ang_dist - aspect_angle)
        dist_from_exact = np.minimum(dist_from_exact, 360 - dist_from_exact)

        # Is aspect active?
        is_active = (dist_from_exact <= MINOR_ASPECT_ORB).astype(np.int8)

        # Exactness score
        exactness = np.where(
            is_active,
            (1 - (dist_from_exact / MINOR_ASPECT_ORB)).astype(np.float32),
            0.0
        )

        advanced_features[f'{p1}_{p2}_{aspect_name}'] = is_active
        advanced_features[f'{p1}_{p2}_{aspect_name}_exact'] = exactness
        feature_count += 2

logger.info(f"  ✓ Created minor aspect features for {len(KEY_MINOR_PAIRS)} pairs")

# ============================================================================
# STEP 6: Calculate Composite Daily Indicators
# ============================================================================
logger.info("\n[6/6] Calculating composite daily indicators...")

# We'll need to load aspect data from Cell 4 to create composites
ASPECTS_FILE = os.path.join(FEATURE_DATA_PATH, 'aspects_features.parquet')

if os.path.exists(ASPECTS_FILE):
    logger.info("  → Loading aspect features from Cell 4...")
    df_aspects = pd.read_parquet(ASPECTS_FILE)

    # Ensure dates match
    if len(df_aspects) == len(df):
        # 1. Benefic Aspect Score (positive aspects)
        benefic_cols = [col for col in df_aspects.columns if any(
            x in col for x in ['trine_strength', 'sextile_strength']
        )]

        if benefic_cols:
            benefic_score = df_aspects[benefic_cols].sum(axis=1).values
            advanced_features['daily_benefic_score'] = benefic_score.astype(np.float32)
            feature_count += 1

        # 2. Malefic Aspect Score (challenging aspects)
        malefic_cols = [col for col in df_aspects.columns if any(
            x in col for x in ['square_strength', 'opposition_strength']
        )]

        if malefic_cols:
            malefic_score = df_aspects[malefic_cols].sum(axis=1).values
            advanced_features['daily_malefic_score'] = malefic_score.astype(np.float32)
            feature_count += 1

        # 3. Net Aspect Quality (benefic - malefic)
        if 'daily_benefic_score' in advanced_features and 'daily_malefic_score' in advanced_features:
            net_quality = (advanced_features['daily_benefic_score'] -
                          advanced_features['daily_malefic_score'])
            advanced_features['daily_aspect_quality'] = net_quality.astype(np.float32)
            feature_count += 1

        # 4. Aspect Intensity (total aspect strength regardless of type)
        all_strength_cols = [col for col in df_aspects.columns if col.endswith('_strength')]
        if all_strength_cols:
            total_intensity = df_aspects[all_strength_cols].sum(axis=1).values
            advanced_features['daily_aspect_intensity'] = total_intensity.astype(np.float32)
            feature_count += 1

        # 5. Harsh Aspect Count (Mars/Saturn involved)
        harsh_cols = [col for col in df_aspects.columns if
                     col.endswith('_active') and any(x in col for x in ['mars_saturn', 'saturn_'])]

        if harsh_cols:
            harsh_count = df_aspects[harsh_cols].sum(axis=1).values
            advanced_features['daily_harsh_aspect_count'] = harsh_count.astype(np.int8)
            feature_count += 1

        logger.info(f"  ✓ Created {5} composite aspect indicators")
    else:
        logger.warning(f"  ⚠ Aspect data length mismatch - skipping composites")
else:
    logger.warning(f"  ⚠ Aspect features file not found - skipping composites")
    logger.info(f"    Run Cell 4 first to generate aspect features")

# ============================================================================
# STEP 7: Calculate Planetary Strength Scores
# ============================================================================
logger.info("\n[7/7] Calculating planetary strength scores...")

# Planetary strength based on multiple factors
for planet in ['sun', 'moon', 'jupiter', 'venus', 'mars', 'mercury', 'saturn']:
    if f'{planet}_longitude' not in df.columns:
        continue

    # Initialize strength score (0-100 scale)
    strength = np.zeros(len(df), dtype=np.float32)

    # Factor 1: Speed (faster = stronger, but not for retrograde)
    if f'{planet}_speed' in df.columns:
        speed = df[f'{planet}_speed'].values

        # Normalize speed to 0-20 points
        if planet == 'sun':
            max_speed = 1.2
        elif planet == 'moon':
            max_speed = 15.0
        elif planet in ['mercury', 'venus']:
            max_speed = 2.0
        else:
            max_speed = 0.3

        speed_score = np.clip((np.abs(speed) / max_speed) * 20, 0, 20)

        # Penalty for retrograde (except Rahu which is always Rx)
        if planet != 'rahu':
            speed_score = np.where(speed < 0, speed_score * 0.5, speed_score)

        strength += speed_score

    # Factor 2: Dignity (from Cell 5 if available)
    # We'll load transit features to check exaltation/debilitation
    # For now, use a simplified approach based on position

    # Factor 3: Aspect reception (simplified - receives benefic aspects)
    # This would require aspect data - skip for now or add bonus

    # Factor 4: House position quality (we don't have houses, skip)

    # Normalize to 0-100 scale
    strength = (strength / 20) * 100
    strength = np.clip(strength, 0, 100).astype(np.float32)

    advanced_features[f'{planet}_strength_score'] = strength
    feature_count += 1

logger.info(f"  ✓ Created strength scores for 7 planets")

# ============================================================================
# STEP 8: Create DataFrame and Save
# ============================================================================
logger.info(f"\n[8/8] Saving advanced features...")

df_advanced = pd.DataFrame(advanced_features)

try:
    df_advanced.to_parquet(OUTPUT_FILE, index=False, engine='pyarrow')
    file_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
    logger.info(f"  ✓ Saved: advanced_features.parquet")
    logger.info(f"  ✓ File size: {file_size_mb:.2f} MB")
except Exception as e:
    logger.critical(f"\n✗ FATAL ERROR: Could not save file")
    logger.critical(f"  Error: {e}")
    raise SystemExit(1)

# ============================================================================
# VALIDATION & SAMPLE OUTPUT
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Planetary Midpoints (First 5 Days)")
logger.info("=" * 70)

midpoint_cols = ['date'] + [col for col in df_advanced.columns if '_midpoint' in col and not 'on_' in col][:5]
existing_mp = [col for col in midpoint_cols if col in df_advanced.columns]

if len(existing_mp) > 1:
    print("\n" + tabulate(df_advanced[existing_mp].head(5),
                         headers='keys', tablefmt='grid',
                         showindex=False, floatfmt=".2f"))
else:
    logger.info("\n  No midpoint features created")

logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Composite Daily Indicators (First 5 Days)")
logger.info("=" * 70)

composite_cols = ['date', 'daily_benefic_score', 'daily_malefic_score',
                  'daily_aspect_quality', 'daily_aspect_intensity']
existing_comp = [col for col in composite_cols if col in df_advanced.columns]

if len(existing_comp) > 1:
    print("\n" + tabulate(df_advanced[existing_comp].head(5),
                         headers='keys', tablefmt='grid',
                         showindex=False, floatfmt=".2f"))
else:
    logger.info("\n  No composite indicators created (run Cell 4 first)")

logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Planetary Strength Scores (First 5 Days)")
logger.info("=" * 70)

strength_cols = ['date'] + [col for col in df_advanced.columns if '_strength_score' in col][:5]
existing_strength = [col for col in strength_cols if col in df_advanced.columns]

if len(existing_strength) > 1:
    print("\n" + tabulate(df_advanced[existing_strength].head(5),
                         headers='keys', tablefmt='grid',
                         showindex=False, floatfmt=".1f"))
else:
    logger.info("\n  No strength scores created")

# ============================================================================
# DATA QUALITY VALIDATION
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("DATA QUALITY VALIDATION")
logger.info("=" * 70)

logger.info(f"\n  Dataset shape: {df_advanced.shape}")
logger.info(f"  Total features: {len(df_advanced.columns)}")
logger.info(f"  Date range: {df_advanced['date'].min().date()} to {df_advanced['date'].max().date()}")

null_counts = df_advanced.isnull().sum().sum()
logger.info(f"  Null values: {null_counts} ({null_counts / df_advanced.size * 100:.2f}%)")

# Feature breakdown
logger.info("\n  Feature breakdown:")
midpoint_features = len([col for col in df_advanced.columns if 'midpoint' in col])
minor_aspect_features = len([col for col in df_advanced.columns if any(
    asp in col for asp in MINOR_ASPECTS.keys()
)])
composite_features = len([col for col in df_advanced.columns if 'daily_' in col])
strength_features = len([col for col in df_advanced.columns if '_strength_score' in col])

logger.info(f"    • Midpoint features: {midpoint_features}")
logger.info(f"    • Minor aspect features: {minor_aspect_features}")
logger.info(f"    • Composite indicators: {composite_features}")
logger.info(f"    • Strength scores: {strength_features}")

# Value ranges for composite indicators
if 'daily_aspect_quality' in df_advanced.columns:
    quality = df_advanced['daily_aspect_quality']
    logger.info(f"\n  Daily aspect quality range: {quality.min():.2f} to {quality.max():.2f}")
    logger.info(f"  Average quality: {quality.mean():.2f}")

# ============================================================================
# FINAL STATUS
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("PHASE 2 (ADVANCED FEATURES) - STATUS: COMPLETE ✓")
logger.info("=" * 70)

logger.info("\n📋 Integration Points for Downstream Phases:")
logger.info("  → Phase 3 (Model Training):")
logger.info("    • Midpoint activations as binary features")
logger.info("    • Minor aspects for nuanced pattern detection")
logger.info("    • Composite scores as continuous features")
logger.info("    • Strength scores for planet weighting")
logger.info("  → Phase 4 (Backtesting):")
logger.info("    • Filter on composite indicators (high benefic days)")
logger.info("    • Isolate specific midpoint activations")
logger.info("  → Phase 5 (Insight Extraction):")
logger.info("    • Identify most influential midpoints per sector")
logger.info("    • Quantify impact of minor aspects")
logger.info("    • Rank planets by strength score correlation")

logger.info("\n📋 Next Steps:")
logger.info("  1. ✓ Planetary aspects calculated (Cell 4)")
logger.info("  2. ✓ Transit & positional features (Cell 5)")
logger.info("  3. ✓ Cyclic & temporal features (Cell 6)")
logger.info("  4. ✓ Advanced astrological indicators (Cell 7 complete)")
logger.info("  5. ▶ Run Cell 8: Feature Integration & Final Dataset")

logger.info("\n📂 Output Files:")
logger.info(f"  {OUTPUT_FILE}")
logger.info(f"  ({len(df_advanced)} rows × {len(df_advanced.columns)} features)")

logger.info("\n" + "=" * 70)

In [ ]:
# Cell 8: Feature Integration & Final Dataset (Phase 2 - Part 5 of 5) - FIXED
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
from datetime import datetime
from tabulate import tabulate
import json

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 2: FEATURE ENGINEERING")
logger.info("Phase 2 Progress: Part 5 of 5 (Feature Integration)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup Paths
# ============================================================================
logger.info("\n[1/8] Setting up paths...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
ALIGNED_DATA_PATH = os.path.join(BASE_PATH, 'aligned_data')
FEATURE_DATA_PATH = os.path.join(BASE_PATH, 'feature_data')

# Input files
MASTER_FILE = os.path.join(ALIGNED_DATA_PATH, 'master_aligned_dataset.parquet')
ASPECTS_FILE = os.path.join(FEATURE_DATA_PATH, 'aspects_features.parquet')
TRANSIT_FILE = os.path.join(FEATURE_DATA_PATH, 'transit_features.parquet')
TEMPORAL_FILE = os.path.join(FEATURE_DATA_PATH, 'temporal_features.parquet')
ADVANCED_FILE = os.path.join(FEATURE_DATA_PATH, 'advanced_features.parquet')

# Output files
OUTPUT_FILE = os.path.join(FEATURE_DATA_PATH, 'master_features_dataset.parquet')
CATALOG_FILE = os.path.join(FEATURE_DATA_PATH, 'feature_catalog.csv')
METADATA_FILE = os.path.join(FEATURE_DATA_PATH, 'dataset_metadata.json')
SPLITS_FILE = os.path.join(FEATURE_DATA_PATH, 'train_val_test_splits.json')

logger.info(f"  ✓ Input files configured (4 feature sets)")
logger.info(f"  ✓ Output files configured")

# ============================================================================
# STEP 2: Load All Feature Sets
# ============================================================================
logger.info("\n[2/8] Loading all feature datasets...")

feature_sets = {}
load_status = []

# Load master aligned dataset (base)
if os.path.exists(MASTER_FILE):
    df_master = pd.read_parquet(MASTER_FILE)
    df_master['date'] = pd.to_datetime(df_master['date'])
    feature_sets['master'] = df_master
    load_status.append(('Master Dataset', len(df_master.columns), '✓'))
    logger.info(f"  ✓ Loaded master dataset: {df_master.shape}")
else:
    logger.critical(f"\n✗ FATAL ERROR: Master dataset not found")
    raise SystemExit(1)

# Load aspect features
if os.path.exists(ASPECTS_FILE):
    df_aspects = pd.read_parquet(ASPECTS_FILE)
    df_aspects['date'] = pd.to_datetime(df_aspects['date'])
    feature_sets['aspects'] = df_aspects
    load_status.append(('Aspect Features', len(df_aspects.columns), '✓'))
    logger.info(f"  ✓ Loaded aspect features: {df_aspects.shape}")
else:
    logger.warning(f"  ⚠ Aspect features not found - skipping")
    load_status.append(('Aspect Features', 0, '✗'))

# Load transit features
if os.path.exists(TRANSIT_FILE):
    df_transit = pd.read_parquet(TRANSIT_FILE)
    df_transit['date'] = pd.to_datetime(df_transit['date'])
    feature_sets['transit'] = df_transit
    load_status.append(('Transit Features', len(df_transit.columns), '✓'))
    logger.info(f"  ✓ Loaded transit features: {df_transit.shape}")
else:
    logger.warning(f"  ⚠ Transit features not found - skipping")
    load_status.append(('Transit Features', 0, '✗'))

# Load temporal features
if os.path.exists(TEMPORAL_FILE):
    df_temporal = pd.read_parquet(TEMPORAL_FILE)
    df_temporal['date'] = pd.to_datetime(df_temporal['date'])
    feature_sets['temporal'] = df_temporal
    load_status.append(('Temporal Features', len(df_temporal.columns), '✓'))
    logger.info(f"  ✓ Loaded temporal features: {df_temporal.shape}")
else:
    logger.warning(f"  ⚠ Temporal features not found - skipping")
    load_status.append(('Temporal Features', 0, '✗'))

# Load advanced features
if os.path.exists(ADVANCED_FILE):
    df_advanced = pd.read_parquet(ADVANCED_FILE)
    df_advanced['date'] = pd.to_datetime(df_advanced['date'])
    feature_sets['advanced'] = df_advanced
    load_status.append(('Advanced Features', len(df_advanced.columns), '✓'))
    logger.info(f"  ✓ Loaded advanced features: {df_advanced.shape}")
else:
    logger.warning(f"  ⚠ Advanced features not found - skipping")
    load_status.append(('Advanced Features', 0, '✗'))

logger.info("\n  Load Status Summary:")
for name, count, status in load_status:
    logger.info(f"    {status} {name}: {count} features")

# ============================================================================
# STEP 3: Merge All Feature Sets (WITH DUPLICATE HANDLING)
# ============================================================================
logger.info("\n[3/8] Merging all feature sets...")

# Start with master dataset
df_merged = df_master.copy()
logger.info(f"  Starting with master: {df_merged.shape}")

# Track columns to avoid duplicates
existing_columns = set(df_merged.columns)

# Merge each feature set on 'date'
for name, df_features in feature_sets.items():
    if name == 'master':
        continue

    # Get all columns except 'date'
    feature_cols = [col for col in df_features.columns if col != 'date']

    # CRITICAL FIX: Filter out columns that already exist
    new_feature_cols = [col for col in feature_cols if col not in existing_columns]
    duplicate_cols = [col for col in feature_cols if col in existing_columns]

    if duplicate_cols:
        logger.warning(f"  ⚠ Skipping {len(duplicate_cols)} duplicate columns from {name}: {duplicate_cols[:5]}...")

    if not new_feature_cols:
        logger.warning(f"  ⚠ No new columns to add from {name}")
        continue

    # Merge only new columns
    df_merged = pd.merge(
        df_merged,
        df_features[['date'] + new_feature_cols],
        on='date',
        how='left',
        suffixes=('', f'_{name}')  # Add suffix if still somehow duplicates
    )

    # Update existing columns set
    existing_columns.update(new_feature_cols)

    logger.info(f"  + Merged {name}: added {len(new_feature_cols)} columns → {df_merged.shape}")

# CRITICAL: Remove any remaining duplicate columns
duplicate_cols = df_merged.columns[df_merged.columns.duplicated()].tolist()
if duplicate_cols:
    logger.warning(f"\n  ⚠ WARNING: Found {len(duplicate_cols)} duplicate columns after merge")
    logger.info(f"    Removing duplicates: {duplicate_cols[:10]}...")
    df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]
    logger.info(f"    After cleanup: {df_merged.shape}")

logger.info(f"\n  ✓ Final merged dataset: {df_merged.shape}")

# ============================================================================
# STEP 4: Quality Checks and Cleanup
# ============================================================================
logger.info("\n[4/8] Performing quality checks...")

# Check for duplicates
duplicates = df_merged.duplicated(subset=['date']).sum()
logger.info(f"  • Duplicate dates: {duplicates}")

if duplicates > 0:
    logger.info(f"    Removing {duplicates} duplicate rows...")
    df_merged = df_merged.drop_duplicates(subset=['date'], keep='first')

# Check for null values
null_counts = df_merged.isnull().sum()
total_nulls = null_counts.sum()
null_pct = (total_nulls / df_merged.size) * 100

logger.info(f"  • Total null values: {total_nulls} ({null_pct:.2f}%)")

if total_nulls > 0:
    # Show columns with most nulls
    top_nulls = null_counts[null_counts > 0].sort_values(ascending=False).head(10)
    logger.info(f"\n  Top columns with nulls:")
    for col, count in top_nulls.items():
        pct = (count / len(df_merged)) * 100
        logger.info(f"    • {col}: {count} ({pct:.1f}%)")

# Sort by date
df_merged = df_merged.sort_values('date').reset_index(drop=True)
logger.info(f"\n  ✓ Data cleaned and sorted")

# ============================================================================
# STEP 5: Create Train/Validation/Test Splits
# ============================================================================
logger.info("\n[5/8] Creating train/validation/test splits...")

# Define split dates
train_end = pd.Timestamp('2020-12-31')
val_end = pd.Timestamp('2023-12-31')

train_mask = df_merged['date'] <= train_end
val_mask = (df_merged['date'] > train_end) & (df_merged['date'] <= val_end)
test_mask = df_merged['date'] > val_end

train_size = train_mask.sum()
val_size = val_mask.sum()
test_size = test_mask.sum()

train_pct = (train_size / len(df_merged)) * 100
val_pct = (val_size / len(df_merged)) * 100
test_pct = (test_size / len(df_merged)) * 100

logger.info(f"\n  Split distribution:")
logger.info(f"    • Train: {train_size} rows ({train_pct:.1f}%) | 2000-01-01 to 2020-12-31")
logger.info(f"    • Val:   {val_size} rows ({val_pct:.1f}%) | 2021-01-01 to 2023-12-31")
logger.info(f"    • Test:  {test_size} rows ({test_pct:.1f}%) | 2024-01-01 to 2025-10-29")

# Save split metadata
splits_metadata = {
    'train': {
        'start_date': df_merged[train_mask]['date'].min().strftime('%Y-%m-%d') if train_size > 0 else 'N/A',
        'end_date': df_merged[train_mask]['date'].max().strftime('%Y-%m-%d') if train_size > 0 else 'N/A',
        'rows': int(train_size),
        'percentage': float(train_pct)
    },
    'validation': {
        'start_date': df_merged[val_mask]['date'].min().strftime('%Y-%m-%d') if val_size > 0 else 'N/A',
        'end_date': df_merged[val_mask]['date'].max().strftime('%Y-%m-%d') if val_size > 0 else 'N/A',
        'rows': int(val_size),
        'percentage': float(val_pct)
    },
    'test': {
        'start_date': df_merged[test_mask]['date'].min().strftime('%Y-%m-%d') if test_size > 0 else 'N/A',
        'end_date': df_merged[test_mask]['date'].max().strftime('%Y-%m-%d') if test_size > 0 else 'N/A',
        'rows': int(test_size),
        'percentage': float(test_pct)
    }
}

with open(SPLITS_FILE, 'w') as f:
    json.dump(splits_metadata, f, indent=2)

logger.info(f"\n  ✓ Split metadata saved: {SPLITS_FILE}")

# ============================================================================
# STEP 6: Create Feature Catalog (FIXED)
# ============================================================================
logger.info("\n[6/8] Creating feature catalog...")

catalog_data = []

for col in df_merged.columns:
    if col == 'date':
        continue

    try:
        # CRITICAL FIX: Use proper Series access
        col_series = df_merged[col]

        # Skip if this returns a DataFrame (shouldn't happen now but safety check)
        if isinstance(col_series, pd.DataFrame):
            logger.warning(f"  ⚠ Skipping duplicate column: {col}")
            continue

        # Determine category
        if any(x in col for x in ['_active', '_tight', '_applying', '_strength', '_exact']):
            category = 'Aspects'
        elif any(x in col for x in ['_sign', '_nakshatra', '_retrograde', '_exalted', '_dignity']):
            category = 'Transits'
        elif any(x in col for x in ['moon_phase', 'mercury_retrograde', 'day_of_week', 'month', 'quarter']):
            category = 'Temporal'
        elif any(x in col for x in ['midpoint', 'daily_', '_score']):
            category = 'Advanced'
        elif any(x in col for x in ['_longitude', '_speed', 'julian_day']):
            category = 'Raw Planetary'
        elif any(x in col for x in ['_open', '_high', '_low', '_close', '_volume', 'currency']):
            category = 'Financial'
        else:
            category = 'Other'

        # Determine data type
        dtype = col_series.dtype
        if dtype in ['int8', 'int16', 'int32', 'int64']:
            data_type = 'Integer'
        elif dtype in ['float16', 'float32', 'float64']:
            data_type = 'Float'
        elif dtype == 'object':
            data_type = 'Object'
        else:
            data_type = str(dtype)

        # Calculate null percentage
        null_pct = (col_series.isnull().sum() / len(df_merged)) * 100

        # Get value range
        if data_type in ['Integer', 'Float']:
            try:
                val_min = col_series.min()
                val_max = col_series.max()
                value_range = f"{val_min:.2f} to {val_max:.2f}"
            except:
                value_range = "N/A"
        else:
            value_range = "N/A"

        catalog_data.append({
            'feature_name': col,
            'category': category,
            'data_type': data_type,
            'null_percentage': f"{null_pct:.2f}%",
            'value_range': value_range,
            'description': f"{category} feature: {col}"
        })

    except Exception as e:
        logger.warning(f"  ⚠ Error processing column {col}: {str(e)[:50]}")
        continue

df_catalog = pd.DataFrame(catalog_data)
df_catalog.to_csv(CATALOG_FILE, index=False)

logger.info(f"  ✓ Feature catalog created: {len(catalog_data)} features")
logger.info(f"  ✓ Saved to: {CATALOG_FILE}")

# Print category summary
category_counts = df_catalog['category'].value_counts()
logger.info(f"\n  Feature breakdown by category:")
for cat, count in category_counts.items():
    logger.info(f"    • {cat}: {count}")

# ============================================================================
# STEP 7: Save Master Features Dataset
# ============================================================================
logger.info("\n[7/8] Saving master features dataset...")

try:
    df_merged.to_parquet(OUTPUT_FILE, index=False, engine='pyarrow')
    file_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
    logger.info(f"  ✓ Saved: master_features_dataset.parquet")
    logger.info(f"  ✓ File size: {file_size_mb:.2f} MB")
    logger.info(f"  ✓ Shape: {df_merged.shape}")
except Exception as e:
    logger.critical(f"\n✗ FATAL ERROR: Could not save file")
    logger.critical(f"  Error: {e}")
    raise SystemExit(1)

# ============================================================================
# STEP 8: Create Dataset Metadata
# ============================================================================
logger.info("\n[8/8] Creating dataset metadata...")

metadata = {
    'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset_info': {
        'total_rows': int(len(df_merged)),
        'total_features': int(len(df_merged.columns)),
        'date_range_start': df_merged['date'].min().strftime('%Y-%m-%d'),
        'date_range_end': df_merged['date'].max().strftime('%Y-%m-%d'),
        'days_covered': int(len(df_merged)),
    },
    'feature_categories': {
        cat: int(count) for cat, count in category_counts.items()
    },
    'data_quality': {
        'null_values': int(total_nulls),
        'null_percentage': float(null_pct),
        'duplicate_rows': int(duplicates),
    },
    'train_val_test_splits': splits_metadata,
    'source_files': {
        'master_aligned': 'aligned_data/master_aligned_dataset.parquet',
        'aspects': 'feature_data/aspects_features.parquet',
        'transit': 'feature_data/transit_features.parquet',
        'temporal': 'feature_data/temporal_features.parquet',
        'advanced': 'feature_data/advanced_features.parquet',
    },
    'phase_2_completion': {
        'cell_4_aspects': 'Complete',
        'cell_5_transits': 'Complete',
        'cell_6_temporal': 'Complete',
        'cell_7_advanced': 'Complete',
        'cell_8_integration': 'Complete',
    }
}

with open(METADATA_FILE, 'w') as f:
    json.dump(metadata, f, indent=2)

logger.info(f"  ✓ Metadata saved: {METADATA_FILE}")

# ============================================================================
# FINAL SUMMARY & VALIDATION
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("PHASE 2 COMPLETION SUMMARY")
logger.info("=" * 70)

logger.info("\n📊 Dataset Statistics:")
logger.info(f"  • Total rows: {len(df_merged):,}")
logger.info(f"  • Total features: {len(df_merged.columns):,}")
logger.info(f"  • Date range: {df_merged['date'].min().date()} to {df_merged['date'].max().date()}")
logger.info(f"  • File size: {file_size_mb:.2f} MB")

logger.info("\n📋 Feature Categories:")
for cat, count in category_counts.items():
    pct = (count / len(df_merged.columns)) * 100
    logger.info(f"  • {cat}: {count} ({pct:.1f}%)")

logger.info("\n✅ Data Quality:")
logger.info(f"  • Null values: {total_nulls} ({null_pct:.2f}%)")
logger.info(f"  • Duplicates: {duplicates}")
logger.error(f"  • Date continuity: {'✓ Verified' if len(df_merged) > 0 else '✗ Failed'}")

logger.info("\n🎯 Train/Val/Test Splits:")
logger.info(f"  • Train: {train_size:,} rows ({train_pct:.1f}%)")
logger.info(f"  • Validation: {val_size:,} rows ({val_pct:.1f}%)")
logger.info(f"  • Test: {test_size:,} rows ({test_pct:.1f}%)")

# Sample preview
logger.info("\n" + "=" * 70)
logger.info("SAMPLE: Master Features Dataset (First 3 Rows)")
logger.info("=" * 70)

# Show a representative sample of columns
sample_cols = ['date', 'sun_longitude', 'moon_phase_angle',
               'sun_moon_conjunction_active', 'daily_aspect_quality',
               'mercury_retrograde', 'jupiter_strength_score']

existing_sample = [col for col in sample_cols if col in df_merged.columns]
if existing_sample:
    print("\n" + tabulate(df_merged[existing_sample].head(3),
                         headers='keys', tablefmt='grid',
                         showindex=False, floatfmt=".2f"))
else:
    logger.info("\n  No sample columns available")

# ============================================================================
# FINAL STATUS
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("PHASE 2 (FEATURE ENGINEERING) - STATUS: COMPLETE ✓✓✓")
logger.info("=" * 70)

logger.info("\n📂 Output Files Created:")
logger.info(f"  1. {OUTPUT_FILE}")
logger.info(f"     → Master dataset with all features")
logger.info(f"  2. {CATALOG_FILE}")
logger.info(f"     → Feature catalog with descriptions")
logger.info(f"  3. {SPLITS_FILE}")
logger.info(f"     → Train/validation/test split definitions")
logger.info(f"  4. {METADATA_FILE}")
logger.info(f"     → Complete dataset metadata")

logger.info("\n📋 Integration Points for Phase 3:")
logger.info("  → Load master_features_dataset.parquet directly")
logger.info("  → Use feature_catalog.csv for feature selection")
logger.info("  → Use train_val_test_splits.json for consistent splits")
logger.info("  → All features properly aligned and ready for ML")

logger.info("\n🎯 Ready for Phase 3: Model Design & Training")
logger.info("  The dataset is now ready for:")
logger.info("  • XGBoost, Random Forest, Neural Networks")
logger.info("  • Time-series cross-validation")
logger.info("  • Feature importance analysis")
logger.info("  • SHAP value calculations")

logger.info("\n" + "=" * 70)
logger.info("🎉 PHASE 2 COMPLETE - ALL FEATURES ENGINEERED SUCCESSFULLY! 🎉")
logger.info("=" * 70)

# PHASE 3 - 📊 Model Design & Training

In [3]:
# Cell 9: Multi-Ticker Data Preparation (Phase 3 - Part 1 of 6) - WITH NAN HANDLING
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
from datetime import datetime
from tabulate import tabulate
import json
import pickle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


np.random.seed(42)

logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 3: PROFESSIONAL ML PIPELINE")
logger.info("Phase 3 Progress: Part 1 of 6 (Multi-Ticker Data Preparation)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup Paths
# ============================================================================
logger.info("\n[1/13] Setting up paths...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
FEATURE_DATA_PATH = os.path.join(BASE_PATH, 'feature_data')
ALIGNED_DATA_PATH = os.path.join(BASE_PATH, 'aligned_data')
PREPARED_DATA_PATH = os.path.join(BASE_PATH, 'prepared_data')
MULTI_TICKER_PATH = os.path.join(PREPARED_DATA_PATH, 'multi_ticker')

# Create directories
os.makedirs(MULTI_TICKER_PATH, exist_ok=True)

logger.info(f"  ✓ Input: {FEATURE_DATA_PATH}")
logger.info(f"  ✓ Output: {MULTI_TICKER_PATH}")

# ============================================================================
# STEP 2: Load Master Features Dataset
# ============================================================================
logger.info("\n[2/13] Loading master features dataset...")

MASTER_FILE = os.path.join(FEATURE_DATA_PATH, 'master_features_dataset.parquet')

if not os.path.exists(MASTER_FILE):
    logger.critical(f"\n✗ FATAL ERROR: Master features dataset not found")
    logger.info("  Please run Cell 8 first to generate the integrated dataset.")
    raise SystemExit(1)

df_features = pd.read_parquet(MASTER_FILE)
df_features['date'] = pd.to_datetime(df_features['date'])

logger.info(f"  ✓ Loaded master dataset")
logger.info(f"  ✓ Shape: {df_features.shape}")
logger.info(f"  ✓ Date range: {df_features['date'].min().date()} to {df_features['date'].max().date()}")

# Get list of astrological feature columns (exclude financial/date columns)
astro_features = [col for col in df_features.columns if col not in ['date'] and not any(
    x in col for x in ['_open', '_high', '_low', '_close', '_volume', 'currency', 'volume_unit', '_adj_close']
)]

logger.info(f"  ✓ Identified {len(astro_features)} astrological features")

# Check for NaN in astrological features
astro_nulls = df_features[astro_features].isnull().sum().sum()
logger.info(f"  ✓ Astrological features NaN count: {astro_nulls} ({astro_nulls / df_features[astro_features].size * 100:.2f}%)")

# ============================================================================
# STEP 3: Handle Missing Values in Astrological Features
# ============================================================================
logger.info("\n[3/13] Handling missing values in astrological features...")

if astro_nulls > 0:
    logger.info(f"  Found {astro_nulls} missing values in astrological features")

    # Show columns with most nulls
    null_counts = df_features[astro_features].isnull().sum()
    top_nulls = null_counts[null_counts > 0].sort_values(ascending=False).head(10)

    logger.info(f"\n  Top columns with missing values:")
    for col, count in top_nulls.items():
        pct = (count / len(df_features)) * 100
        logger.info(f"    • {col}: {count} ({pct:.1f}%)")

    # Strategy 1: Drop columns with >50% missing values (unreliable)
    high_null_threshold = 0.5
    high_null_cols = null_counts[null_counts / len(df_features) > high_null_threshold].index.tolist()

    if high_null_cols:
        logger.info(f"\n  Dropping {len(high_null_cols)} columns with >{high_null_threshold*100:.0f}% missing values:")
        for col in high_null_cols[:5]:
            logger.info(f"    • {col}")
        if len(high_null_cols) > 5:
            logger.info(f"    • ... and {len(high_null_cols) - 5} more")

        astro_features = [col for col in astro_features if col not in high_null_cols]

    # Strategy 2: Impute remaining missing values with median
    remaining_nulls = df_features[astro_features].isnull().sum().sum()

    if remaining_nulls > 0:
        logger.info(f"\n  Imputing {remaining_nulls} remaining missing values with median...")

        astro_imputer = SimpleImputer(strategy='median', add_indicator=False)
        df_features[astro_features] = astro_imputer.fit_transform(df_features[astro_features])

        # Save imputer for documentation
        with open(os.path.join(MULTI_TICKER_PATH, 'astro_imputer.pkl'), 'wb') as f:
            pickle.dump(astro_imputer, f)

        logger.info(f"  ✓ Imputation complete")

    # Verify no NaN values remain
    final_nulls = df_features[astro_features].isnull().sum().sum()
    logger.info(f"  ✓ Remaining NaN in astro features: {final_nulls}")
else:
    logger.info(f"  ✓ No missing values in astrological features")

logger.info(f"  ✓ Final astrological features: {len(astro_features)}")

# ============================================================================
# STEP 4: Define Ticker Universe and Metadata
# ============================================================================
logger.info("\n[4/13] Defining ticker universe and metadata...")

# Define comprehensive ticker metadata
TICKER_METADATA = {
    # US Tech
    'AAPL': {'sector': 'Technology', 'region': 'US', 'market_cap': 'Large'},
    'MSFT': {'sector': 'Technology', 'region': 'US', 'market_cap': 'Large'},
    'NVDA': {'sector': 'Technology', 'region': 'US', 'market_cap': 'Large'},

    # US Indices
    'GSPC': {'sector': 'Indices', 'region': 'US', 'market_cap': 'Index'},
    'DJI': {'sector': 'Indices', 'region': 'US', 'market_cap': 'Index'},
    'NDX': {'sector': 'Indices', 'region': 'US', 'market_cap': 'Index'},
    'RUT': {'sector': 'Indices', 'region': 'US', 'market_cap': 'Index'},
    'VIX': {'sector': 'Indices', 'region': 'US', 'market_cap': 'Index'},
    'TNX': {'sector': 'Indices', 'region': 'US', 'market_cap': 'Index'},

    # India Indices
    'NSEI': {'sector': 'Indices', 'region': 'India', 'market_cap': 'Index'},
    'NSEBANK': {'sector': 'Finance', 'region': 'India', 'market_cap': 'Index'},
    'NIFTY_FIN_SERVICE_NS': {'sector': 'Finance', 'region': 'India', 'market_cap': 'Index'},
    'CNXIT': {'sector': 'Technology', 'region': 'India', 'market_cap': 'Index'},
    'CNXPHARMA': {'sector': 'Pharma', 'region': 'India', 'market_cap': 'Index'},
    'CNXAUTO': {'sector': 'Indices', 'region': 'India', 'market_cap': 'Index'},
    'CNXMETAL': {'sector': 'Commodities', 'region': 'India', 'market_cap': 'Index'},
    'CNXFMCG': {'sector': 'Indices', 'region': 'India', 'market_cap': 'Index'},
    'INDIAVIX': {'sector': 'Indices', 'region': 'India', 'market_cap': 'Index'},

    # India Stocks
    'RELIANCE_NS': {'sector': 'Commodities', 'region': 'India', 'market_cap': 'Large'},
    'TCS_NS': {'sector': 'Technology', 'region': 'India', 'market_cap': 'Large'},
    'HDFCBANK_NS': {'sector': 'Finance', 'region': 'India', 'market_cap': 'Large'},

    # Global Indices
    'N225': {'sector': 'Indices', 'region': 'Asia', 'market_cap': 'Index'},
    'FTSE': {'sector': 'Indices', 'region': 'Europe', 'market_cap': 'Index'},
    'GDAXI': {'sector': 'Indices', 'region': 'Europe', 'market_cap': 'Index'},
    '000001_SS': {'sector': 'Indices', 'region': 'Asia', 'market_cap': 'Index'},
    'HSI': {'sector': 'Indices', 'region': 'Asia', 'market_cap': 'Index'},

    # Commodities
    'GC': {'sector': 'Commodities', 'region': 'Global', 'market_cap': 'Commodity'},
    'CL': {'sector': 'Commodities', 'region': 'Global', 'market_cap': 'Commodity'},
    'SI': {'sector': 'Commodities', 'region': 'Global', 'market_cap': 'Commodity'},

    # Currencies
    'DX_Y_NYB': {'sector': 'Currencies', 'region': 'Global', 'market_cap': 'Currency'},
    'USDINR_X': {'sector': 'Currencies', 'region': 'Global', 'market_cap': 'Currency'},
    'EURUSD_X': {'sector': 'Currencies', 'region': 'Global', 'market_cap': 'Currency'},
}

# Find which tickers have data in our dataset
available_tickers = []
for ticker, metadata in TICKER_METADATA.items():
    close_col = f'{ticker}_close'
    if close_col in df_features.columns:
        # Check if ticker has sufficient non-null data
        non_null_pct = (1 - df_features[close_col].isnull().mean()) * 100
        if non_null_pct >= 50:  # At least 50% data availability
            available_tickers.append(ticker)
        else:
            logger.warning(f"  ⚠ Skipping {ticker}: only {non_null_pct:.1f}% data available")

logger.info(f"\n  ✓ Found {len(available_tickers)} tickers with sufficient data")
logger.info(f"  Available tickers: {', '.join(available_tickers[:10])}...")

# ============================================================================
# STEP 5: Stack All Ticker Data (OPTIMIZED)
# ============================================================================
logger.info("\n[5/13] Stacking all ticker data...")
logger.info("  (This creates ~190,000 samples from multi-ticker expansion)")

# Use list to collect DataFrames, then concat once at the end
ticker_dfs = []
processing_summary = []

for i, ticker in enumerate(available_tickers):
    if (i + 1) % 5 == 0:
        logger.info(f"  Processing ticker {i+1}/{len(available_tickers)}...")

    # Get financial columns for this ticker
    close_col = f'{ticker}_close'
    volume_col = f'{ticker}_volume'

    if close_col not in df_features.columns:
        continue

    # Create ticker-specific DataFrame
    ticker_data = {
        'date': df_features['date'],
        'ticker': ticker,
        'close': df_features[close_col],
    }

    # Add volume if available
    if volume_col in df_features.columns:
        ticker_data['volume'] = df_features[volume_col]
    else:
        ticker_data['volume'] = np.nan

    # Create DataFrame
    ticker_df = pd.DataFrame(ticker_data)

    # Remove rows with null close prices
    initial_rows = len(ticker_df)
    ticker_df = ticker_df.dropna(subset=['close'])
    final_rows = len(ticker_df)

    if final_rows < 100:  # Need at least 100 days of data
        logger.warning(f"  ⚠ Skipping {ticker}: only {final_rows} valid days")
        continue

    # Add metadata
    metadata = TICKER_METADATA[ticker]
    ticker_df['sector'] = metadata['sector']
    ticker_df['region'] = metadata['region']
    ticker_df['market_cap'] = metadata['market_cap']

    # Calculate technical indicators
    close_prices = ticker_df['close'].values
    volume = ticker_df['volume'].values

    # 1. Returns (1-day, 5-day, 20-day)
    returns_1d = np.zeros(len(close_prices), dtype=np.float32)
    returns_1d[1:] = (close_prices[1:] / close_prices[:-1] - 1) * 100

    returns_5d = np.zeros(len(close_prices), dtype=np.float32)
    returns_5d[5:] = (close_prices[5:] / close_prices[:-5] - 1) * 100

    returns_20d = np.zeros(len(close_prices), dtype=np.float32)
    returns_20d[20:] = (close_prices[20:] / close_prices[:-20] - 1) * 100

    # 2. Moving averages
    sma_20 = pd.Series(close_prices).rolling(window=20, min_periods=1).mean().values
    sma_50 = pd.Series(close_prices).rolling(window=50, min_periods=1).mean().values

    # 3. RSI (14-day)
    delta = np.diff(close_prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=14, min_periods=1).mean().values
    avg_loss = pd.Series(loss).rolling(window=14, min_periods=1).mean().values

    rs = avg_gain / (avg_loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    rsi = np.concatenate([[50], rsi])  # Add initial value

    # 4. Bollinger Bands
    bb_middle = sma_20
    bb_std = pd.Series(close_prices).rolling(window=20, min_periods=1).std().values
    bb_upper = bb_middle + (2 * bb_std)
    bb_lower = bb_middle - (2 * bb_std)
    bb_position = (close_prices - bb_lower) / (bb_upper - bb_lower + 1e-10)

    # 5. ATR (Average True Range - volatility)
    high_low = pd.Series(close_prices).rolling(window=2).apply(
        lambda x: abs(x.iloc[-1] - x.iloc[0]) if len(x) == 2 else 0
    ).values
    atr = pd.Series(high_low).rolling(window=14, min_periods=1).mean().values

    # 6. Volume ratio (handle NaN volumes)
    volume_clean = np.nan_to_num(volume, nan=0.0)
    volume_ma = pd.Series(volume_clean).rolling(window=20, min_periods=1).mean().values
    volume_ratio = volume_clean / (volume_ma + 1e-10)

    # 7. Volatility (20-day rolling std of returns)
    volatility_20d = pd.Series(returns_1d).rolling(window=20, min_periods=1).std().values

    # Create all technical features at once using a dictionary
    tech_features = pd.DataFrame({
        'returns_1d': returns_1d,
        'returns_5d': returns_5d,
        'returns_20d': returns_20d,
        'sma_20': sma_20,
        'sma_50': sma_50,
        'rsi_14': rsi,
        'bb_position': bb_position,
        'atr_14': atr,
        'volume_ratio': volume_ratio,
        'volatility_20d': volatility_20d,
    }, index=ticker_df.index)

    # OPTIMIZED: Concatenate all new columns at once
    ticker_df = pd.concat([ticker_df, tech_features], axis=1)

    # Calculate targets (forward returns)
    # Target 1: 5-day forward return > 1%
    target_5day = np.zeros(len(close_prices), dtype=np.int8)
    forward_returns_5d = np.zeros(len(close_prices), dtype=np.float32)
    if len(close_prices) > 5:
        forward_returns_5d[:-5] = (close_prices[5:] / close_prices[:-5] - 1) * 100
        target_5day[:-5] = (forward_returns_5d[:-5] > 1.0).astype(np.int8)

    # Target 2: 3-day forward return > 0.5%
    target_3day = np.zeros(len(close_prices), dtype=np.int8)
    forward_returns_3d = np.zeros(len(close_prices), dtype=np.float32)
    if len(close_prices) > 3:
        forward_returns_3d[:-3] = (close_prices[3:] / close_prices[:-3] - 1) * 100
        target_3day[:-3] = (forward_returns_3d[:-3] > 0.5).astype(np.int8)

    # Target 3: Volatility regime (next 5 days)
    target_volatility = np.zeros(len(close_prices), dtype=np.int8)
    if len(close_prices) > 5:
        future_vol = pd.Series(close_prices).rolling(window=5).std().shift(-5).values
        median_vol = np.nanmedian(future_vol)
        target_volatility = (future_vol > median_vol).astype(np.int8)

    # Target 4: Continuous 5-day return
    target_magnitude = forward_returns_5d

    # Add targets using DataFrame (optimized)
    target_features = pd.DataFrame({
        'target_5day': target_5day,
        'target_3day': target_3day,
        'target_volatility': target_volatility,
        'target_magnitude': target_magnitude,
    }, index=ticker_df.index)

    # OPTIMIZED: Concatenate targets
    ticker_df = pd.concat([ticker_df, target_features], axis=1)

    # Remove last 5 rows (no valid targets)
    ticker_df = ticker_df.iloc[:-5]

    # Add to list
    ticker_dfs.append(ticker_df)

    processing_summary.append({
        'ticker': ticker,
        'sector': metadata['sector'],
        'rows': len(ticker_df),
        'null_pct': f"{ticker_df.isnull().mean().mean()*100:.1f}%"
    })

# CRITICAL OPTIMIZATION: Concatenate all ticker DataFrames at once
logger.info(f"\n  Concatenating {len(ticker_dfs)} ticker datasets...")
df_stacked = pd.concat(ticker_dfs, axis=0, ignore_index=True)

logger.info(f"  ✓ Stacked dataset created: {df_stacked.shape}")
logger.info(f"  ✓ Total samples: {len(df_stacked):,}")

# Display processing summary
logger.info("\n  Processing summary:")
summary_df = pd.DataFrame(processing_summary)
print("\n" + tabulate(summary_df.head(10), headers='keys', tablefmt='grid', showindex=False))

# ============================================================================
# STEP 6: Add Astrological Features to Each Row
# ============================================================================
logger.info("\n[6/13] Adding astrological features...")

# Merge astrological features based on date
astro_cols = ['date'] + astro_features
df_astro = df_features[astro_cols].copy()

# OPTIMIZED: Single merge operation
df_stacked = pd.merge(
    df_stacked,
    df_astro,
    on='date',
    how='left'
)

logger.info(f"  ✓ Added {len(astro_features)} astrological features")
logger.info(f"  ✓ Shape after merge: {df_stacked.shape}")

# Check for NaN after merge
merge_nulls = df_stacked.isnull().sum().sum()
if merge_nulls > 0:
    logger.warning(f"  ⚠ Found {merge_nulls} NaN values after merge")

    # This shouldn't happen since we imputed earlier, but handle it
    null_cols = df_stacked.isnull().sum()
    null_cols = null_cols[null_cols > 0].sort_values(ascending=False).head(5)

    logger.info(f"  Top columns with NaN:")
    for col, count in null_cols.items():
        logger.info(f"    • {col}: {count}")

    # Forward fill then backward fill (for any edge cases)
    df_stacked = df_stacked.fillna(method='ffill').fillna(method='bfill').fillna(0)

    final_nulls = df_stacked.isnull().sum().sum()
    logger.info(f"  ✓ After filling: {final_nulls} NaN remaining")

logger.info(f"  ✓ Final shape: {df_stacked.shape}")

# ============================================================================
# STEP 7: Handle Infinite and Invalid Values
# ============================================================================
logger.info("\n[7/13] Checking for infinite and invalid values...")

# Check for infinite values
inf_mask = np.isinf(df_stacked.select_dtypes(include=[np.number]).values)
inf_count = inf_mask.sum()

if inf_count > 0:
    logger.warning(f"  ⚠ Found {inf_count} infinite values")
    numeric_cols = df_stacked.select_dtypes(include=[np.number]).columns
    df_stacked[numeric_cols] = df_stacked[numeric_cols].replace([np.inf, -np.inf], 0)
    logger.info(f"  ✓ Replaced infinite values with 0")
else:
    logger.info(f"  ✓ No infinite values found")

# Final NaN check
final_nan_count = df_stacked.isnull().sum().sum()
logger.info(f"  ✓ Final NaN count: {final_nan_count}")

if final_nan_count > 0:
    logger.warning(f"  ⚠ WARNING: {final_nan_count} NaN values remain. Filling with 0...")
    df_stacked = df_stacked.fillna(0)
    logger.info(f"  ✓ All NaN values resolved")

# ============================================================================
# STEP 8: Encode Categorical Variables
# ============================================================================
logger.info("\n[8/13] Encoding categorical variables...")

label_encoders = {}

# Encode ticker
le_ticker = LabelEncoder()
df_stacked['ticker_id'] = le_ticker.fit_transform(df_stacked['ticker'])
label_encoders['ticker'] = le_ticker

# Encode sector
le_sector = LabelEncoder()
df_stacked['sector_id'] = le_sector.fit_transform(df_stacked['sector'])
label_encoders['sector'] = le_sector

# Encode region
le_region = LabelEncoder()
df_stacked['region_id'] = le_region.fit_transform(df_stacked['region'])
label_encoders['region'] = le_region

logger.info(f"  ✓ Encoded ticker: {len(le_ticker.classes_)} unique values")
logger.info(f"  ✓ Encoded sector: {len(le_sector.classes_)} unique values")
logger.info(f"  ✓ Encoded region: {len(le_region.classes_)} unique values")

# Save encoders
with open(os.path.join(MULTI_TICKER_PATH, 'label_encoders.pkl'), 'wb') as f:
    pickle.dump(label_encoders, f)

logger.info(f"  ✓ Saved label encoders")

# ============================================================================
# STEP 9: Create Train/Validation/Test Splits (Time-Based)
# ============================================================================
logger.info("\n[9/13] Creating time-based splits...")

# Define split dates
train_end = pd.Timestamp('2020-12-31')
val_end = pd.Timestamp('2023-12-31')

train_mask = df_stacked['date'] <= train_end
val_mask = (df_stacked['date'] > train_end) & (df_stacked['date'] <= val_end)
test_mask = df_stacked['date'] > val_end

df_train = df_stacked[train_mask].copy()
df_val = df_stacked[val_mask].copy()
df_test = df_stacked[test_mask].copy()

logger.info(f"\n  Split distribution:")
logger.info(f"    • Train: {len(df_train):,} rows ({len(df_train)/len(df_stacked)*100:.1f}%)")
logger.info(f"    • Val:   {len(df_val):,} rows ({len(df_val)/len(df_stacked)*100:.1f}%)")
logger.info(f"    • Test:  {len(df_test):,} rows ({len(df_test)/len(df_stacked)*100:.1f}%)")

# ============================================================================
# STEP 10: Prepare Features and Targets
# ============================================================================
logger.info("\n[10/13] Preparing features and targets...")

# Define feature columns (exclude metadata and targets)
exclude_cols = ['date', 'ticker', 'sector', 'region', 'market_cap', 'close', 'volume',
                'target_5day', 'target_3day', 'target_volatility', 'target_magnitude']

feature_cols = [col for col in df_stacked.columns if col not in exclude_cols]

logger.info(f"  ✓ Selected {len(feature_cols)} features for modeling")

# Prepare X (features) and y (target)
X_train = df_train[['date'] + feature_cols].copy()
X_val = df_val[['date'] + feature_cols].copy()
X_test = df_test[['date'] + feature_cols].copy()

# Primary target: 5-day direction
y_train = df_train[['target_5day']].copy()
y_train.columns = ['target']

y_val = df_val[['target_5day']].copy()
y_val.columns = ['target']

y_test = df_test[['target_5day']].copy()
y_test.columns = ['target']

logger.info(f"  ✓ X_train: {X_train.shape}")
logger.info(f"  ✓ X_val: {X_val.shape}")
logger.info(f"  ✓ X_test: {X_test.shape}")

# Check class balance
train_balance = y_train['target'].value_counts()
logger.info(f"\n  Target distribution (train):")
logger.info(f"    • Class 0 (down): {train_balance.get(0, 0):,} ({train_balance.get(0, 0)/len(y_train)*100:.1f}%)")
logger.info(f"    • Class 1 (up):   {train_balance.get(1, 0):,} ({train_balance.get(1, 0)/len(y_train)*100:.1f}%)")

# ============================================================================
# STEP 11: Feature Scaling
# ============================================================================
logger.info("\n[11/13] Applying feature scaling...")

# Identify numeric columns (exclude categorical IDs and date)
numeric_cols = [col for col in feature_cols if col not in ['ticker_id', 'sector_id', 'region_id', 'date']]

logger.info(f"  ✓ Scaling {len(numeric_cols)} numeric features")

# Fit scaler on training data only
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_val[numeric_cols] = scaler.transform(X_val[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Save scaler
with open(os.path.join(MULTI_TICKER_PATH, 'feature_scaler.pkl'), 'wb') as f:
    pickle.dump(scaler, f)

logger.info(f"  ✓ Feature scaler saved")

# ============================================================================
# STEP 12: Final Data Quality Verification
# ============================================================================
logger.info("\n[12/13] Final data quality verification...")

# Check for any remaining issues
train_issues = {
    'NaN': X_train.isnull().sum().sum(),
    'Inf': np.isinf(X_train.select_dtypes(include=[np.number]).values).sum(),
}

val_issues = {
    'NaN': X_val.isnull().sum().sum(),
    'Inf': np.isinf(X_val.select_dtypes(include=[np.number]).values).sum(),
}

test_issues = {
    'NaN': X_test.isnull().sum().sum(),
    'Inf': np.isinf(X_test.select_dtypes(include=[np.number]).values).sum(),
}

logger.info(f"\n  Data quality report:")
logger.info(f"    Train - NaN: {train_issues['NaN']}, Inf: {train_issues['Inf']}")
logger.info(f"    Val   - NaN: {val_issues['NaN']}, Inf: {val_issues['Inf']}")
logger.info(f"    Test  - NaN: {test_issues['NaN']}, Inf: {test_issues['Inf']}")

if any(v > 0 for issues in [train_issues, val_issues, test_issues] for v in issues.values()):
    logger.warning(f"\n  ⚠ WARNING: Data quality issues detected!")
    raise ValueError("Data contains NaN or Inf values after all processing steps")
else:
    logger.info(f"\n  ✓ All data quality checks passed!")

# ============================================================================
# STEP 13: Save Prepared Data
# ============================================================================
logger.info("\n[13/13] Saving prepared datasets...")

# Save datasets
X_train.to_parquet(os.path.join(MULTI_TICKER_PATH, 'X_train.parquet'), index=False)
X_val.to_parquet(os.path.join(MULTI_TICKER_PATH, 'X_val.parquet'), index=False)
X_test.to_parquet(os.path.join(MULTI_TICKER_PATH, 'X_test.parquet'), index=False)

y_train.to_parquet(os.path.join(MULTI_TICKER_PATH, 'y_train.parquet'), index=False)
y_val.to_parquet(os.path.join(MULTI_TICKER_PATH, 'y_val.parquet'), index=False)
y_test.to_parquet(os.path.join(MULTI_TICKER_PATH, 'y_test.parquet'), index=False)

logger.info(f"  ✓ Saved X_train.parquet")
logger.info(f"  ✓ Saved X_val.parquet")
logger.info(f"  ✓ Saved X_test.parquet")
logger.info(f"  ✓ Saved y_train.parquet")
logger.info(f"  ✓ Saved y_val.parquet")
logger.info(f"  ✓ Saved y_test.parquet")

# Save metadata
metadata = {
    'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'num_tickers': len(available_tickers),
    'num_sectors': len(le_sector.classes_),
    'num_regions': len(le_region.classes_),
    'total_samples': len(df_stacked),
    'train_samples': len(df_train),
    'val_samples': len(df_val),
    'test_samples': len(df_test),
    'num_features': len(feature_cols),
    'tickers': available_tickers,
    'sectors': le_sector.classes_.tolist(),
    'regions': le_region.classes_.tolist(),
    'feature_columns': feature_cols,
    'data_quality': {
        'nan_values': 0,
        'inf_values': 0,
        'imputation_applied': bool(astro_nulls > 0),
    }
}

with open(os.path.join(MULTI_TICKER_PATH, 'dataset_metadata.json'), 'w') as f:
    json.dump(metadata, f, indent=2)

logger.info(f"  ✓ Saved dataset_metadata.json")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("PHASE 3 PART 1 (MULTI-TICKER PREPARATION) - COMPLETE ✓")
logger.info("=" * 70)

logger.info(f"\n📊 Dataset Summary:")
logger.info(f"  • Total samples: {len(df_stacked):,}")
logger.info(f"  • Train: {len(df_train):,} | Val: {len(df_val):,} | Test: {len(df_test):,}")
logger.info(f"  • Features: {len(feature_cols)}")
logger.info(f"  • Tickers: {len(available_tickers)}")
logger.info(f"  • Sectors: {len(le_sector.classes_)}")

logger.info(f"\n✅ Data Quality:")
logger.info(f"  • NaN values: 0 (all handled)")
logger.info(f"  • Infinite values: 0 (all handled)")
logger.info(f"  • Ready for ML training")

logger.info(f"\n📋 Next Steps:")
logger.info(f"  1. ✓ Multi-ticker data prepared ({len(df_stacked):,} samples)")
logger.info(f"  2. ▶ Run Cell 10: LightGBM Training")
logger.info(f"  3. ▶ Run Cell 11: Sector-Specific Models")
logger.info(f"  4. ▶ Run Cell 12: Ensemble Methods")
logger.info(f"  5. ▶ Run Cell 13: Walk-Forward Validation")
logger.info(f"  6. ▶ Run Cell 14: SHAP Analysis")

logger.info(f"\n📂 Output: {MULTI_TICKER_PATH}/")
logger.info("=" * 70)

ASTRO-FINANCE PROJECT - PHASE 3: PROFESSIONAL ML PIPELINE
Phase 3 Progress: Part 1 of 6 (Multi-Ticker Data Preparation)

[1/13] Setting up paths...
  ✓ Input: /content/drive/MyDrive/AstroFinanceProject/feature_data
  ✓ Output: /content/drive/MyDrive/AstroFinanceProject/prepared_data/multi_ticker

[2/13] Loading master features dataset...
  ✓ Loaded master dataset
  ✓ Shape: (9434, 1127)
  ✓ Date range: 2000-01-01 to 2025-10-29
  ✓ Identified 934 astrological features
  ✓ Astrological features NaN count: 0 (0.00%)

[3/13] Handling missing values in astrological features...
  ✓ No missing values in astrological features
  ✓ Final astrological features: 934

[4/13] Defining ticker universe and metadata...
  ⚠ Skipping NSEI: only 47.1% data available
  ⚠ Skipping NSEBANK: only 44.2% data available
  ⚠ Skipping NIFTY_FIN_SERVICE_NS: only 36.8% data available
  ⚠ Skipping CNXIT: only 44.0% data available
  ⚠ Skipping CNXPHARMA: only 38.5% data available
  ⚠ Skipping CNXAUTO: only 37.2% data 

In [4]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, log_loss
)

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

# Cell 10: LightGBM Training - v5 (Phase 3 - Part 2 of 6)
# ================================================================
#
# v5 FIX: AGGRESSIVE REGULARIZATION
# - Even more conservative than v4
# - Addresses the moderate performance by reducing overfitting further
# - Lower learning rates, more dropout, higher regularization
# - Target: Get Test AUC closer to Val AUC
#
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
import json
import pickle
from datetime import datetime
from tabulate import tabulate
import lightgbm as lgb
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 3: LIGHTGBM v5")
logger.info("Phase 3 Progress: Part 2 of 6 (AGGRESSIVE Anti-Overfitting)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup and Load Data
# ============================================================================
logger.info("\n[1/10] Loading prepared data...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
MULTI_TICKER_PATH = os.path.join(BASE_PATH, 'prepared_data', 'multi_ticker')
MODEL_PATH = os.path.join(BASE_PATH, 'models')
LGBM_PATH = os.path.join(MODEL_PATH, 'lightgbm_improved')

os.makedirs(LGBM_PATH, exist_ok=True)

# Load data
X_train = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_train.parquet'))
X_val = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_val.parquet'))
X_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_test.parquet'))

y_train = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_train.parquet'))['target'].values
y_val = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_val.parquet'))['target'].values
y_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_test.parquet'))['target'].values

# Extract and remove dates
dates_train = X_train['date']
dates_val = X_val['date']
dates_test = X_test['date']

X_train = X_train.drop('date', axis=1)
X_val = X_val.drop('date', axis=1)
X_test = X_test.drop('date', axis=1)

logger.info(f"  ✓ Train: {X_train.shape}")
logger.info(f"  ✓ Val: {X_val.shape}")
logger.info(f"  ✓ Test: {X_test.shape}")

# Load metadata
with open(os.path.join(MULTI_TICKER_PATH, 'dataset_metadata.json'), 'r') as f:
    metadata = json.load(f)

logger.info(f"\n  Dataset: {metadata['num_tickers']} tickers, {metadata['total_samples']:,} samples")

# ============================================================================
# STEP 2: Analyze Dataset Characteristics
# ============================================================================
logger.info("\n[2/10] Analyzing dataset characteristics...")

class_counts = np.bincount(y_train)
imbalance_ratio = class_counts[0] / class_counts[1] if class_counts[1] > 0 else 1
minority_class_pct = min(class_counts) / sum(class_counts) * 100

logger.info(f"  Class 0: {class_counts[0]:,} ({class_counts[0]/len(y_train)*100:.1f}%)")
logger.info(f"  Class 1: {class_counts[1]:,} ({class_counts[1]/len(y_train)*100:.1f}%)")
logger.info(f"  Imbalance: {imbalance_ratio:.2f}")

categorical_features = [col for col in X_train.columns if col in ['ticker_id', 'sector_id', 'region_id']]

# ============================================================================
# STEP 3: Train Baseline Models
# ============================================================================
logger.info("\n[3/10] Training baseline models...")

baseline_results = []

# Random
random_clf = DummyClassifier(strategy='stratified', random_state=42)
random_clf.fit(X_train, y_train)
y_pred_random = random_clf.predict(X_test)
acc_random = accuracy_score(y_test, y_pred_random)

baseline_results.append({'Model': 'Random', 'Accuracy': f"{acc_random:.4f}", 'AUC': '0.5000'})

# Logistic Regression
lr_clf = LogisticRegression(max_iter=200, random_state=42, n_jobs=-1, verbose=0)
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
y_proba_lr = lr_clf.predict_proba(X_test)[:, 1]
acc_lr = accuracy_score(y_test, y_pred_lr)
auc_lr = roc_auc_score(y_test, y_proba_lr)

baseline_results.append({'Model': 'Logistic Regression', 'Accuracy': f"{acc_lr:.4f}", 'AUC': f"{auc_lr:.4f}"})

logger.info(f"  ✓ Baselines: Random={acc_random:.4f}, LR AUC={auc_lr:.4f}")

# ============================================================================
# STEP 4: AGGRESSIVE REGULARIZATION PARAMETERS
# ============================================================================
logger.info("\n[4/10] Setting AGGRESSIVE regularization parameters...")

base_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'seed': 42,
    'n_jobs': -1,
    'force_col_wise': True,

    # AGGRESSIVE REGULARIZATION
    'num_leaves': 15,                # Reduced from 25
    'max_depth': 4,                  # Reduced from 6
    'learning_rate': 0.02,           # Reduced from 0.04
    'min_child_samples': 100,        # Increased significantly

    # Feature sampling (dropout)
    'feature_fraction': 0.5,         # Use only 50% of features per tree
    'feature_fraction_bynode': 0.5,  # Additional node-level dropout
    'bagging_fraction': 0.6,         # Use only 60% of data per tree
    'bagging_freq': 5,

    # Regularization
    'lambda_l1': 1.0,                # Strong L1
    'lambda_l2': 1.0,                # Strong L2
    'min_gain_to_split': 0.02,      # Higher split threshold
    'max_bin': 200,                  # Reduced from default 255

    # Additional anti-overfitting
    'path_smooth': 1.0,              # Smooth leaf values
    'min_data_per_group': 50,        # For categorical features

    # Class imbalance
    'scale_pos_weight': float(imbalance_ratio * 0.7) if imbalance_ratio > 1.5 else 1.0
}

logger.info(f"  ✓ Ultra-conservative parameters set:")
logger.info(f"    • Learning rate: {base_params['learning_rate']} (VERY LOW)")
logger.info(f"    • Max depth: {base_params['max_depth']} (SHALLOW)")
logger.info(f"    • Feature fraction: {base_params['feature_fraction']} (HIGH DROPOUT)")
logger.info(f"    • Regularization: L1={base_params['lambda_l1']}, L2={base_params['lambda_l2']}")

# ============================================================================
# STEP 5: Validation-Based Early Stopping
# ============================================================================
logger.info("\n[5/10] Training with validation-based early stopping...")

train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features, reference=train_data)

logger.info(f"  Training with very conservative early stopping (rounds=150)...")
evals_result = {}

model_val = lgb.train(
    base_params,
    train_data,
    num_boost_round=3000,
    valid_sets=[train_data, val_data],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=150, verbose=False),  # Very conservative
        lgb.record_evaluation(evals_result)
    ]
)

optimal_rounds = model_val.best_iteration
train_auc = evals_result['train']['auc'][optimal_rounds - 1]
val_auc = evals_result['valid']['auc'][optimal_rounds - 1]
auc_gap = train_auc - val_auc

logger.info(f"\n  ✓ Stopping results:")
logger.info(f"    • Rounds: {optimal_rounds}")
logger.info(f"    • Train AUC: {train_auc:.4f}")
logger.info(f"    • Val AUC: {val_auc:.4f}")
logger.warning(f"    • Gap: {auc_gap:.4f} {'✓ EXCELLENT' if auc_gap < 0.03 else '✓ OK' if auc_gap < 0.05 else '⚠ HIGH'}")

# ============================================================================
# STEP 6: Train Final Model
# ============================================================================
logger.info(f"\n[6/10] Training final model with {optimal_rounds} rounds...")

start_time = datetime.now()

lgb_model = lgb.train(
    base_params,
    train_data,
    num_boost_round=optimal_rounds,
    valid_sets=[train_data, val_data],
    valid_names=['train', 'valid'],
    callbacks=[lgb.log_evaluation(period=200)]
)

training_time = (datetime.now() - start_time).total_seconds()
logger.info(f"  ✓ Complete in {training_time:.1f}s")

# ============================================================================
# STEP 7: Evaluate on Test Set
# ============================================================================
logger.info("\n[7/10] Evaluating on test set...")

y_pred_proba = lgb_model.predict(X_test)
y_pred = (y_pred_proba >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred_proba)
logloss = log_loss(y_test, y_pred_proba)

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

logger.info(f"\n  Test Performance:")
logger.info(f"    • Accuracy:  {acc:.4f}")
logger.info(f"    • Precision: {precision:.4f}")
logger.info(f"    • Recall:    {recall:.4f}")
logger.info(f"    • F1-Score:  {f1:.4f}")
logger.info(f"    • ROC-AUC:   {auc:.4f}")

logger.info(f"\n  Generalization Check:")
logger.info(f"    • Train AUC: {train_auc:.4f}")
logger.info(f"    • Val AUC:   {val_auc:.4f}")
logger.info(f"    • Test AUC:  {auc:.4f}")
logger.warning(f"    • Train→Val: {abs(train_auc - val_auc):.4f} {'✓' if abs(train_auc - val_auc) < 0.05 else '⚠'}")
logger.warning(f"    • Val→Test:  {abs(val_auc - auc):.4f} {'✓' if abs(val_auc - auc) < 0.03 else '⚠'}")

# ============================================================================
# STEP 8: Multi-Threshold Analysis
# ============================================================================
logger.info("\n[8/10] Multi-threshold analysis...")

thresholds = [0.45, 0.50, 0.55, 0.60, 0.65]
threshold_results = []

for threshold in thresholds:
    y_pred_t = (y_pred_proba >= threshold).astype(int)
    if y_pred_t.sum() == 0:
        continue

    acc_t = accuracy_score(y_test, y_pred_t)
    prec_t = precision_score(y_test, y_pred_t, zero_division=0)
    predicted_ups = y_pred_t.sum()
    actual_wins = (y_pred_t & y_test).sum()
    win_rate = actual_wins / predicted_ups if predicted_ups > 0 else 0

    threshold_results.append({
        'Threshold': threshold,
        'Predictions': predicted_ups,
        'Win_Rate': f"{win_rate:.2%}",
        'Precision': f"{prec_t:.4f}"
    })

threshold_df = pd.DataFrame(threshold_results)
print("\n" + tabulate(threshold_df, headers='keys', tablefmt='grid', showindex=False))

# ============================================================================
# STEP 9: Feature Importance
# ============================================================================
logger.info("\n[9/10] Feature importance...")

importance = lgb_model.feature_importance(importance_type='gain')
feature_names = lgb_model.feature_name()

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importance
}).sort_values('importance', ascending=False)

importance_df['importance_pct'] = importance_df['importance'] / importance_df['importance'].sum() * 100

def categorize_feature(feat_name):
    if feat_name in ['ticker_id', 'sector_id', 'region_id']:
        return 'Categorical'
    elif any(x in feat_name for x in ['sun_', 'moon_', 'mercury_', 'venus_', 'mars_', 'jupiter_', 'saturn_']):
        return 'Planetary'
    elif any(x in feat_name for x in ['aspect_', 'conjunction', 'opposition', 'trine', 'square']):
        return 'Aspects'
    elif any(x in feat_name for x in ['rsi', 'sma', 'bb_', 'atr', 'volume_ratio', 'returns_']):
        return 'Technical'
    else:
        return 'Other'

importance_df['category'] = importance_df['feature'].apply(categorize_feature)

importance_df.to_csv(os.path.join(LGBM_PATH, 'feature_importance_detailed.csv'), index=False)

logger.info(f"\n  Top 15 Features:")
for idx, row in importance_df.head(15).iterrows():
    logger.info(f"    {row['feature']:35s}: {row['importance_pct']:5.2f}% [{row['category']}]")

category_importance = importance_df.groupby('category')['importance_pct'].sum().sort_values(ascending=False)
logger.info(f"\n  By Category:")
for cat, pct in category_importance.items():
    logger.info(f"    • {cat:15s}: {pct:5.1f}%")

# ============================================================================
# STEP 10: Save Results
# ============================================================================
logger.info("\n[10/10] Saving model and results...")

lgb_model.save_model(os.path.join(LGBM_PATH, 'lightgbm_model.txt'))

with open(os.path.join(LGBM_PATH, 'baseline_models.pkl'), 'wb') as f:
    pickle.dump({'random': random_clf, 'logistic_regression': lr_clf}, f)

performance = {
    'model_info': {
        'version': 'v5_aggressive_regularization',
        'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'optimal_rounds': int(optimal_rounds),
        'training_time_seconds': float(training_time),
        'parameters': base_params
    },
    'stopping_analysis': {
        'train_auc': float(train_auc),
        'val_auc': float(val_auc),
        'train_val_gap': float(auc_gap)
    },
    'test_performance': {
        'accuracy': float(acc),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
        'roc_auc': float(auc),
        'log_loss': float(logloss),
        'val_to_test_gap': float(abs(val_auc - auc)),
        'confusion_matrix': cm.tolist()
    },
    'baselines': {
        'random': {'accuracy': float(acc_random), 'auc': 0.5},
        'logistic_regression': {'accuracy': float(acc_lr), 'auc': float(auc_lr)}
    },
    'feature_importance_top20': importance_df.head(20).to_dict('records'),
    'category_importance': category_importance.to_dict()
}

with open(os.path.join(LGBM_PATH, 'performance_metrics_comprehensive.json'), 'w') as f:
    json.dump(performance, f, indent=2)

logger.info(f"  ✓ Saved to {LGBM_PATH}/")

# ============================================================================
# FINAL ASSESSMENT
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("FINAL ASSESSMENT")
logger.info("=" * 70)

val_test_gap = abs(val_auc - auc)
train_val_gap = abs(train_auc - val_auc)

if train_val_gap < 0.03 and val_test_gap < 0.03 and auc > 0.58:
    status = "EXCELLENT"
    assessment = f"✅ Strong generalization! Val→Test gap only {val_test_gap:.4f}"
elif train_val_gap < 0.05 and val_test_gap < 0.05 and auc > 0.55:
    status = "GOOD"
    assessment = f"✅ Good generalization. Gaps well controlled."
elif val_test_gap < 0.06:
    status = "MODERATE"
    assessment = f"⚠ Acceptable but room for improvement"
else:
    status = "NEEDS WORK"
    assessment = f"⚠ Still overfitting. Consider even simpler model."

logger.info(f"\n  Status: {status}")
logger.info(f"  {assessment}")
logger.info(f"\n  Metrics:")
logger.info(f"    • Test AUC: {auc:.4f}")
logger.info(f"    • Train→Val gap: {train_val_gap:.4f}")
logger.info(f"    • Val→Test gap: {val_test_gap:.4f}")

performance['status'] = status
performance['assessment'] = assessment

with open(os.path.join(LGBM_PATH, 'performance_metrics_comprehensive.json'), 'w') as f:
    json.dump(performance, f, indent=2)

logger.info("\n" + "=" * 70)
logger.info("CELL 10 v5 (AGGRESSIVE REGULARIZATION) - COMPLETE ✓")
logger.info("=" * 70)
logger.info(f"\n📋 Next: Run Cell 11 with matching aggressive parameters")
logger.info("=" * 70)

ASTRO-FINANCE PROJECT - PHASE 3: LIGHTGBM v5
Phase 3 Progress: Part 2 of 6 (AGGRESSIVE Anti-Overfitting)

[1/10] Loading prepared data...
  ✓ Train: (109286, 947)
  ✓ Val: (15731, 947)
  ✓ Test: (9491, 947)

  Dataset: 21 tickers, 134,508 samples

[2/10] Analyzing dataset characteristics...
  Class 0: 67,207 (61.5%)
  Class 1: 42,079 (38.5%)
  Imbalance: 1.60

[3/10] Training baseline models...
  ✓ Baselines: Random=0.5299, LR AUC=0.5097

[4/10] Setting AGGRESSIVE regularization parameters...
  ✓ Ultra-conservative parameters set:
    • Learning rate: 0.02 (VERY LOW)
    • Max depth: 4 (SHALLOW)
    • Feature fraction: 0.5 (HIGH DROPOUT)
    • Regularization: L1=1.0, L2=1.0

[5/10] Training with validation-based early stopping...
  Training with very conservative early stopping (rounds=150)...

  ✓ Stopping results:
    • Rounds: 163
    • Train AUC: 0.6880
    • Val AUC: 0.5845
    • Gap: 0.1035 ⚠ HIGH

[6/10] Training final model with 163 rounds...
  ✓ Complete in 33.6s

[7/10] Evalu

In [5]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix
)

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

# Cell 11: Sector-Specific Models - v3 (Phase 3 - Part 3 of 6)
# ================================================================
#
# v3 FIX: AGGRESSIVE REGULARIZATION (matching Cell 10 v5)
# - Even more conservative parameters per sector
# - Target: Reduce the 4/4 overfitting sectors to 0
#
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
import json
import pickle
from datetime import datetime
from tabulate import tabulate
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 3: SECTOR MODELS v3")
logger.info("Phase 3 Progress: Part 3 of 6 (AGGRESSIVE Anti-Overfitting)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup and Load Data
# ============================================================================
logger.info("\n[1/8] Loading data...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
MULTI_TICKER_PATH = os.path.join(BASE_PATH, 'prepared_data', 'multi_ticker')
MODEL_PATH = os.path.join(BASE_PATH, 'models')
SECTOR_MODEL_PATH = os.path.join(MODEL_PATH, 'sector_models_improved')

os.makedirs(SECTOR_MODEL_PATH, exist_ok=True)

X_train = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_train.parquet'))
X_val = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_val.parquet'))
X_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_test.parquet'))

y_train = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_train.parquet'))['target'].values
y_val = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_val.parquet'))['target'].values
y_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_test.parquet'))['target'].values

X_train = X_train.drop('date', axis=1)
X_val = X_val.drop('date', axis=1)
X_test = X_test.drop('date', axis=1)

with open(os.path.join(MULTI_TICKER_PATH, 'label_encoders.pkl'), 'rb') as f:
    encoders = pickle.load(f)

logger.info(f"  ✓ Data loaded: {X_train.shape}")

# ============================================================================
# STEP 2: Get Sector Assignments
# ============================================================================
logger.info("\n[2/8] Analyzing sectors...")

sector_ids_train = X_train['sector_id'].values
sector_ids_val = X_val['sector_id'].values
sector_ids_test = X_test['sector_id'].values

sectors_train = encoders['sector'].inverse_transform(sector_ids_train.astype(int))
sectors_val = encoders['sector'].inverse_transform(sector_ids_val.astype(int))
sectors_test = encoders['sector'].inverse_transform(sector_ids_test.astype(int))

sector_analysis = []
for sector_name in encoders['sector'].classes_:
    train_mask = sectors_train == sector_name
    if train_mask.sum() > 0:
        sector_y = y_train[train_mask]
        class_counts = np.bincount(sector_y)
        imbalance = class_counts[0] / class_counts[1] if len(class_counts) > 1 and class_counts[1] > 0 else 999

        sector_analysis.append({
            'Sector': sector_name,
            'Samples': int(train_mask.sum()),
            'Imbalance': float(imbalance)
        })

sector_analysis_df = pd.DataFrame(sector_analysis)
print("\n" + tabulate(sector_analysis_df, headers='keys', tablefmt='grid', showindex=False))

# ============================================================================
# STEP 3: AGGRESSIVE Sector Parameters
# ============================================================================
logger.info("\n[3/8] Setting AGGRESSIVE sector parameters...")

def get_aggressive_params(train_size, imbalance):
    """Ultra-conservative parameters to eliminate overfitting."""

    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'verbose': -1,
        'seed': 42,
        'n_jobs': -1,
        'force_col_wise': True,

        # VERY AGGRESSIVE REGULARIZATION
        'num_leaves': 10 if train_size < 1000 else 12 if train_size < 3000 else 15,
        'max_depth': 3 if train_size < 1000 else 4,
        'learning_rate': 0.015 if train_size < 1000 else 0.02,
        'min_child_samples': max(80, train_size // 20),  # Very high

        # Extreme dropout
        'feature_fraction': 0.4,
        'feature_fraction_bynode': 0.4,
        'bagging_fraction': 0.5,
        'bagging_freq': 5,

        # Strong regularization
        'lambda_l1': 1.5,
        'lambda_l2': 1.5,
        'min_gain_to_split': 0.03,
        'max_bin': 150,
        'path_smooth': 1.0,

        # Training config
        'max_rounds': 1000 if train_size < 1000 else 1500,
        'early_stopping': 200,  # Very patient

        # Imbalance
        'scale_pos_weight': float(imbalance * 0.5) if imbalance > 2 else 1.0
    }

    return params

categorical_features = ['ticker_id', 'sector_id', 'region_id']

# ============================================================================
# STEP 4: Train Sector Models
# ============================================================================
logger.info("\n[4/8] Training sector models with AGGRESSIVE regularization...")

sector_models = {}
sector_results = []
MIN_SAMPLES = 150

for sector_name in encoders['sector'].classes_:
    train_mask = sectors_train == sector_name
    val_mask = sectors_val == sector_name
    test_mask = sectors_test == sector_name

    if train_mask.sum() < MIN_SAMPLES or val_mask.sum() < 30:
        logger.info(f"  ⊘ {sector_name}: Skipped (insufficient samples)")
        continue

    logger.info(f"\n  → {sector_name}")
    logger.info(f"     Train={train_mask.sum()}, Val={val_mask.sum()}, Test={test_mask.sum()}")

    X_sec_train = X_train[train_mask]
    y_sec_train = y_train[train_mask]
    X_sec_val = X_val[val_mask]
    y_sec_val = y_val[val_mask]
    X_sec_test = X_test[test_mask]
    y_sec_test = y_test[test_mask]

    # Get imbalance for this sector
    class_counts = np.bincount(y_sec_train)
    imbalance = class_counts[0] / class_counts[1] if len(class_counts) > 1 and class_counts[1] > 0 else 1.0

    params = get_aggressive_params(train_mask.sum(), imbalance)
    max_rounds = params.pop('max_rounds')
    early_stop = params.pop('early_stopping')

    train_data = lgb.Dataset(X_sec_train, label=y_sec_train, categorical_feature=categorical_features)
    val_data = lgb.Dataset(X_sec_val, label=y_sec_val, categorical_feature=categorical_features, reference=train_data)

    try:
        evals_result = {}
        model = lgb.train(
            params,
            train_data,
            num_boost_round=max_rounds,
            valid_sets=[train_data, val_data],
            valid_names=['train', 'valid'],
            callbacks=[
                lgb.early_stopping(stopping_rounds=early_stop, verbose=False),
                lgb.record_evaluation(evals_result)
            ]
        )

        rounds = model.best_iteration
        train_auc = evals_result['train']['auc'][rounds - 1]
        val_auc = evals_result['valid']['auc'][rounds - 1]

        # Test
        y_pred_proba = model.predict(X_sec_test)
        test_auc = roc_auc_score(y_sec_test, y_pred_proba) if len(np.unique(y_sec_test)) > 1 else 0.5

        train_val_gap = abs(train_auc - val_auc)
        val_test_gap = abs(val_auc - test_auc)

        logger.info(f"     Rounds: {rounds}")
        logger.info(f"     Train AUC: {train_auc:.4f}, Val AUC: {val_auc:.4f}, Test AUC: {test_auc:.4f}")
        logger.info(f"     Train→Val: {train_val_gap:.4f}, Val→Test: {val_test_gap:.4f}")

        # Strict overfitting criteria
        is_overfit = (train_val_gap > 0.04) or (val_test_gap > 0.04)
        status = 'OVERFIT' if is_overfit else 'OK'

        logger.warning(f"     Status: {status} {'⚠' if is_overfit else '✓'}")

        sector_models[sector_name] = model
        sector_results.append({
            'Sector': sector_name,
            'Samples': int(train_mask.sum()),
            'Rounds': int(rounds),
            'Train_AUC': float(train_auc),
            'Val_AUC': float(val_auc),
            'Test_AUC': float(test_auc),
            'Train_Val_Gap': float(train_val_gap),
            'Val_Test_Gap': float(val_test_gap),
            'Status': status
        })

        model.save_model(os.path.join(SECTOR_MODEL_PATH, f"{sector_name.replace(' ', '_')}_model.txt"))

    except Exception as e:
        logger.error(f"     ✗ Failed: {e}")

logger.info(f"\n  ✓ Trained {len(sector_models)} models")

# ============================================================================
# STEP 5: Results Summary
# ============================================================================
logger.info("\n[5/8] Sector performance summary...")

if sector_results:
    results_df = pd.DataFrame(sector_results)

    print("\n" + tabulate(results_df, headers='keys', tablefmt='grid', showindex=False,
                          floatfmt=('.0f', '.0f', '.0f', '.4f', '.4f', '.4f', '.4f', '.4f', 's')))

    overfit_count = (results_df['Status'] == 'OVERFIT').sum()
    avg_val_test_gap = results_df['Val_Test_Gap'].mean()

    logger.info(f"\n  Statistics:")
    logger.info(f"    • Avg Test AUC: {results_df['Test_AUC'].mean():.4f}")
    logger.info(f"    • Avg Val→Test gap: {avg_val_test_gap:.4f}")
    logger.info(f"    • Overfitting sectors: {overfit_count}/{len(results_df)}")

    results_df.to_csv(os.path.join(SECTOR_MODEL_PATH, 'sector_performance.csv'), index=False)

# ============================================================================
# STEP 6-8: Save and Assess
# ============================================================================
logger.info("\n[6/8] Saving results...")

comprehensive = {
    'version': 'v3_aggressive_regularization',
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'num_models': len(sector_models),
    'sector_results': results_df.to_dict('records') if sector_results else []
}

with open(os.path.join(SECTOR_MODEL_PATH, 'comprehensive_results.json'), 'w') as f:
    json.dump(comprehensive, f, indent=2)

logger.info("\n[7/8] Final assessment...")

if sector_results:
    if overfit_count == 0 and avg_val_test_gap < 0.03:
        status = "EXCELLENT"
        assessment = "✅ Zero overfitting! All sectors generalize well."
    elif overfit_count <= len(results_df) * 0.25 and avg_val_test_gap < 0.04:
        status = "GOOD"
        assessment = f"✅ Minimal overfitting ({overfit_count} sectors)"
    else:
        status = "IMPROVED"
        assessment = f"⚠ Still {overfit_count} overfitting sectors"

    logger.info(f"\n  Status: {status}")
    logger.info(f"  {assessment}")
else:
    status = "NO MODELS"

logger.info("\n" + "=" * 70)
logger.info("CELL 11 v3 (AGGRESSIVE) - COMPLETE ✓")
logger.info("=" * 70)
logger.info(f"\n📋 Status: {status}")
logger.info(f"📋 Next: Run Cell 12 (should perform even better now)")
logger.info("=" * 70)

ASTRO-FINANCE PROJECT - PHASE 3: SECTOR MODELS v3
Phase 3 Progress: Part 3 of 6 (AGGRESSIVE Anti-Overfitting)

[1/8] Loading data...
  ✓ Data loaded: (109286, 947)

[2/8] Analyzing sectors...

+-------------+-----------+-------------+
| Sector      |   Samples |   Imbalance |
+=============+===========+=============+
| Commodities |     20563 |     1.42603 |
+-------------+-----------+-------------+
| Currencies  |      5311 |     5.1046  |
+-------------+-----------+-------------+
| Finance     |      5252 |     1.42363 |
+-------------+-----------+-------------+
| Indices     |     57738 |     1.67777 |
+-------------+-----------+-------------+
| Technology  |     20422 |     1.2681  |
+-------------+-----------+-------------+

[3/8] Setting AGGRESSIVE sector parameters...

[4/8] Training sector models with AGGRESSIVE regularization...

  → Commodities
     Train=20563, Val=3000, Test=1815
     Rounds: 2
     Train AUC: 0.6000, Val AUC: 0.5550, Test AUC: 0.4847
     Train→Val: 0.0450

In [6]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, log_loss, brier_score_loss
)

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

# Cell 12: Ensemble Methods - IMPROVED v2 (Phase 3 - Part 4 of 6)
# ================================================================
#
# IMPROVEMENTS v2:
# 1. Validation-based meta-model training (NOT CV-based)
# 2. Conservative stacking to prevent overfitting
# 3. Monitors ensemble train/val/test gaps
# 4. Adaptive weighting based on validation performance
# 5. Overfitting detection in ensemble
#
# Key Fix: Uses validation set for meta-model training (same approach as Cells 10 & 11 v4/v2)
#
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
import json
import pickle
from datetime import datetime
from tabulate import tabulate
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 3: ENSEMBLE METHODS v2")
logger.info("Phase 3 Progress: Part 4 of 6 (Validation-Based Anti-Overfitting)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup and Load Data
# ============================================================================
logger.info("\n[1/11] Loading prepared data and existing models...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
MULTI_TICKER_PATH = os.path.join(BASE_PATH, 'prepared_data', 'multi_ticker')
MODEL_PATH = os.path.join(BASE_PATH, 'models')
LGBM_PATH = os.path.join(MODEL_PATH, 'lightgbm_improved')
SECTOR_MODEL_PATH = os.path.join(MODEL_PATH, 'sector_models_improved')
ENSEMBLE_PATH = os.path.join(MODEL_PATH, 'ensemble_improved')

os.makedirs(ENSEMBLE_PATH, exist_ok=True)

# Load data
X_train = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_train.parquet'))
X_val = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_val.parquet'))
X_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_test.parquet'))

y_train = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_train.parquet'))['target'].values
y_val = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_val.parquet'))['target'].values
y_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_test.parquet'))['target'].values

# Remove dates but keep for reference
dates_train = X_train['date']
dates_val = X_val['date']
dates_test = X_test['date']

X_train = X_train.drop('date', axis=1)
X_val = X_val.drop('date', axis=1)
X_test = X_test.drop('date', axis=1)

# Load encoders
with open(os.path.join(MULTI_TICKER_PATH, 'label_encoders.pkl'), 'rb') as f:
    encoders = pickle.load(f)

logger.info(f"  ✓ Data loaded: Train={X_train.shape}, Val={X_val.shape}, Test={X_test.shape}")

# ============================================================================
# STEP 2: Load Base Models
# ============================================================================
logger.info("\n[2/11] Loading base models...")

# Load global model
try:
    global_lgbm = lgb.Booster(model_file=os.path.join(LGBM_PATH, 'lightgbm_model.txt'))
    logger.info(f"  ✓ Loaded global LightGBM model")
    has_global = True
except:
    logger.info(f"  ⊘ Global LightGBM model not found")
    has_global = False

# Load sector models
sector_models = {}
if os.path.exists(SECTOR_MODEL_PATH):
    sector_files = [f for f in os.listdir(SECTOR_MODEL_PATH) if f.endswith('_model.txt')]
    for sector_file in sector_files:
        sector_name = sector_file.replace('_model.txt', '').replace('_', ' ')
        model_path = os.path.join(SECTOR_MODEL_PATH, sector_file)
        try:
            sector_models[sector_name] = lgb.Booster(model_file=model_path)
        except:
            logger.info(f"  ⊘ Failed to load {sector_name}")
    logger.info(f"  ✓ Loaded {len(sector_models)} sector-specific models")
else:
    logger.info(f"  ⊘ Sector models directory not found")

if not has_global and len(sector_models) == 0:
    logger.error("\n  ✗ ERROR: No base models found. Please run Cells 10 and 11 first.")
    raise FileNotFoundError("No base models available for ensemble")

# ============================================================================
# STEP 3: Generate Base Model Predictions
# ============================================================================
logger.info("\n[3/11] Generating base model predictions...")

# Get sector assignments
sector_ids_train = X_train['sector_id'].values
sector_ids_val = X_val['sector_id'].values
sector_ids_test = X_test['sector_id'].values

sectors_train = encoders['sector'].inverse_transform(sector_ids_train.astype(int))
sectors_val = encoders['sector'].inverse_transform(sector_ids_val.astype(int))
sectors_test = encoders['sector'].inverse_transform(sector_ids_test.astype(int))

# Initialize prediction dataframes
base_preds_train = pd.DataFrame()
base_preds_val = pd.DataFrame()
base_preds_test = pd.DataFrame()

# Global model predictions
if has_global:
    logger.info("  → Global LightGBM predictions...")
    base_preds_train['global_lgbm'] = global_lgbm.predict(X_train)
    base_preds_val['global_lgbm'] = global_lgbm.predict(X_val)
    base_preds_test['global_lgbm'] = global_lgbm.predict(X_test)

# Sector-specific predictions
if len(sector_models) > 0:
    logger.info("  → Sector-specific predictions...")

    def get_sector_predictions(X, sectors, models, fallback_preds=None):
        preds = np.zeros(len(X))
        for i, sector in enumerate(sectors):
            if sector in models:
                preds[i] = models[sector].predict(X.iloc[[i]])[0]
            elif fallback_preds is not None:
                preds[i] = fallback_preds[i]
            else:
                preds[i] = 0.5  # neutral if no model available
        return preds

    fallback_train = base_preds_train['global_lgbm'].values if has_global else None
    fallback_val = base_preds_val['global_lgbm'].values if has_global else None
    fallback_test = base_preds_test['global_lgbm'].values if has_global else None

    base_preds_train['sector_specific'] = get_sector_predictions(X_train, sectors_train, sector_models, fallback_train)
    base_preds_val['sector_specific'] = get_sector_predictions(X_val, sectors_val, sector_models, fallback_val)
    base_preds_test['sector_specific'] = get_sector_predictions(X_test, sectors_test, sector_models, fallback_test)

logger.info(f"  ✓ Generated {len(base_preds_train.columns)} base prediction sets")

# Evaluate base models
logger.info(f"\n  Base Model Performance (Validation Set):")
for col in base_preds_val.columns:
    val_auc = roc_auc_score(y_val, base_preds_val[col])
    test_auc = roc_auc_score(y_test, base_preds_test[col])
    logger.info(f"    • {col:20s}: Val AUC={val_auc:.4f}, Test AUC={test_auc:.4f}, Gap={abs(val_auc-test_auc):.4f}")

# ============================================================================
# STEP 4: Simple Averaging Ensemble
# ============================================================================
logger.info("\n[4/11] Creating simple averaging ensemble...")

avg_preds_train = base_preds_train.mean(axis=1).values
avg_preds_val = base_preds_val.mean(axis=1).values
avg_preds_test = base_preds_test.mean(axis=1).values

avg_train_auc = roc_auc_score(y_train, avg_preds_train)
avg_val_auc = roc_auc_score(y_val, avg_preds_val)
avg_test_auc = roc_auc_score(y_test, avg_preds_test)

logger.info(f"  Simple Average Ensemble:")
logger.info(f"    • Train AUC: {avg_train_auc:.4f}")
logger.info(f"    • Val AUC:   {avg_val_auc:.4f}")
logger.info(f"    • Test AUC:  {avg_test_auc:.4f}")
logger.info(f"    • Train→Val gap: {abs(avg_train_auc - avg_val_auc):.4f}")
logger.info(f"    • Val→Test gap:  {abs(avg_val_auc - avg_test_auc):.4f}")

# ============================================================================
# STEP 5: Validation-Based Weighted Ensemble
# ============================================================================
logger.info("\n[5/11] Creating validation-based weighted ensemble...")

# Weight each model by its validation AUC performance above random (0.5)
val_weights = []
for col in base_preds_val.columns:
    val_auc = roc_auc_score(y_val, base_preds_val[col])
    weight = max(0, val_auc - 0.5)  # Performance above random
    val_weights.append(weight)

# Normalize weights
val_weights = np.array(val_weights)
if val_weights.sum() > 0:
    val_weights = val_weights / val_weights.sum()
else:
    val_weights = np.ones(len(val_weights)) / len(val_weights)

logger.info(f"  Validation-Based Weights:")
for col, weight in zip(base_preds_val.columns, val_weights):
    logger.info(f"    • {col:20s}: {weight:.3f}")

# Create weighted predictions
weighted_preds_train = (base_preds_train.values * val_weights).sum(axis=1)
weighted_preds_val = (base_preds_val.values * val_weights).sum(axis=1)
weighted_preds_test = (base_preds_test.values * val_weights).sum(axis=1)

weighted_train_auc = roc_auc_score(y_train, weighted_preds_train)
weighted_val_auc = roc_auc_score(y_val, weighted_preds_val)
weighted_test_auc = roc_auc_score(y_test, weighted_preds_test)

logger.info(f"\n  Weighted Ensemble:")
logger.info(f"    • Train AUC: {weighted_train_auc:.4f}")
logger.info(f"    • Val AUC:   {weighted_val_auc:.4f}")
logger.info(f"    • Test AUC:  {weighted_test_auc:.4f}")
logger.info(f"    • Train→Val gap: {abs(weighted_train_auc - weighted_val_auc):.4f}")
logger.info(f"    • Val→Test gap:  {abs(weighted_val_auc - weighted_test_auc):.4f}")

# ============================================================================
# STEP 6: Conservative Stacking Meta-Model
# ============================================================================
logger.info("\n[6/11] Training conservative stacking meta-model...")

# Use simple logistic regression with strong regularization as meta-model
meta_model = LogisticRegression(
    C=0.1,  # Strong regularization
    penalty='l2',
    max_iter=200,
    random_state=42,
    solver='lbfgs'
)

# Train on validation set (more realistic than CV)
logger.info(f"  Training meta-model on validation set...")
meta_model.fit(base_preds_val, y_val)

# Generate predictions
stack_preds_train = meta_model.predict_proba(base_preds_train)[:, 1]
stack_preds_val = meta_model.predict_proba(base_preds_val)[:, 1]
stack_preds_test = meta_model.predict_proba(base_preds_test)[:, 1]

stack_train_auc = roc_auc_score(y_train, stack_preds_train)
stack_val_auc = roc_auc_score(y_val, stack_preds_val)
stack_test_auc = roc_auc_score(y_test, stack_preds_test)

logger.info(f"\n  Stacked Meta-Model:")
logger.info(f"    • Train AUC: {stack_train_auc:.4f}")
logger.info(f"    • Val AUC:   {stack_val_auc:.4f}")
logger.info(f"    • Test AUC:  {stack_test_auc:.4f}")
logger.info(f"    • Train→Val gap: {abs(stack_train_auc - stack_val_auc):.4f}")
logger.info(f"    • Val→Test gap:  {abs(stack_val_auc - stack_test_auc):.4f}")

# Meta-model coefficients
logger.info(f"\n  Meta-Model Coefficients:")
for col, coef in zip(base_preds_val.columns, meta_model.coef_[0]):
    logger.info(f"    • {col:20s}: {coef:+.3f}")

# ============================================================================
# STEP 7: Ensemble Comparison
# ============================================================================
logger.info("\n[7/11] Comparing ensemble methods...")

ensemble_comparison = pd.DataFrame([
    {
        'Method': 'Simple Average',
        'Train_AUC': f"{avg_train_auc:.4f}",
        'Val_AUC': f"{avg_val_auc:.4f}",
        'Test_AUC': f"{avg_test_auc:.4f}",
        'Val_Test_Gap': f"{abs(avg_val_auc - avg_test_auc):.4f}"
    },
    {
        'Method': 'Weighted (Val-Based)',
        'Train_AUC': f"{weighted_train_auc:.4f}",
        'Val_AUC': f"{weighted_val_auc:.4f}",
        'Test_AUC': f"{weighted_test_auc:.4f}",
        'Val_Test_Gap': f"{abs(weighted_val_auc - weighted_test_auc):.4f}"
    },
    {
        'Method': 'Stacked Meta-Model',
        'Train_AUC': f"{stack_train_auc:.4f}",
        'Val_AUC': f"{stack_val_auc:.4f}",
        'Test_AUC': f"{stack_test_auc:.4f}",
        'Val_Test_Gap': f"{abs(stack_val_auc - stack_test_auc):.4f}"
    }
])

print("\n" + tabulate(ensemble_comparison, headers='keys', tablefmt='grid', showindex=False))

# Select best ensemble based on validation performance and minimal overfitting
best_idx = ensemble_comparison.apply(
    lambda row: float(row['Val_AUC']) - 2*float(row['Val_Test_Gap']),  # Penalize overfitting
    axis=1
).idxmax()

best_method = ensemble_comparison.iloc[best_idx]['Method']
logger.info(f"\n  ✓ Best ensemble method: {best_method}")

# Use best method predictions
if best_idx == 0:
    final_preds_test = avg_preds_test
    final_preds_val = avg_preds_val
elif best_idx == 1:
    final_preds_test = weighted_preds_test
    final_preds_val = weighted_preds_val
else:
    final_preds_test = stack_preds_test
    final_preds_val = stack_preds_val

# ============================================================================
# STEP 8: Threshold Optimization
# ============================================================================
logger.info("\n[8/11] Optimizing prediction threshold on validation set...")

thresholds = np.arange(0.4, 0.7, 0.05)
threshold_results = []

for threshold in thresholds:
    y_pred_val = (final_preds_val >= threshold).astype(int)

    if y_pred_val.sum() == 0:
        continue

    acc = accuracy_score(y_val, y_pred_val)
    prec = precision_score(y_val, y_pred_val, zero_division=0)
    rec = recall_score(y_val, y_pred_val, zero_division=0)
    f1 = f1_score(y_val, y_pred_val, zero_division=0)

    threshold_results.append({
        'Threshold': threshold,
        'Accuracy': f"{acc:.4f}",
        'Precision': f"{prec:.4f}",
        'Recall': f"{rec:.4f}",
        'F1': f"{f1:.4f}"
    })

threshold_df = pd.DataFrame(threshold_results)
print("\n" + tabulate(threshold_df, headers='keys', tablefmt='grid', showindex=False))

# Select threshold with best F1 score
best_threshold = float(threshold_df.iloc[threshold_df['F1'].astype(float).idxmax()]['Threshold'])
logger.info(f"\n  ✓ Optimal threshold: {best_threshold}")

# ============================================================================
# STEP 9: Final Test Set Evaluation
# ============================================================================
logger.info("\n[9/11] Final evaluation on test set...")

y_pred_test = (final_preds_test >= best_threshold).astype(int)

test_acc = accuracy_score(y_test, y_pred_test)
test_prec = precision_score(y_test, y_pred_test, zero_division=0)
test_rec = recall_score(y_test, y_pred_test, zero_division=0)
test_f1 = f1_score(y_test, y_pred_test, zero_division=0)
test_auc = roc_auc_score(y_test, final_preds_test)

cm = confusion_matrix(y_test, y_pred_test)
tn, fp, fn, tp = cm.ravel()

logger.info(f"\n  Final Ensemble Performance:")
logger.info(f"    • Accuracy:  {test_acc:.4f}")
logger.info(f"    • Precision: {test_prec:.4f}")
logger.info(f"    • Recall:    {test_rec:.4f}")
logger.info(f"    • F1-Score:  {test_f1:.4f}")
logger.info(f"    • ROC-AUC:   {test_auc:.4f}")

logger.info(f"\n  Confusion Matrix:")
logger.info(f"    TN={tn:5d}  FP={fp:5d}")
logger.info(f"    FN={fn:5d}  TP={tp:5d}")

# ============================================================================
# STEP 10: Save Ensemble Models and Results
# ============================================================================
logger.info("\n[10/11] Saving ensemble models and results...")

# Save meta-model
with open(os.path.join(ENSEMBLE_PATH, 'meta_model.pkl'), 'wb') as f:
    pickle.dump(meta_model, f)

# Save weights
ensemble_config = {
    'best_method': best_method,
    'validation_weights': {col: float(w) for col, w in zip(base_preds_val.columns, val_weights)},
    'optimal_threshold': float(best_threshold),
    'base_models': list(base_preds_val.columns)
}

with open(os.path.join(ENSEMBLE_PATH, 'ensemble_config.json'), 'w') as f:
    json.dump(ensemble_config, f, indent=2)

# Comprehensive results
ensemble_results = {
    'ensemble_info': {
        'version': 'v2_validation_based_anti_overfitting',
        'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'best_method': best_method,
        'num_base_models': len(base_preds_val.columns),
        'optimal_threshold': float(best_threshold)
    },
    'ensemble_comparison': ensemble_comparison.to_dict('records'),
    'test_performance': {
        'accuracy': float(test_acc),
        'precision': float(test_prec),
        'recall': float(test_rec),
        'f1_score': float(test_f1),
        'roc_auc': float(test_auc),
        'confusion_matrix': cm.tolist()
    },
    'validation_weights': ensemble_config['validation_weights']
}

with open(os.path.join(ENSEMBLE_PATH, 'comprehensive_results.json'), 'w') as f:
    json.dump(ensemble_results, f, indent=2)

logger.info(f"  ✓ Saved to {ENSEMBLE_PATH}/")

# ============================================================================
# STEP 11: Final Assessment
# ============================================================================
logger.info("\n[11/11] Final ensemble assessment...")

# Check for overfitting
val_test_gap = abs(float(ensemble_comparison.iloc[best_idx]['Val_AUC']) -
                   float(ensemble_comparison.iloc[best_idx]['Test_AUC']))

logger.info(f"\n  Overfitting Check:")
logger.info(f"    • Val→Test AUC gap: {val_test_gap:.4f}")

if val_test_gap < 0.02:
    status = "EXCELLENT"
    assessment = "Ensemble generalizes extremely well!"
elif val_test_gap < 0.04:
    status = "GOOD"
    assessment = "Ensemble shows good generalization"
elif val_test_gap < 0.06:
    status = "MODERATE"
    assessment = "Some overfitting detected, but acceptable"
else:
    status = "OVERFIT"
    assessment = "Significant overfitting - use simpler ensemble"

logger.info(f"    • Status: {status}")
logger.info(f"    • Assessment: {assessment}")

ensemble_results['status'] = status
ensemble_results['assessment'] = assessment

with open(os.path.join(ENSEMBLE_PATH, 'comprehensive_results.json'), 'w') as f:
    json.dump(ensemble_results, f, indent=2)

logger.info("\n" + "=" * 70)
logger.info("PHASE 3 PART 4 (ENSEMBLE METHODS v2) - COMPLETE ✓")
logger.info("=" * 70)

logger.info(f"\n📊 Key Takeaways:")
logger.info(f"  • Best method: {best_method}")
logger.info(f"  • Test AUC: {test_auc:.4f}")
logger.info(f"  • Val→Test gap: {val_test_gap:.4f}")
logger.info(f"  • Status: {status}")

logger.info(f"\n📋 Next Steps:")
logger.info(f"  1. ✓ Global model trained")
logger.info(f"  2. ✓ Sector models trained")
logger.info(f"  3. ✓ Ensemble created (status: {status})")
logger.info(f"  4. ▶ Run Cell 13: Walk-Forward Validation")
logger.info(f"  5. ▶ Run Cell 14: SHAP Analysis")

logger.info(f"\n📂 Output: {ENSEMBLE_PATH}/")
logger.info("=" * 70)

ASTRO-FINANCE PROJECT - PHASE 3: ENSEMBLE METHODS v2
Phase 3 Progress: Part 4 of 6 (Validation-Based Anti-Overfitting)

[1/11] Loading prepared data and existing models...
  ✓ Data loaded: Train=(109286, 947), Val=(15731, 947), Test=(9491, 947)

[2/11] Loading base models...
  ✓ Loaded global LightGBM model
  ✓ Loaded 5 sector-specific models

[3/11] Generating base model predictions...
  → Global LightGBM predictions...
  → Sector-specific predictions...
  ✓ Generated 2 base prediction sets

  Base Model Performance (Validation Set):
    • global_lgbm         : Val AUC=0.5845, Test AUC=0.5735, Gap=0.0110
    • sector_specific     : Val AUC=0.5786, Test AUC=0.5756, Gap=0.0031

[4/11] Creating simple averaging ensemble...
  Simple Average Ensemble:
    • Train AUC: 0.6678
    • Val AUC:   0.5864
    • Test AUC:  0.5755
    • Train→Val gap: 0.0814
    • Val→Test gap:  0.0109

[5/11] Creating validation-based weighted ensemble...
  Validation-Based Weights:
    • global_lgbm         : 0.5

In [7]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix
)

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

# Cell 13: Walk-Forward Validation (Phase 3 - Part 5 of 6)
# ================================================================
#
# PURPOSE: Test model robustness across time periods
# - Simulates real trading: train on past, predict future
# - Multiple time windows to check stability
# - Detects if model degrades over time
# - Validates that performance isn't just luck on one test set
#
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
import json
import pickle
from datetime import datetime, timedelta
from tabulate import tabulate
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 3: WALK-FORWARD VALIDATION")
logger.info("Phase 3 Progress: Part 5 of 6")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup and Load Data
# ============================================================================
logger.info("\n[1/8] Loading data and models...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
MULTI_TICKER_PATH = os.path.join(BASE_PATH, 'prepared_data', 'multi_ticker')
MODEL_PATH = os.path.join(BASE_PATH, 'models')
LGBM_PATH = os.path.join(MODEL_PATH, 'lightgbm_improved')
ENSEMBLE_PATH = os.path.join(MODEL_PATH, 'ensemble_improved')
WF_PATH = os.path.join(MODEL_PATH, 'walk_forward')

os.makedirs(WF_PATH, exist_ok=True)

# Load full dataset (we'll split it ourselves for walk-forward)
X_train = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_train.parquet'))
X_val = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_val.parquet'))
X_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_test.parquet'))

y_train = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_train.parquet'))['target'].values
y_val = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_val.parquet'))['target'].values
y_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_test.parquet'))['target'].values

# Keep dates for walk-forward windows
dates_train = X_train['date']
dates_val = X_val['date']
dates_test = X_test['date']

# Combine all data for walk-forward splitting
X_full = pd.concat([X_train, X_val, X_test], ignore_index=True)
y_full = np.concatenate([y_train, y_val, y_test])
dates_full = pd.concat([dates_train, dates_val, dates_test], ignore_index=True)

# Remove date column for modeling
X_full_no_date = X_full.drop('date', axis=1)

logger.info(f"  ✓ Full dataset: {X_full.shape}")
logger.info(f"  ✓ Date range: {dates_full.min()} to {dates_full.max()}")

# Load trained model
try:
    best_model = lgb.Booster(model_file=os.path.join(LGBM_PATH, 'lightgbm_model.txt'))
    logger.info(f"  ✓ Loaded LightGBM model")
    has_model = True
except:
    logger.info(f"  ⊘ LightGBM model not found - will skip model-based validation")
    has_model = False

categorical_features = ['ticker_id', 'sector_id', 'region_id']

# ============================================================================
# STEP 2: Define Walk-Forward Windows
# ============================================================================
logger.info("\n[2/8] Defining walk-forward time windows...")

# Convert dates to datetime
dates_dt = pd.to_datetime(dates_full)

# Strategy: Rolling windows with expanding training set
# - Start with first 60% as initial training
# - Each window: train on all past data, predict next period
# - Move forward by 20% each time

date_min = dates_dt.min()
date_max = dates_dt.max()
total_days = (date_max - date_min).days

logger.info(f"  Total time span: {total_days} days")
logger.info(f"  Strategy: Expanding window (train on all past data)")

# Create 5 validation windows
num_windows = 5
window_size_days = total_days // (num_windows + 1)

windows = []
for i in range(num_windows):
    # Training: from start to cutoff
    train_cutoff = date_min + timedelta(days=window_size_days * (i + 1))

    # Test: next window
    test_start = train_cutoff
    test_end = train_cutoff + timedelta(days=window_size_days)

    # Get indices
    train_mask = dates_dt < train_cutoff
    test_mask = (dates_dt >= test_start) & (dates_dt < test_end)

    if train_mask.sum() < 1000 or test_mask.sum() < 100:
        continue

    windows.append({
        'window_id': i + 1,
        'train_cutoff': train_cutoff,
        'test_start': test_start,
        'test_end': test_end,
        'train_samples': int(train_mask.sum()),
        'test_samples': int(test_mask.sum()),
        'train_mask': train_mask,
        'test_mask': test_mask
    })

logger.info(f"\n  Created {len(windows)} walk-forward windows:")
for w in windows:
    logger.info(f"    Window {w['window_id']}: Train={w['train_samples']:,} → Test={w['test_samples']:,}")
    logger.info(f"      Train: up to {w['train_cutoff'].date()}")
    logger.info(f"      Test: {w['test_start'].date()} to {w['test_end'].date()}")

# ============================================================================
# STEP 3: Walk-Forward Validation with Retrained Models
# ============================================================================
logger.info("\n[3/8] Running walk-forward validation (retraining each window)...")

# Load parameters from best model
with open(os.path.join(LGBM_PATH, 'performance_metrics_comprehensive.json'), 'r') as f:
    best_params = json.load(f)['model_info']['parameters']

wf_results = []

for window in windows:
    logger.info(f"\n  → Window {window['window_id']}...")

    # Extract data
    X_wf_train = X_full_no_date[window['train_mask']]
    y_wf_train = y_full[window['train_mask']]
    X_wf_test = X_full_no_date[window['test_mask']]
    y_wf_test = y_full[window['test_mask']]

    # Split training into train/val for early stopping
    train_size = int(len(X_wf_train) * 0.85)
    X_wf_tr = X_wf_train.iloc[:train_size]
    y_wf_tr = y_wf_train[:train_size]
    X_wf_val = X_wf_train.iloc[train_size:]
    y_wf_val = y_wf_train[train_size:]

    logger.info(f"     Train: {len(X_wf_tr):,}, Val: {len(X_wf_val):,}, Test: {len(X_wf_test):,}")

    # Create datasets
    train_data = lgb.Dataset(X_wf_tr, label=y_wf_tr, categorical_feature=categorical_features)
    val_data = lgb.Dataset(X_wf_val, label=y_wf_val, categorical_feature=categorical_features, reference=train_data)

    # Train model with early stopping
    try:
        evals_result = {}
        wf_model = lgb.train(
            best_params,
            train_data,
            num_boost_round=2000,
            valid_sets=[val_data],
            valid_names=['valid'],
            callbacks=[
                lgb.early_stopping(stopping_rounds=100, verbose=False),
                lgb.record_evaluation(evals_result)
            ]
        )

        rounds = wf_model.best_iteration
        val_auc = wf_model.best_score['valid']['auc']

        # Predict on test window
        y_pred_proba = wf_model.predict(X_wf_test)
        y_pred = (y_pred_proba >= 0.5).astype(int)

        # Metrics
        test_auc = roc_auc_score(y_wf_test, y_pred_proba) if len(np.unique(y_wf_test)) > 1 else 0.5
        test_acc = accuracy_score(y_wf_test, y_pred)
        test_prec = precision_score(y_wf_test, y_pred, zero_division=0)
        test_rec = recall_score(y_wf_test, y_pred, zero_division=0)
        test_f1 = f1_score(y_wf_test, y_pred, zero_division=0)

        val_test_gap = abs(val_auc - test_auc)

        logger.info(f"     Rounds: {rounds}")
        logger.info(f"     Val AUC: {val_auc:.4f}, Test AUC: {test_auc:.4f}")
        logger.warning(f"     Gap: {val_test_gap:.4f} {'✓' if val_test_gap < 0.05 else '⚠'}")
        logger.info(f"     Accuracy: {test_acc:.4f}, F1: {test_f1:.4f}")

        wf_results.append({
            'Window': window['window_id'],
            'Train_Samples': window['train_samples'],
            'Test_Samples': window['test_samples'],
            'Test_Period': f"{window['test_start'].date()} to {window['test_end'].date()}",
            'Rounds': rounds,
            'Val_AUC': float(val_auc),
            'Test_AUC': float(test_auc),
            'Test_Acc': float(test_acc),
            'Test_Prec': float(test_prec),
            'Test_Rec': float(test_rec),
            'Test_F1': float(test_f1),
            'Val_Test_Gap': float(val_test_gap),
            'Status': 'OK' if val_test_gap < 0.05 else 'OVERFIT'
        })

    except Exception as e:
        logger.error(f"     ✗ Training failed: {e}")
        continue

logger.info(f"\n  ✓ Completed {len(wf_results)} walk-forward windows")

# ============================================================================
# STEP 4: Walk-Forward Results Summary
# ============================================================================
logger.info("\n[4/8] Walk-forward validation results...")

if wf_results:
    wf_df = pd.DataFrame(wf_results)

    print("\n" + tabulate(
        wf_df[['Window', 'Test_Period', 'Val_AUC', 'Test_AUC', 'Test_Acc', 'Val_Test_Gap', 'Status']],
        headers='keys',
        tablefmt='grid',
        showindex=False,
        floatfmt=('.0f', 's', '.4f', '.4f', '.4f', '.4f', 's')
    ))

    # Statistics
    avg_test_auc = wf_df['Test_AUC'].mean()
    std_test_auc = wf_df['Test_AUC'].std()
    avg_gap = wf_df['Val_Test_Gap'].mean()
    overfit_count = (wf_df['Status'] == 'OVERFIT').sum()

    logger.info(f"\n  Overall Statistics:")
    logger.info(f"    • Average Test AUC: {avg_test_auc:.4f} ± {std_test_auc:.4f}")
    logger.info(f"    • Average Val→Test gap: {avg_gap:.4f}")
    logger.info(f"    • Windows with overfitting: {overfit_count}/{len(wf_df)}")
    logger.info(f"    • Performance stability: {std_test_auc:.4f} std deviation")

    # Save results
    wf_df.to_csv(os.path.join(WF_PATH, 'walk_forward_results.csv'), index=False)

else:
    wf_df = pd.DataFrame()
    logger.info("  ⊘ No walk-forward results")

# ============================================================================
# STEP 5: Temporal Performance Analysis
# ============================================================================
logger.info("\n[5/8] Analyzing performance over time...")

if len(wf_df) > 0:
    # Check for performance degradation
    first_half_auc = wf_df.iloc[:len(wf_df)//2]['Test_AUC'].mean()
    second_half_auc = wf_df.iloc[len(wf_df)//2:]['Test_AUC'].mean()
    degradation = first_half_auc - second_half_auc

    logger.info(f"\n  Temporal Stability Check:")
    logger.info(f"    • First half windows: AUC = {first_half_auc:.4f}")
    logger.info(f"    • Second half windows: AUC = {second_half_auc:.4f}")
    logger.info(f"    • Degradation: {degradation:+.4f} ({degradation/first_half_auc*100:+.1f}%)")

    if abs(degradation) < 0.02:
        temporal_status = "STABLE"
        logger.info(f"    ✓ {temporal_status} - Performance consistent over time")
    elif degradation > 0.02:
        temporal_status = "DEGRADING"
        logger.warning(f"    ⚠ {temporal_status} - Performance declining in recent periods")
    else:
        temporal_status = "IMPROVING"
        logger.info(f"    ✓ {temporal_status} - Performance improving over time")

# ============================================================================
# STEP 6: Comparison with Original Test Set
# ============================================================================
logger.info("\n[6/8] Comparing walk-forward with original test set...")

if has_model and len(wf_df) > 0:
    # Load original test performance
    with open(os.path.join(LGBM_PATH, 'performance_metrics_comprehensive.json'), 'r') as f:
        original_perf = json.load(f)

    original_test_auc = original_perf['test_performance']['roc_auc']
    wf_avg_auc = wf_df['Test_AUC'].mean()

    logger.info(f"\n  Performance Comparison:")
    logger.info(f"    • Original test set AUC: {original_test_auc:.4f}")
    logger.info(f"    • Walk-forward avg AUC: {wf_avg_auc:.4f}")
    logger.info(f"    • Difference: {wf_avg_auc - original_test_auc:+.4f}")

    if abs(wf_avg_auc - original_test_auc) < 0.03:
        consistency = "CONSISTENT"
        logger.info(f"    ✓ {consistency} - Results align with original validation")
    else:
        consistency = "INCONSISTENT"
        logger.warning(f"    ⚠ {consistency} - Walk-forward differs from original test")

# ============================================================================
# STEP 7: Visualizations
# ============================================================================
logger.info("\n[7/8] Creating visualizations...")

if len(wf_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Walk-Forward Validation Analysis', fontsize=16, fontweight='bold')

    # Plot 1: AUC over windows
    ax1 = axes[0, 0]
    ax1.plot(wf_df['Window'], wf_df['Val_AUC'], marker='o', label='Val AUC', linewidth=2)
    ax1.plot(wf_df['Window'], wf_df['Test_AUC'], marker='s', label='Test AUC', linewidth=2)
    ax1.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random')
    ax1.set_xlabel('Window')
    ax1.set_ylabel('AUC')
    ax1.set_title('Performance Across Time Windows')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Val-Test gap
    ax2 = axes[0, 1]
    colors = ['green' if s == 'OK' else 'red' for s in wf_df['Status']]
    ax2.bar(wf_df['Window'], wf_df['Val_Test_Gap'], color=colors, alpha=0.7)
    ax2.axhline(y=0.05, color='orange', linestyle='--', label='Threshold (0.05)')
    ax2.set_xlabel('Window')
    ax2.set_ylabel('Val→Test Gap')
    ax2.set_title('Overfitting Check by Window')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # Plot 3: Metrics comparison
    ax3 = axes[1, 0]
    metrics = ['Test_Acc', 'Test_Prec', 'Test_Rec', 'Test_F1']
    metric_avgs = [wf_df[m].mean() for m in metrics]
    ax3.bar(['Accuracy', 'Precision', 'Recall', 'F1'], metric_avgs, color='steelblue', alpha=0.7)
    ax3.set_ylabel('Score')
    ax3.set_title('Average Performance Metrics')
    ax3.set_ylim(0, 1)
    ax3.grid(True, alpha=0.3, axis='y')

    # Plot 4: Performance stability
    ax4 = axes[1, 1]
    ax4.boxplot([wf_df['Val_AUC'], wf_df['Test_AUC']], labels=['Val AUC', 'Test AUC'])
    ax4.set_ylabel('AUC')
    ax4.set_title('Performance Distribution')
    ax4.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig(os.path.join(WF_PATH, 'walk_forward_analysis.png'), dpi=300, bbox_inches='tight')
    plt.close()

    logger.info(f"  ✓ Saved visualization: walk_forward_analysis.png")

# ============================================================================
# STEP 8: Save Comprehensive Results
# ============================================================================
logger.info("\n[8/8] Saving comprehensive walk-forward results...")

wf_comprehensive = {
    'validation_info': {
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'num_windows': len(wf_df) if len(wf_df) > 0 else 0,
        'strategy': 'expanding_window',
        'retrain_each_window': True
    },
    'window_results': wf_df.to_dict('records') if len(wf_df) > 0 else [],
    'overall_statistics': {
        'avg_test_auc': float(wf_df['Test_AUC'].mean()) if len(wf_df) > 0 else 0,
        'std_test_auc': float(wf_df['Test_AUC'].std()) if len(wf_df) > 0 else 0,
        'avg_val_test_gap': float(wf_df['Val_Test_Gap'].mean()) if len(wf_df) > 0 else 0,
        'overfit_windows': int((wf_df['Status'] == 'OVERFIT').sum()) if len(wf_df) > 0 else 0,
        'temporal_status': temporal_status if len(wf_df) > 0 else 'N/A'
    }
}

with open(os.path.join(WF_PATH, 'walk_forward_comprehensive.json'), 'w') as f:
    json.dump(wf_comprehensive, f, indent=2)

logger.info(f"  ✓ Saved to {WF_PATH}/")

# ============================================================================
# FINAL ASSESSMENT
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("WALK-FORWARD VALIDATION ASSESSMENT")
logger.info("=" * 70)

if len(wf_df) > 0:
    avg_auc = wf_df['Test_AUC'].mean()
    std_auc = wf_df['Test_AUC'].std()
    avg_gap = wf_df['Val_Test_Gap'].mean()

    if avg_auc > 0.60 and std_auc < 0.05 and avg_gap < 0.04:
        status = "EXCELLENT"
        assessment = "✅ Model is robust across time periods"
    elif avg_auc > 0.55 and std_auc < 0.08 and avg_gap < 0.06:
        status = "GOOD"
        assessment = "✅ Model shows reasonable stability"
    elif avg_auc > 0.52:
        status = "MODERATE"
        assessment = "⚠ Some instability detected"
    else:
        status = "WEAK"
        assessment = "⚠ Model not robust across time"

    logger.info(f"\n  Status: {status}")
    logger.info(f"  {assessment}")
    logger.info(f"\n  Key Metrics:")
    logger.info(f"    • Avg AUC: {avg_auc:.4f} ± {std_auc:.4f}")
    logger.info(f"    • Avg Val→Test gap: {avg_gap:.4f}")
    logger.info(f"    • Temporal: {temporal_status}")
else:
    status = "INCOMPLETE"
    logger.info(f"\n  Status: {status}")
    logger.info(f"  No walk-forward results available")

wf_comprehensive['status'] = status
wf_comprehensive['assessment'] = assessment if len(wf_df) > 0 else "N/A"

with open(os.path.join(WF_PATH, 'walk_forward_comprehensive.json'), 'w') as f:
    json.dump(wf_comprehensive, f, indent=2)

logger.info("\n" + "=" * 70)
logger.info("PHASE 3 PART 5 (WALK-FORWARD VALIDATION) - COMPLETE ✓")
logger.info("=" * 70)

logger.info(f"\n📋 Next Steps:")
logger.info(f"  1. ✓ Data prepared")
logger.info(f"  2. ✓ Global model trained")
logger.info(f"  3. ✓ Sector models trained")
logger.info(f"  4. ✓ Ensemble created")
logger.info(f"  5. ✓ Walk-forward validation (status: {status})")
logger.info(f"  6. ▶ Run Cell 14: SHAP Analysis")

logger.info(f"\n📂 Output: {WF_PATH}/")
logger.info("=" * 70)

ASTRO-FINANCE PROJECT - PHASE 3: WALK-FORWARD VALIDATION
Phase 3 Progress: Part 5 of 6

[1/8] Loading data and models...
  ✓ Full dataset: (134508, 948)
  ✓ Date range: 2000-01-03 00:00:00 to 2025-10-22 00:00:00
  ✓ Loaded LightGBM model

[2/8] Defining walk-forward time windows...
  Total time span: 9424 days
  Strategy: Expanding window (train on all past data)

  Created 5 walk-forward windows:
    Window 1: Train=21,530 → Test=22,659
      Train: up to 2004-04-21
      Test: 2004-04-21 to 2008-08-08
    Window 2: Train=44,189 → Test=22,571
      Train: up to 2008-08-08
      Test: 2008-08-08 to 2012-11-25
    Window 3: Train=66,760 → Test=22,530
      Train: up to 2012-11-25
      Test: 2012-11-25 to 2017-03-14
    Window 4: Train=89,290 → Test=22,583
      Train: up to 2017-03-14
      Test: 2017-03-14 to 2021-07-01
    Window 5: Train=111,873 → Test=22,576
      Train: up to 2021-07-01
      Test: 2021-07-01 to 2025-10-18

[3/8] Running walk-forward validation (retraining each wi

In [10]:
# Cell 14: SHAP Analysis (Phase 3 - Part 6 of 6)
# ================================================================
#
# PURPOSE: Model interpretability and feature importance
# - Understand what drives predictions
# - Identify most influential astrological features
# - Validate model is using sensible patterns
# - Detect potential data leakage or spurious correlations
#
# ================================================================

import logging
import sys

import os
import pandas as pd
import numpy as np
import json
import pickle
from datetime import datetime
from tabulate import tabulate
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configure logging with timestamps and levels
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

warnings.filterwarnings('ignore')

np.random.seed(42)

logger.info("=" * 70)
logger.info("ASTRO-FINANCE PROJECT - PHASE 3: SHAP ANALYSIS")
logger.info("Phase 3 Progress: Part 6 of 6 (FINAL)")
logger.info("=" * 70)

# ============================================================================
# STEP 1: Setup and Load Data
# ============================================================================
logger.info("\n[1/7] Loading data and models...")

BASE_PATH = '/content/drive/MyDrive/AstroFinanceProject'
MULTI_TICKER_PATH = os.path.join(BASE_PATH, 'prepared_data', 'multi_ticker')
MODEL_PATH = os.path.join(BASE_PATH, 'models')
LGBM_PATH = os.path.join(MODEL_PATH, 'lightgbm_improved')
SHAP_PATH = os.path.join(MODEL_PATH, 'shap_analysis')

os.makedirs(SHAP_PATH, exist_ok=True)

# Load test data (for SHAP analysis)
X_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join(MULTI_TICKER_PATH, 'y_test.parquet'))['target'].values

dates_test = X_test['date']
X_test = X_test.drop('date', axis=1)

logger.info(f"  ✓ Test data: {X_test.shape}")

# Load model
try:
    model = lgb.Booster(model_file=os.path.join(LGBM_PATH, 'lightgbm_model.txt'))
    logger.info(f"  ✓ Loaded LightGBM model")
except:
    logger.error(f"  ✗ Model not found. Please run Cell 10 first.")
    raise FileNotFoundError("LightGBM model required for SHAP analysis")

# Load feature importance
importance_df = pd.read_csv(os.path.join(LGBM_PATH, 'feature_importance_detailed.csv'))

# ============================================================================
# STEP 2: Sample Selection for SHAP (Computational Efficiency)
# ============================================================================
logger.info("\n[2/7] Selecting representative samples for SHAP analysis...")

# SHAP is computationally expensive - use subset
# Strategy: Sample from different prediction ranges
y_pred_proba = model.predict(X_test)

# Create bins: low, medium, high confidence predictions
bins = [0, 0.4, 0.6, 1.0]
bin_labels = ['low_conf', 'medium_conf', 'high_conf']
prediction_bins = pd.cut(y_pred_proba, bins=bins, labels=bin_labels)

# Sample from each bin
samples_per_bin = 100
sample_indices = []

for bin_label in bin_labels:
    bin_mask = prediction_bins == bin_label
    bin_indices = np.where(bin_mask)[0]

    if len(bin_indices) > 0:
        n_samples = min(samples_per_bin, len(bin_indices))
        sampled = np.random.choice(bin_indices, size=n_samples, replace=False)
        sample_indices.extend(sampled)

sample_indices = np.array(sample_indices)  # Convert to numpy array
X_shap = X_test.iloc[sample_indices]
y_shap = y_test[sample_indices]

# FIX: Use the array directly instead of iloc on Categorical
prediction_bins_sampled = prediction_bins[sample_indices]

logger.info(f"  ✓ Selected {len(X_shap)} samples for SHAP")
logger.info(f"    • Low confidence: {(prediction_bins_sampled == 'low_conf').sum()}")
logger.info(f"    • Medium confidence: {(prediction_bins_sampled == 'medium_conf').sum()}")
logger.info(f"    • High confidence: {(prediction_bins_sampled == 'high_conf').sum()}")

# ============================================================================
# STEP 3: Compute SHAP Values
# ============================================================================
logger.info("\n[3/7] Computing SHAP values...")
logger.info("  (This may take a few minutes...)")

# Create SHAP explainer
explainer = shap.TreeExplainer(model)

# Compute SHAP values
shap_values = explainer.shap_values(X_shap)

# For binary classification, SHAP returns values for positive class
if isinstance(shap_values, list):
    shap_values = shap_values[1]  # Get positive class SHAP values

logger.info(f"  ✓ SHAP values computed: {shap_values.shape}")

# ============================================================================
# STEP 4: Global Feature Importance (SHAP-based)
# ============================================================================
logger.info("\n[4/7] Analyzing global feature importance...")

# Calculate mean absolute SHAP value for each feature
shap_importance = np.abs(shap_values).mean(axis=0)

shap_importance_df = pd.DataFrame({
    'feature': X_shap.columns,
    'shap_importance': shap_importance
}).sort_values('shap_importance', ascending=False)

shap_importance_df['shap_importance_pct'] = shap_importance_df['shap_importance'] / shap_importance_df['shap_importance'].sum() * 100

# Categorize features
def categorize_feature(feat_name):
    if feat_name in ['ticker_id', 'sector_id', 'region_id']:
        return 'Categorical'
    elif any(x in feat_name for x in ['sun_', 'moon_', 'mercury_', 'venus_', 'mars_', 'jupiter_', 'saturn_']):
        return 'Planetary'
    elif any(x in feat_name for x in ['aspect_', 'conjunction', 'opposition', 'trine', 'square']):
        return 'Aspects'
    elif any(x in feat_name for x in ['moon_phase', 'mercury_retrograde', 'day_of_week', 'month']):
        return 'Temporal'
    elif any(x in feat_name for x in ['rsi', 'sma', 'bb_', 'atr', 'volume_ratio', 'returns_']):
        return 'Technical'
    else:
        return 'Other'

shap_importance_df['category'] = shap_importance_df['feature'].apply(categorize_feature)

logger.info(f"\n  Top 20 Features by SHAP Importance:")
for idx, row in shap_importance_df.head(20).iterrows():
    logger.info(f"    {row['feature']:35s}: {row['shap_importance_pct']:5.2f}% [{row['category']}]")

# Category-wise SHAP importance
category_shap = shap_importance_df.groupby('category')['shap_importance_pct'].sum().sort_values(ascending=False)
logger.info(f"\n  SHAP Importance by Category:")
for cat, pct in category_shap.items():
    logger.info(f"    • {cat:15s}: {pct:5.1f}%")

# Save
shap_importance_df.to_csv(os.path.join(SHAP_PATH, 'shap_feature_importance.csv'), index=False)

# ============================================================================
# STEP 5: Compare SHAP vs Gain Importance
# ============================================================================
logger.info("\n[5/7] Comparing SHAP importance with Gain importance...")

# Merge SHAP and Gain importance
comparison_df = shap_importance_df[['feature', 'shap_importance_pct', 'category']].merge(
    importance_df[['feature', 'importance_pct']],
    on='feature',
    how='left'
)

comparison_df.columns = ['feature', 'SHAP_%', 'category', 'Gain_%']
comparison_df['Gain_%'] = comparison_df['Gain_%'].fillna(0)

# Calculate agreement (correlation)
correlation = comparison_df[['SHAP_%', 'Gain_%']].corr().iloc[0, 1]
logger.info(f"\n  Correlation between SHAP and Gain: {correlation:.3f}")

if correlation > 0.7:
    agreement = "HIGH"
    logger.info(f"  ✓ {agreement} agreement - Both methods identify similar features")
elif correlation > 0.5:
    agreement = "MODERATE"
    logger.warning(f"  ⚠ {agreement} agreement - Some differences in feature rankings")
else:
    agreement = "LOW"
    logger.warning(f"  ⚠ {agreement} agreement - Methods disagree on important features")

# Top features that differ
comparison_df['rank_diff'] = abs(
    comparison_df['SHAP_%'].rank(ascending=False) -
    comparison_df['Gain_%'].rank(ascending=False)
)
disagreement_features = comparison_df.nlargest(10, 'rank_diff')

logger.info(f"\n  Top 10 Features with Largest Ranking Disagreement:")
for idx, row in disagreement_features.iterrows():
    logger.info(f"    {row['feature']:30s}: SHAP={row['SHAP_%']:5.2f}%, Gain={row['Gain_%']:5.2f}%")

comparison_df.to_csv(os.path.join(SHAP_PATH, 'importance_comparison.csv'), index=False)

# ============================================================================
# STEP 6: Visualizations
# ============================================================================
logger.info("\n[6/7] Creating SHAP visualizations...")

# === Plot 1: Summary Plot ===
logger.info("  Creating summary plot...")
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_shap, show=False, max_display=20)
plt.title('SHAP Summary Plot - Top 20 Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(os.path.join(SHAP_PATH, 'shap_summary_plot.png'), dpi=300, bbox_inches='tight')
plt.close()
logger.info(f"  ✓ Saved: shap_summary_plot.png")

# === Plot 2: Bar Plot (Mean Absolute SHAP) ===
logger.info("  Creating bar plot...")
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_shap, plot_type='bar', show=False, max_display=20)
plt.title('SHAP Feature Importance - Top 20', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(SHAP_PATH, 'shap_bar_plot.png'), dpi=300, bbox_inches='tight')
plt.close()
logger.info(f"  ✓ Saved: shap_bar_plot.png")

# === Plot 3: Category Comparison ===
logger.info("  Creating category comparison...")
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# SHAP by category
axes[0].barh(category_shap.index, category_shap.values, color='steelblue', alpha=0.7)
axes[0].set_xlabel('SHAP Importance (%)')
axes[0].set_title('SHAP Importance by Feature Category')
axes[0].grid(True, alpha=0.3, axis='x')

# Gain by category
category_gain = importance_df.groupby(importance_df['feature'].apply(categorize_feature))['importance_pct'].sum().sort_values(ascending=False)
axes[1].barh(category_gain.index, category_gain.values, color='coral', alpha=0.7)
axes[1].set_xlabel('Gain Importance (%)')
axes[1].set_title('Gain Importance by Feature Category')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(os.path.join(SHAP_PATH, 'category_comparison.png'), dpi=300, bbox_inches='tight')
plt.close()
logger.info(f"  ✓ Saved: category_comparison.png")

# === Plot 4: Dependence Plots for Top 3 Features ===
logger.info("  Creating dependence plots for top features...")
top_3_features = shap_importance_df.head(3)['feature'].tolist()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, feature in enumerate(top_3_features):
    feature_idx = X_shap.columns.get_loc(feature)
    shap.dependence_plot(feature_idx, shap_values, X_shap, ax=axes[idx], show=False)
    axes[idx].set_title(f'{feature}', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(SHAP_PATH, 'top3_dependence_plots.png'), dpi=300, bbox_inches='tight')
plt.close()
logger.info(f"  ✓ Saved: top3_dependence_plots.png")

# ============================================================================
# STEP 7: Save Comprehensive SHAP Results
# ============================================================================
logger.info("\n[7/7] Saving comprehensive SHAP analysis...")

shap_comprehensive = {
    'analysis_info': {
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'samples_analyzed': len(X_shap),
        'total_features': len(X_shap.columns),
        'shap_gain_correlation': float(correlation)
    },
    'top_20_features_shap': shap_importance_df.head(20).to_dict('records'),
    'category_importance_shap': category_shap.to_dict(),
    'category_importance_gain': category_gain.to_dict(),
    'importance_agreement': agreement,
    'key_insights': {
        'most_important_feature': shap_importance_df.iloc[0]['feature'],
        'most_important_category': category_shap.index[0],
        'astrological_importance': float(category_shap.get('Planetary', 0) + category_shap.get('Aspects', 0)),
        'technical_importance': float(category_shap.get('Technical', 0))
    }
}

with open(os.path.join(SHAP_PATH, 'shap_comprehensive.json'), 'w') as f:
    json.dump(shap_comprehensive, f, indent=2)

# Save SHAP values for future use
np.save(os.path.join(SHAP_PATH, 'shap_values.npy'), shap_values)
X_shap.to_parquet(os.path.join(SHAP_PATH, 'X_shap_samples.parquet'))

logger.info(f"  ✓ Saved all SHAP results to {SHAP_PATH}/")

# ============================================================================
# FINAL ASSESSMENT
# ============================================================================
logger.info("\n" + "=" * 70)
logger.info("SHAP ANALYSIS ASSESSMENT")
logger.info("=" * 70)

astro_importance = float(category_shap.get('Planetary', 0) + category_shap.get('Aspects', 0))
tech_importance = float(category_shap.get('Technical', 0))

logger.info(f"\n  Key Findings:")
logger.info(f"    • Most important feature: {shap_importance_df.iloc[0]['feature']}")
logger.info(f"    • Most important category: {category_shap.index[0]}")
logger.info(f"    • Astrological features: {astro_importance:.1f}% importance")
logger.info(f"    • Technical features: {tech_importance:.1f}% importance")
logger.info(f"    • SHAP-Gain correlation: {correlation:.3f} ({agreement} agreement)")

if astro_importance > 20:
    astro_status = "SIGNIFICANT"
    logger.info(f"\n  ✅ {astro_status} - Astrological features contribute meaningfully")
elif astro_importance > 10:
    astro_status = "MODERATE"
    logger.warning(f"\n  ⚠ {astro_status} - Astrological features have some influence")
else:
    astro_status = "MINIMAL"
    logger.warning(f"\n  ⚠ {astro_status} - Astrological features contribute little")

shap_comprehensive['astro_status'] = astro_status

with open(os.path.join(SHAP_PATH, 'shap_comprehensive.json'), 'w') as f:
    json.dump(shap_comprehensive, f, indent=2)

logger.info("\n" + "=" * 70)
logger.info("PHASE 3 PART 6 (SHAP ANALYSIS) - COMPLETE ✓")
logger.info("=" * 70)

logger.info(f"\n🎉 PHASE 3 COMPLETE!")
logger.info(f"\n📋 All Cells Completed:")
logger.info(f"  1. ✓ Data preparation")
logger.info(f"  2. ✓ Global model training")
logger.info(f"  3. ✓ Sector-specific models")
logger.info(f"  4. ✓ Ensemble methods")
logger.info(f"  5. ✓ Walk-forward validation")
logger.info(f"  6. ✓ SHAP analysis")

logger.info(f"\n📂 Outputs:")
logger.info(f"  • {LGBM_PATH}/")
logger.info(f"  • {SHAP_PATH}/")

logger.info(f"\n📊 Model Interpretability Status: {astro_status}")
logger.info("=" * 70)

ASTRO-FINANCE PROJECT - PHASE 3: SHAP ANALYSIS
Phase 3 Progress: Part 6 of 6 (FINAL)

[1/7] Loading data and models...
  ✓ Test data: (9491, 947)
  ✓ Loaded LightGBM model

[2/7] Selecting representative samples for SHAP analysis...
  ✓ Selected 200 samples for SHAP
    • Low confidence: 100
    • Medium confidence: 100
    • High confidence: 0

[3/7] Computing SHAP values...
  (This may take a few minutes...)
  ✓ SHAP values computed: (200, 947)

[4/7] Analyzing global feature importance...

  Top 20 Features by SHAP Importance:
    volatility_20d                     : 13.78% [Other]
    ticker_id                          :  8.54% [Categorical]
    returns_5d                         :  5.04% [Technical]
    jupiter_saturn_midpoint            :  2.79% [Planetary]
    sector_id                          :  2.52% [Categorical]
    volume_ratio                       :  2.50% [Technical]
    saturn_longitude                   :  2.09% [Planetary]
    day_of_month                       :  1.